# V-polyhedral disjunctive cuts plotting worksheet
1. Table 1: Summary statistics for percent gap closed by VPCs --- avg (%) and number of strict wins (best by at least `EPS`), including set of all instances and set of ≥ 10% gap closed instances
2. Table 2: Average percent gap closed by num disj terms
3. Table 3: Summary statistics for time to solve instances with branch-and-bound

We select instances that meet the following criteria:
1. Belong to MIPLIB, NEOS, or COR@L
2. IP optimal value is known
3. ≤ 5000 variables and 5000 constraints (in presolved instance)
4. The partial branch-and-bound tree with 64 leaves does not find an IP optimal solution
5. The disjunctive lower bound is strictly less than the maximum objective value on any leaf node

There are some instances for which we do not have data for all 6 partial tree sizes. We include these instances in most tables, except if we are showing how some statistic changes as the disjunction increases in size.

# Section 0: Set variables, import whatever is needed, and read in data

### Global variables

In [1]:
## Global variables
EPS = 1e-7

## Set up variables containing relevant directories
import os
repos_key = 'REPOS_DIR'
try:
    REPOS_DIR = os.environ[repos_key]
    print("REPOS_DIR set to \"%s\"." % REPOS_DIR)
except KeyError:
    print("*** ERROR: %s not found!" % repos_key)

VPC_DIR = REPOS_DIR + "/vpc/"
#RESULTS_DIR = VPC_DIR + "results/saved/"
RESULTS_DIR = VPC_DIR + "results/2023-06-25/"
DATA_DIR = VPC_DIR + "data/"

ONLY_PURE_BINARY = False
ONLY_MIXED_BINARY = False

REPOS_DIR set to "/Users/akazachkov/repos".


### Import data processing, plotting, and export packages and functions

In [2]:
## Import data processing, plotting, and export packages and functions
from IPython.display import display

import pandas as pd
pd.set_option("multi_sparse", True)

import numpy as np
import matplotlib.lines as mlines
from matrix2latex import matrix2latex

import matplotlib.pyplot as plt
scale=2
DPI = 200
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('axes.spines', **{'bottom':True, 'left':True, 'right':False, 'top':False})
plt.rc('axes', titlesize=12*scale)
plt.rc('axes', labelsize=8*scale)
plt.rc('xtick', labelsize=8*scale)
plt.rc('ytick', labelsize=8*scale)
plt.rc("legend", fontsize=8*scale)
plt.rc("figure", figsize=[6*scale,4*scale])
# plt.rc("figure", figsize=[6,4])
#plt.rc("figure", figsize=[3,2])
plt.rc("savefig", dpi=DPI)

### LaTeX helper functions

In [3]:
# LaTeX helper functions
import re

def tex_escape(text):
    """
        :param text: a plain text message
        :return: the message escaped to appear correctly in LaTeX
    """
    conv = {
        '&': r'\&',
        '%': r'\%',
        '$': r'\$',
        '#': r'\#',
        '_': r'\_',
        '{': r'\{',
        '}': r'\}',
        '~': r'\textasciitilde{}',
        '^': r'\^{}',
        '\\': r'\textbackslash{}',
        '<': r'\textless{}',
        '>': r'\textgreater{}',
        '≥': r'$\ge$'
    }
    regex = re.compile('|'.join(re.escape(str(key)) for key in sorted(conv.keys(), key = lambda item: - len(item))))
    return regex.sub(lambda match: conv[match.group()], text)


def remove_presolved_from_name(name:str) -> str:
    """Remove _presolved from instance names"""
    return name.removesuffix("_presolved")


def create_multirow_string(strval: str, num_rows: int = 2, alignment: str = 'c', extra_format: str = ""):
    """
    Wrap \p strval in a multirow environment for a table.
    """
    return \
        "{" + \
        "\\multirow[" + alignment + "]{"+ str(num_rows) + "}{*}{" + \
        (extra_format + "{" if extra_format != "" else "") + \
        str(strval) + \
        ("}" if extra_format != "" else "") + \
        "}" + "}"


def format_col_as_multirow(curr_series: pd.core.series.Series):
    start_val = ''
    start_row = -1
    end_row = -1
    for val in curr_series:
        end_row += 1
        is_last_row = end_row == len(curr_series)-1
        if val != start_val or is_last_row:
            num_rows = (end_row - start_row) + is_last_row
            if start_row >= 0 and num_rows > 1:
                multirow_string = create_multirow_string(str(start_val), num_rows = num_rows)
                curr_series[start_row] = multirow_string
                if is_last_row:
                    curr_series[end_row] = ""
            start_row = end_row
            start_val = val
        else:
            curr_series[end_row] = ""


# Some columns report both floats and ints
# This is a problem for siunitx that we need to fix explicitly
# We check for any int values in the table and apply a format to all of them
from math import floor, ceil


def is_val(val1: float, val2: float) -> bool:
    return abs(val1 - val2) < 1e-7


def is_int(val):
    """
    Checks whether given value should be treated as an int.

    Currently treats zero as a float always which is not ideal.
    """
    if isinstance(val, str) and val == '':
        return False
    try:
        floatval = float(val)
    except ValueError:
        # print("ValueError: ", val)
        return False
    # print("DEBUG:", val, ":", type(val))
    rounds_to_int = is_val(floatval, floor(floatval)) and is_val(floatval, ceil(floatval))
    is_zero = is_val(floatval, 0.0)
    # is_float_zero = (isinstance(val,str) and val.find('.') >= 0 and is_zero)
    return rounds_to_int and (not is_zero)


# def is_int_style(col : pd.core.series.Series):
#     # return ['background-color: green' if is_int(v) else '' for v in col]
#     return ['background-color: green' if is_int(v) else '' for v in col]


def int_format(val, num_digits = 3, add_phantom = False):
    if is_int(val):
        new_str = "{\\tablenum[table-format=" + str(num_digits) +  ".0]{" + str(val) + "}"
        new_str += "\\phantom{.00}" if add_phantom else ''
        new_str += "}"
        return new_str
    else:
        return val


def enclose_in_braces(val, val_to_match : str) -> str:
    if str(val)==str(val_to_match):
        return "{" + str(val) + "}"
    else:
        return str(val)


# The styler from pandas has some limitations, particularly no way to add \midrule at arbitrary places
def add_midrule(latex: str, index: int) -> str:
    """
    Adds a midrule either `index` lines after the start or -index lines before the last line of the table

    Args:
        latex: latex table
        index: index of horizontal line insertion (in lines)
    """
    lines = latex.splitlines()
    if index >= 0:
        lines.insert(index, r'\midrule')
    else:
        index_from_bottom = -index
        lines.insert(len(lines) - index_from_bottom - 2, r'\midrule')
    return '\n'.join(lines).replace('NaN', '')


# To add adjustbox, needs to be done after LaTeX string has been generated
def add_adjustbox_environment(latex: str) -> str:
    lines = latex.splitlines()
    start_env_ind = -1
    end_env_ind = -1
    curr_ind = -1
    for line in lines:
        curr_ind += 1
        if line.startswith(r"\begin{tabular}"):
            start_env_ind = curr_ind
        if line.startswith(r"\end{tabular}"):
            end_env_ind = curr_ind+1
    if (start_env_ind >= 0 and end_env_ind >= 0):
        lines.insert(start_env_ind, r'\begin{adjustbox}{width=1\textwidth}')
        lines.insert(end_env_ind+1, r'\end{adjustbox}')
    return '\n'.join(lines).replace('NaN', '')


def add_sisetup(latex: str, table_format = "2.2") -> str:
    latex = \
"""
{
\sisetup{
    table-alignment-mode = format,
    table-number-alignment = center,
    table-format = """ + \
    table_format + ",\n" + \
        "}\n" + \
        latex + \
        "\n}"
    return latex

### `initialize_df`: common way to process each data frame that we need

In [4]:
## Common way to process each data frame that we need
def initialize_df(filename):
    """
    Create a multilevel index df out of data from file `filename`.
    """
    df = pd.read_csv(filename, sep=',', index_col=False, skiprows=1)
    df.sort_values(by = ['INSTANCE','disj_terms'], inplace=True)
    df.set_index(['INSTANCE','disj_terms'], inplace=True)
    df.replace({"\'-inf\'": -np.inf, "\'inf\'": np.inf}, inplace=True)
    return df

### `df_ipopt`: Retrieve best known IP objective values

In [5]:
## Best known IP objective values
df_ipopt = pd.read_csv(DATA_DIR + "ip_obj.csv")
df_ipopt = df_ipopt.set_index(df_ipopt[df_ipopt.columns[0]])
df_ipopt.rename(columns = {'IP_OBJ' : 'IP OBJ'}, inplace=True) # for consistency with other dfs
df_ipopt = df_ipopt[~df_ipopt.index.duplicated()]
display(df_ipopt.head())
display(df_ipopt['IP OBJ']['bm23_presolved'])

,INSTANCE,IP OBJ,SET
INSTANCE,,,
22433,22433,21477,miplib2017
23588,23588,8090,miplib2017
10teams,10teams,924,miplib2017
50v-10,50v-10,3311.179984,miplib2017
a1c1s1,a1c1s1,11503.44413,miplib2017


'34'

### `df_preprocess`: Results from preprocessing instances

In [6]:
## Results from preprocessing instances
df_preprocess = pd.read_csv(RESULTS_DIR + "vpc-preprocess.csv", sep=',', index_col=False, skiprows=1)
df_preprocess = df_preprocess.set_index(df_preprocess[df_preprocess.columns[0]])
display(df_preprocess.head())
display(df_preprocess.loc['bm23','CLEANED LP OBJ'])

,INSTANCE,STRATEGY,ORIG LP OBJ,CLEANED LP OBJ,ORIG FIRST GUR NODES,CLEANED FIRST GUR NODES,ORIG BEST GUR NODES,CLEANED BEST GUR NODES,ORIG FIRST GUR TIME,CLEANED FIRST GUR TIME,...,vpc_version,cbc_version,clp_version,gurobi_version,cplex_version,ExitReason,end_time_string,time elapsed,instname,Unnamed: 137
INSTANCE,,,,,,,,,,,,,,,,,,,,,
22433,22433,536,21240.526171,21240.526171,9,12,9,12,0.485,0.262,...,#444d871,#600f7d1,#5de0d70,10.01,NaN,SUCCESS,Wed Jun 14 14:58:15 2023,0,22433,DONE
23588,23588,536,7649.866134,7649.866134,1612,654,1612,654,2.766,1.472,...,#444d871,#600f7d1,#5de0d70,10.01,NaN,SUCCESS,Wed Jun 14 14:58:19 2023,4,23588,DONE
10teams,10teams,536,917.000000,917.000000,1,1,1,1,1.454,0.937,...,#444d871,#600f7d1,#5de0d70,10.01,NaN,SUCCESS,Wed Jun 14 14:58:17 2023,2,10teams,DONE
2club200v15p5scn,2club200v15p5scn,536,-121.222222,-120.076923,54528,55547,54528,55547,7200.002,7200.004,...,#444d871,#600f7d1,#5de0d70,10.01,NaN,SUCCESS,Wed Jun 14 18:58:17 2023,14402,2club200v15p5scn,DONE
30_70_45_05_100,30_70_45_05_100,536,8.100000,8.100000,1,1,1,1,11.538,8.890,...,#444d871,#600f7d1,#5de0d70,10.01,NaN,SUCCESS,Wed Jun 14 14:59:08 2023,53,30_70_45_05_100,DONE


20.570921763235567

### `df_bb`: Results from generating VPCs for various number of disjunctive terms

In [7]:
## Results from generating VPCs for various number of disjunctive terms
df_bb = initialize_df(RESULTS_DIR + "vpc-bb.csv")
display(df_bb.head())

cutlimit  gomory  mode  partial_bb_strategy  \
INSTANCE          disj_terms                                                
10teams_presolved 2                 -1      -1     0                    4   
                  4                 -1      -1     0                    4   
                  8                 -1      -1     0                    4   
                  16                -1      -1     0                    4   
                  32                -1      -1     0                    4   

                              partial_bb_keep_pruned_nodes  \
INSTANCE          disj_terms                                 
10teams_presolved 2                                      0   
                  4                                      0   
                  8                                      0   
                  16                                     0   
                  32                                     0   

                              partial_bb_num_strong  preprocess  \
INSTANCE          disj_terms                                      
10teams_presolved 2                               5           0   
                  4                               5           0   
                  8                               5           0   
                  16                              5           0   
                  32                              5           0   

                              prlp_flip_beta  rounds  strengthen  ...  \
INSTANCE          disj_terms                                      ...   
10teams_presolved 2                        0       1           1  ...   
                  4                        0       1           1  ...   
                  8                        0       1           1  ...   
                  16                       0       1           1  ...   
                  32                       0       1           1  ...   

                              vpc_version  cbc_version  clp_version  \
INSTANCE          disj_terms                                          
10teams_presolved 2              #dc59789     #600f7d1     #5de0d70   
                  4              #dc59789     #600f7d1     #5de0d70   
                  8              #dc59789     #600f7d1     #5de0d70   
                  16             #dc59789     #600f7d1     #5de0d70   
                  32             #dc59789     #600f7d1     #5de0d70   

                              gurobi_version  cplex_version       ExitReason  \
INSTANCE          disj_terms                                                   
10teams_presolved 2                    10.01            NaN          SUCCESS   
                  4                    10.01            NaN       FAIL_LIMIT   
                  8                    10.01            NaN          SUCCESS   
                  16                   10.01            NaN       FAIL_LIMIT   
                  32                   10.01            NaN  PRLP_TIME_LIMIT   

                                       end_time_string  time elapsed  \
INSTANCE          disj_terms                                           
10teams_presolved 2           Tue Jun 27 10:25:01 2023            96   
                  4           Sun Jun 25 15:06:57 2023            94   
                  8           Fri Jun 30 13:35:01 2023          1621   
                  16          Sun Jun 25 18:23:17 2023          2275   
                  32          Tue Jun 27 19:57:19 2023           251   

                                       instname  Unnamed: 286  
INSTANCE          disj_terms                                   
10teams_presolved 2           10teams_presolved          DONE  
                  4           10teams_presolved          DONE  
                  8           10teams_presolved          DONE  
                  16          10teams_presolved          DONE  
                  32          10teams_presolved          DONE  

[5 rows x 285 columns]

### `df`: Append to `df_bb` results from running baseline solver 7 times

In [8]:
## Append results from running baseline solver 7 times
#df = df_bb.append(initialize_df(RESULTS_DIR + "vpc-bb0.csv")) # deprecated
df = pd.concat([df_bb, initialize_df(RESULTS_DIR + "vpc-bb0.csv")])
df.sort_values(by = ['INSTANCE','disj_terms'], inplace=True)

col_list = ["BEST DISJ OBJ", "WORST DISJ OBJ"]
for col in col_list:
    df[col] = pd.to_numeric(df[col])

df['NUM DISJ TERMS'] = df.index.get_level_values(1)

## Identify pure binary instances, which are those where 'CLEANED BINARY' column equals 'CLEANED COLS'
df['IS PURE BINARY'] = (df['BINARY'] == df['COLS'])

## Identify mixed binary instances, which are those where 'CLEANED GEN INT' column = 0
df['IS MIXED BINARY'] = (df['GEN INT'] == 0)

# col_list = ['NUM DISJ TERMS']
# for col in col_list:
#     df[col] = pd.to_numeric(df[col])

# start = 220
# end = start + 15
# print(df.columns[start:end])
# print(df.dtypes[start:end])

display(df.head())

cutlimit  gomory  mode  partial_bb_strategy  \
INSTANCE          disj_terms                                                
10teams_presolved 0                 -1       0     0                    4   
                  2                 -1      -1     0                    4   
                  4                 -1      -1     0                    4   
                  8                 -1      -1     0                    4   
                  16                -1      -1     0                    4   

                              partial_bb_keep_pruned_nodes  \
INSTANCE          disj_terms                                 
10teams_presolved 0                                      0   
                  2                                      0   
                  4                                      0   
                  8                                      0   
                  16                                     0   

                              partial_bb_num_strong  preprocess  \
INSTANCE          disj_terms                                      
10teams_presolved 0                               5           0   
                  2                               5           0   
                  4                               5           0   
                  8                               5           0   
                  16                              5           0   

                              prlp_flip_beta  rounds  strengthen  ...  \
INSTANCE          disj_terms                                      ...   
10teams_presolved 0                        0       1           1  ...   
                  2                        0       1           1  ...   
                  4                        0       1           1  ...   
                  8                        0       1           1  ...   
                  16                       0       1           1  ...   

                              gurobi_version  cplex_version      ExitReason  \
INSTANCE          disj_terms                                                  
10teams_presolved 0                    10.01            NaN  NO_DISJUNCTION   
                  2                    10.01            NaN         SUCCESS   
                  4                    10.01            NaN      FAIL_LIMIT   
                  8                    10.01            NaN         SUCCESS   
                  16                   10.01            NaN      FAIL_LIMIT   

                                       end_time_string  time elapsed  \
INSTANCE          disj_terms                                           
10teams_presolved 0           Thu Jun 15 13:08:52 2023            36   
                  2           Tue Jun 27 10:25:01 2023            96   
                  4           Sun Jun 25 15:06:57 2023            94   
                  8           Fri Jun 30 13:35:01 2023          1621   
                  16          Sun Jun 25 18:23:17 2023          2275   

                                       instname  Unnamed: 286  Unnamed: 280  \
INSTANCE          disj_terms                                                  
10teams_presolved 0           10teams_presolved           NaN          DONE   
                  2           10teams_presolved          DONE           NaN   
                  4           10teams_presolved          DONE           NaN   
                  8           10teams_presolved          DONE           NaN   
                  16          10teams_presolved          DONE           NaN   

                              IS PURE BINARY  IS MIXED BINARY  
INSTANCE          disj_terms                                   
10teams_presolved 0                     True             True  
                  2                     True             True  
                  4                     True             True  
                  8                     True             True  
                  16                    True             True  

[5 rows x 2

### Remove `stein*` instances (keep modified `stein*_nocard` instances)

In [9]:
# Remove unmodified stein instances from consideration
df.drop(index = ['stein09_presolved', 'stein15_presolved', 'stein27_presolved', 'stein45_presolved'], inplace=True)
df.index = df.index.remove_unused_levels()

### Fix mistake in code for one root pass containing wrong bound

In [10]:
prefix_list = ["FIRST", "AVG", "BEST"]
for prefix in prefix_list:
    inst_set = df[prefix + " REF+V ROOT_PASSES"] == 1
    tmp_df = df[inst_set]
    tmp_df = tmp_df[["LP OBJ",
                     prefix + " REF+V BOUND", 
                     prefix + " REF+V FIRST_CUT_PASS",
                     prefix + " REF+V LAST_CUT_PASS"]]
    tmp_df.tail(30)

    tmp_tmp_df = tmp_df["LP OBJ"] - tmp_df[prefix + " REF+V FIRST_CUT_PASS"]
    assert(tmp_tmp_df.max() < EPS)

    refcol = prefix + " REF+V BOUND"
    col = prefix + " REF+V FIRST_CUT_PASS"
    df.loc[inst_set,col] = tmp_df[refcol].values
    col = prefix + " REF+V LAST_CUT_PASS"
    df.loc[inst_set,col] = tmp_df[refcol].values


df.loc  ["misc02_presolved",
         [refcol,
          "FIRST REF+V FIRST_CUT_PASS",
          "FIRST REF+V LAST_CUT_PASS"
         ]
        ]

,FIRST REF+V BOUND,FIRST REF+V FIRST_CUT_PASS,FIRST REF+V LAST_CUT_PASS
disj_terms,,,
0,0.0,0.000000,0.000000
2,1690.0,1045.000000,1252.692308
4,1690.0,1052.692308,1250.235294
8,1690.0,1690.000000,1690.000000
16,1690.0,1690.000000,1690.000000
32,1690.0,1690.000000,1690.000000
64,1690.0,1690.000000,1690.000000


### `instances`: get unique instance list

In [11]:
# Get unique instance list
if (ONLY_PURE_BINARY):
    # Select only instances in df in which column ['IS PURE BINARY'] is True
    tmp_df = df[df['IS PURE BINARY'] == True]
    tmp_df.index = tmp_df.index.remove_unused_levels()
    instances = tmp_df.index.levels[0]
elif (ONLY_MIXED_BINARY):
    tmp_df = df[df['IS MIXED BINARY'] == True]
    tmp_df.index = tmp_df.index.remove_unused_levels()
    instances = tmp_df.index.levels[0]
else:
    instances = df.index.levels[0]

instances.set_names(names = 'Instance', inplace=True)

print("Number of selected instances: ", len(instances))

Number of selected instances:  431


### `df_rejection_reason`: Track why instances were not selected for our statistics

In [12]:
rejection_reasons = [
    'SELECTED_GAP', # *not* rejected for gap experiments; _must_ be first column
    'SELECTED_TIME', # *not* rejected for time experiments; _must_ be second column
    'SELECTED_6TREES', # *not* rejected for 6trees set for time experiments; _must_ be third column
    'NUM_WITH_OBJS', # number of attempts that successfully tried solving the PRLP
    'NUM_WITH_CUTS', # number of attempts that successfully yielded cuts
    'IP_OPT_UNKNOWN', # ip opt val must be known
    'TOO_MANY_ROWS_OR_COLS', # require max(nrows, ncols) ≤ 5K
    'OPTIMAL_SOLUTION_FOUND', # optimal solution should not be found by any of the partial trees
    'LP_OPT_IS_NOT_CUT', # check if lp opt < ip opt
    'DLB=DUB', # check if disj lb < disj ub
    'LP=DLB=DUB', # require either lp opt < disj lb or disj lb < disj ub
    'PRLP_INFEASIBLE', # require PRLP is feasible and solves within timelimit for at least one of the attempts
    'PRLP_TIME_LIMIT', # require PRLP solves within timelimit for at least one of the attempts
    'NO_CUTS', # there must be cuts from at least one of the partial b&b trees
    'NO_GAP', # require that ip opt != lp opt
    'GUR_TIMEOUT', # require Gur7 < 3600 (Gurobi is able to solve the instance to optimality within an hour either with or without using VPCs)'
    '<7_ATTEMPTS', # indicates not all partial trees were successfully run
]
df_rejection_reason = pd.DataFrame(index = instances, columns = rejection_reasons, dtype=bool)
df_rejection_reason.iloc[:,3:] = False # no rejection criteria at true

for col in ['OPTIMAL_SOLUTION_FOUND']:
    df_rejection_reason[col] = df_rejection_reason[col].astype(np.int64)
for col in ['NUM_WITH_OBJS', 'NUM_WITH_CUTS', 'LP_OPT_IS_NOT_CUT', 'DLB=DUB', 'LP=DLB=DUB', 'PRLP_INFEASIBLE', 'PRLP_TIME_LIMIT']:
    df_rejection_reason[col] = df_rejection_reason[col].astype(np.int8)
display(df_rejection_reason.head())

,SELECTED_GAP,SELECTED_TIME,SELECTED_6TREES,NUM_WITH_OBJS,NUM_WITH_CUTS,IP_OPT_UNKNOWN,TOO_MANY_ROWS_OR_COLS,OPTIMAL_SOLUTION_FOUND,LP_OPT_IS_NOT_CUT,DLB=DUB,LP=DLB=DUB,PRLP_INFEASIBLE,PRLP_TIME_LIMIT,NO_CUTS,NO_GAP,GUR_TIMEOUT,<7_ATTEMPTS
Instance,,,,,,,,,,,,,,,,,
10teams_presolved,True,True,True,0,0,False,False,0,0,0,0,0,0,False,False,False,False
22433_presolved,True,True,True,0,0,False,False,0,0,0,0,0,0,False,False,False,False
23588_presolved,True,True,True,0,0,False,False,0,0,0,0,0,0,False,False,False,False
30n20b8_presolved,True,True,True,0,0,False,False,0,0,0,0,0,0,False,False,False,False
50v-10_presolved,True,True,True,0,0,False,False,0,0,0,0,0,0,False,False,False,False


### `map_rejection_reason_to_number`: Reference paper's rejection criteria

In [13]:
# map_rejection_reason_to_number = {
#     'OPTIMAL_SOLUTION_FOUND':   '(3)',
#     'LP=DLB=DUB':               '(4a)',
#     'PRLP_INFEASIBLE':          '(4b)',
#     'PRLP_TIME_LIMIT':          '(4c)',
#     '<7_ATTEMPTS':              '(?)',
# }
map_rejection_reason_to_number = {
    'IP_OPT_UNKNOWN':           '\\ref{selection-criterion:ip-opt-known}',
    'NO_GAP':                   '\\ref{selection-criterion:ip-opt-known}',
    'TOO_MANY_ROWS_OR_COLS':    '\\ref{selection-criterion:max-instance-size}',
    'OPTIMAL_SOLUTION_FOUND':   '\\ref{selection-criterion:partial-tree-does-not-find-opt}',
    'LP=DLB=DUB':               '\\ref{selection-criterion:cuts-are-generated:not_lp=dlb=dub}',
    'PRLP_INFEASIBLE':          '\\ref{selection-criterion:cuts-are-generated:PRLP-primal-feasible}',
    'PRLP_TIME_LIMIT':          '\\ref{selection-criterion:cuts-are-generated:PRLP-time-limit}',
    'NO_CUTS':                  '\\ref{selection-criterion:cuts-are-generated:cuts-are-generated}',
    'GUR_TIMEOUT':              'G',
    '<7_ATTEMPTS':              '?',
}

### `df_status_by_depth`: Track success or failure reason by depth

In [14]:
sizes = [2, 4, 8, 16, 32, 64]
df_status_by_depth = pd.DataFrame(index = instances, columns = sizes, dtype=str)

DEFAULT_STATUS = map_rejection_reason_to_number['<7_ATTEMPTS']

df_status_by_depth[:] = DEFAULT_STATUS

display(df_status_by_depth.head())

,2,4,8,16,32,64
Instance,,,,,,
10teams_presolved,?,?,?,?,?,?
22433_presolved,?,?,?,?,?,?
23588_presolved,?,?,?,?,?,?
30n20b8_presolved,?,?,?,?,?,?
50v-10_presolved,?,?,?,?,?,?


### DEBUG

In [15]:
# col = "REF+V FIRST_CUT_PASS"
# tmp = df[col]
# display(tmp)

# for col in df.columns:
#     if str(col).endswith("FIRST_CUT_PASS"):
#         print("{}".format(col))

# inst = 'neos22_presolved'
# col = 'NUM DISJ TERMS'
# df.loc[inst][col]

# display(df.loc[('bppc4-08_presolved',2)]['LP OBJ'])
# display(df.loc[('bppc4-08_presolved',2)]['BEST DISJ OBJ'])
# display(df.loc[('bppc4-08_presolved',2)]['WORST DISJ OBJ'])
# display(df['BEST DISJ OBJ'])

# Section 1: Select instances

### `selected_gap_instances_dict` (original index, instance): Select instances for gap closed calculations

Criteria to filter gap closed instances:
* ip opt val is known
* lp opt < ip opt
* max(nrows, ncols) ≤ 5K
* optimal solution should not be found by any of the partial trees
* either lp opt < disj lb or disj lb < disj ub
* PRLP is feasible and solves within timelimit for at least one of the attempts

In [16]:
## Select instances for gap closed calculations
#
# Criteria to filter gap closed instances:
# * ip opt val is known
# * lp opt < ip opt
# * max(nrows, ncols) ≤ 5K
# * optimal solution should not be found by any of the partial trees
# * either lp opt < disj lb or disj lb < disj ub
# * PRLP is feasible and solves within timelimit for at least one of the attempts

# Constants
MAX_ROWS = 5000
MAX_COLS = MAX_ROWS
PRINT_SKIP_REASON = False

# Information to save
selected_gap_instances_dict = {} # dictionary of (original index, instance)
#selected_indices = []
num_gap_errors = 0

inst_set = instances
num_attempts = np.zeros(len(inst_set), dtype=int)

for i, inst in enumerate(inst_set):
    print("{}/{}".format(i+1,len(inst_set)), end='\r', flush=True)
    skip_instance = False
    curr_df = df.loc[inst]
    
    # Count number of times instance appears
    num_attempts[i] = len(curr_df)

    if num_attempts[i] < 7:
        print("*** ERROR: Instance {:d} -- {}: {:d} < 7 attempts.".format(i, inst, num_attempts[i]))
        skip_instance = True
        num_gap_errors += 1
        df_rejection_reason.loc[inst, '<7_ATTEMPTS'] = True

    # Check that LP opt < IP opt
    lp_obj = np.float64(df_preprocess.loc[remove_presolved_from_name(inst),'CLEANED LP OBJ'])
    ip_obj = np.float64(df_ipopt.loc[inst,'IP OBJ'])
    YES_GAP = abs(ip_obj - lp_obj) >= 1e-7
    if not YES_GAP:
        print("*** ERROR: Instance {:d} -- {}: not YES GAP (lp = {:.10f}; ip = {:.10f})".format(i, inst, lp_obj, ip_obj))
        skip_instance = True
        num_gap_errors += 1
        df_rejection_reason.loc[inst, 'NO_GAP'] = True
        
    # Check that ExitReason != OPTIMAL_SOLUTION_FOUND
    OPT_SOL_FOUND = False
    for curr_index, row in curr_df.iterrows():
        #print(i,j, curr_df['ExitReason'])
        curr_depth = int(curr_index)
        if curr_depth == 0:
            continue
        exitreason = row['ExitReason']
        if exitreason == 'OPTIMAL_SOLUTION_FOUND' and not OPT_SOL_FOUND:
            if PRINT_SKIP_REASON:
                print("Skipping instance {:d} -- {}: optimal IP solution found at depth {:d}.".format(
                    i, inst, curr_depth
                ))
            skip_instance = True
            OPT_SOL_FOUND = True
            df_rejection_reason.loc[inst, 'OPTIMAL_SOLUTION_FOUND'] = curr_depth
        if OPT_SOL_FOUND:
            df_status_by_depth.loc[inst, curr_depth] = map_rejection_reason_to_number['OPTIMAL_SOLUTION_FOUND']
        else:
            df_status_by_depth.loc[inst, curr_depth] = ''

    # Check that best and worst bound on leaf nodes is not same (likely cause of primal infeasible PRLP)
    num_successful_attempts = 0
    has_zero = False
    terms = curr_df.index
    for curr_index in terms:
        if curr_df['NUM DISJ TERMS'][curr_index] == 0:
            has_zero = True
            continue
            
        lp_obj = curr_df['LP OBJ'][curr_index]
        ip_obj = curr_df['IP OBJ'][curr_index]
        best_disj_obj = curr_df['BEST DISJ OBJ'][curr_index]
        worst_disj_obj = curr_df['WORST DISJ OBJ'][curr_index]
        num_frac = curr_df['NUM FRAC'][curr_index]
        num_obj_tried = curr_df['NUM OBJ'][curr_index]
        num_cuts = curr_df['NUM VPC'][curr_index] # can be > 0 even if num_obj_tried = 0, b/c of OPTIMAL_SOLUTION_FOUND exit reason
        exitreason = curr_df['ExitReason'][curr_index]

        YES_GAP = abs(ip_obj - lp_obj) >= 1e-7
        LP_OPT_IS_CUT = (num_frac > 0) and YES_GAP and abs(lp_obj - worst_disj_obj) >= 1e-7
        DLB_NE_DUB = (num_frac > 0) and abs(best_disj_obj - worst_disj_obj) >= 1e-7
        df_rejection_reason.loc[inst, 'NO_GAP'] += (not YES_GAP)
        df_rejection_reason.loc[inst, 'LP_OPT_IS_NOT_CUT'] += (not LP_OPT_IS_CUT)
        df_rejection_reason.loc[inst, 'DLB=DUB'] += (not DLB_NE_DUB)
        df_rejection_reason.loc[inst, 'PRLP_INFEASIBLE'] += (exitreason == 'PRLP_INFEASIBLE')
        df_rejection_reason.loc[inst, 'PRLP_TIME_LIMIT'] += (exitreason == 'PRLP_TIME_LIMIT')
        # if not DLB_NE_DUB and num_obj_tried > 0:
        #     raise ValueError(
        #         "*** ERROR: Instance {:d} -- {}: at depth {:d}, num obj tried = {:d} (num cuts = {:d}) but lp opj {:.10f}, best_disj_obj {:.10f} = worst_disj_obj {:.10f} with exit reason {}".format(
        #             i, inst, curr_index, num_obj_tried, num_cuts, lp_obj, best_disj_obj, worst_disj_obj, curr_df['ExitReason'][curr_index]
        #         )
        #     )
        if LP_OPT_IS_CUT or DLB_NE_DUB:
            if (num_obj_tried == 0) and (exitreason not in ['PRLP_TIME_LIMIT','PRLP_INFEASIBLE','OPTIMAL_SOLUTION_FOUND','TIME_LIMIT']):
                # We should be trying objectives at this point, unless the initial PRLP timed out or was infeasible or an optimal solution was found
                raise ValueError(
                    "*** ERROR: Instance {:d} -- {}: at depth {:d}, num obj tried = 0 but lp opj {:.10f} < best_disj_obj {:.10f} < worst_disj_obj {:.10f} with exit reason {}".format(
                        i, inst, curr_index, lp_obj, best_disj_obj, worst_disj_obj, curr_df['ExitReason'][curr_index]
                    )
                )
            if num_obj_tried > 0:
                df_rejection_reason.loc[inst, 'NUM_WITH_OBJS'] += 1
                if num_cuts > 0:
                    num_successful_attempts += 1
                    df_rejection_reason.loc[inst, 'NUM_WITH_CUTS'] += 1
                else:
                    df_status_by_depth.loc[inst, curr_index] = map_rejection_reason_to_number['NO_CUTS']
            elif exitreason == 'PRLP_INFEASIBLE':
                df_status_by_depth.loc[inst, curr_index] = map_rejection_reason_to_number[exitreason]
            elif exitreason == 'PRLP_TIME_LIMIT':
                df_status_by_depth.loc[inst, curr_index] = map_rejection_reason_to_number[exitreason]
        else:
            # check that num obj tried is 0
            if (num_obj_tried > 0):
                raise ValueError(
                    "*** ERROR: Instance {:d} -- {}: at depth {:d}, num obj tried = {:d} > 0 but best_disj_obj {:f} = worst_disj_obj {:f}".format(
                        i, inst, curr_index, num_obj_tried, best_disj_obj, worst_disj_obj
                    )
                )
            df_rejection_reason.loc[inst, 'LP=DLB=DUB'] += 1
            df_status_by_depth.loc[inst, int(curr_index)] = map_rejection_reason_to_number['LP=DLB=DUB']

    if not has_zero:
        raise ValueError(
            "*** ERROR: Instance {:d} -- {}: has no bb0 entry.".format(
                i, inst, curr_index
            )
        )        
    
    if num_successful_attempts == 0 and not skip_instance:
        if PRINT_SKIP_REASON:
            print("Skipping instance {:d} -- {}: best and worst bound on leaf nodes coincide for all trees, no objectives ever tried, or no objectives successfully produced cuts.".format(
                i, inst, num_attempts[i]))
        skip_instance = True
        exitreason = 'NO_CUTS'
        df_rejection_reason.loc[inst, exitreason] = True
    else:        
        # Ensure IP objective value is known
        ip_obj = curr_df['IP OBJ'][curr_df.index[0]]
        if not isinstance(ip_obj,float):
            if PRINT_SKIP_REASON:
                print(
                    "Skipping instance {:d} -- {}: IP objective value ({}) is not detected to be a float value.".format(
                    i, inst, ip_obj))
            skip_instance = True
            df_rejection_reason.loc[inst, 'IP_OPT_UNKNOWN'] = True
            
        # Ensure nrows and ncols is not too many
        nrows = curr_df['ROWS'][curr_df.index[0]]
        ncols = curr_df['COLS'][curr_df.index[0]]
        if (nrows > MAX_ROWS) or (ncols > MAX_COLS):
            if PRINT_SKIP_REASON:
                print("Skipping instance {:d} -- {}: nrows = {:d} > {:d} or ncols = {:d} > {:d}.".format(
                        i, inst, nrows, ncols, MAX_ROWS, MAX_COLS))
            skip_instance = True
            df_rejection_reason.loc[inst, 'TOO_MANY_ROWS_OR_COLS'] = True
    
    if not skip_instance:
        #selected_gap_instances_dict[len(selected_gap_instances_dict)] = inst
        selected_gap_instances_dict[inst] = i
    else:
        df_rejection_reason.loc[inst, 'SELECTED_GAP'] = False

num_selected_gap_instances = len(selected_gap_instances_dict)
print("Total number of errors: {}".format(num_gap_errors))
print("Total number of selected instances for gap closed reporting: {}/{:d}".format(num_selected_gap_instances,len(instances)))

*** ERROR: Instance 207 -- neos-3048764-nadi_presolved: 1 < 7 attempts.
*** ERROR: Instance 211 -- neos-3214367-sovi_presolved: 4 < 7 attempts.
*** ERROR: Instance 226 -- neos-3661949-lesse_presolved: 1 < 7 attempts.
*** ERROR: Instance 228 -- neos-3734794-moppy_presolved: 1 < 7 attempts.
*** ERROR: Instance 251 -- neos-530627_presolved: 5 < 7 attempts.
Total number of errors: 5
Total number of selected instances for gap closed reporting: 328/431


### `selected_time_instances_dict` and `all6_instances_dict` (original index, instance): Select instances for time tables

Criteria to filter instances for reporting time:
* ip opt val is known
* lp opt < ip opt
* max(nrows, ncols) ≤ 5K
* optimal solution should not be found by any of the partial trees
* either lp opt < disj lb or disj lb < disj ub
* PRLP is feasible and solves within timelimit for at least one of the attempts
* Gur7 < 3600 (Gurobi is able to solve the instance to optimality within an hour either with or without using VPCs)

6 trees set
* all six partial tree sizes produced VPCs

In [17]:
## Select instances for time tables
#
# Criteria to filter instances for reporting time:
# * ip opt val is known
# * lp opt < ip opt 
# * max(nrows, ncols) ≤ 5K
# * optimal solution should not be found by any of the partial trees
# * either lp opt < disj lb or disj lb < disj ub
# * PRLP is feasible and solves within timelimit for at least one of the attempts
# * Gur7 < 3600 (Gurobi is able to solve the instance to optimality within an hour either with or without using VPCs)
#
# 6 trees set
# * all six partial tree sizes produced VPCs

# Constants
MAX_TIME = 3600
PRINT_SKIP_REASON = False

# Information to save
selected_time_instances_dict = {}   # dictionary of (original index, instance)
all6_instances_dict = {}            # dictionary of (original index, instance)
skipped_instances_dict = {}         # dictionary of (original index, instance)
error_instances_dict = {}           # dictionary of (original index, instance)

num_timeouts = 0
num_time_errors = 0

inst_set = list(selected_gap_instances_dict.keys())
# inst_set = ['lotsize_presolved']
for i, inst in enumerate(inst_set):
    print("{}/{}".format(i+1,len(inst_set)), end='\r', flush=True)
    skip_instance = False
    curr_df = df.loc[inst]

    # Check Gur < 3600 (Gurobi is able to solve the instance to optimality within an hour without using VPCs)
    col = 'BEST REF TIME'
    mintime_gur = float(curr_df.loc[0,col].min())
    
    # Check Gur7 < 3600 (Gurobi is able to solve the instance to optimality within an hour either with or without using VPCs)
    col = 'BEST REF+V TIME'
    mintime_gur7 = float(curr_df.loc[2:64,col].min())

    mintime = min(mintime_gur, mintime_gur7)
    if mintime > MAX_TIME - EPS:
        if PRINT_SKIP_REASON:
            print("{:d}: Skipping instance {:d} -- {}: Gurobi's best time (with or without VPCs) is {:.7f} >= {:.7f}.".format(
                    len(skipped_instances_dict), i, inst, mintime, MAX_TIME-EPS
                ))
        skip_instance = True
        skipped_instances_dict[inst] = i
        num_timeouts += 1
        df_rejection_reason.loc[inst, 'GUR_TIMEOUT'] += 1
        # df_status_by_depth.loc[inst, int(curr_index)] = map_rejection_reason_to_number['GUR_TIMEOUT']

    # Check how many times VPCs were successfully generated
    num_successful_attempts = 0
    has_zero = False
    for curr_index, row in curr_df.iterrows():
        if row['NUM DISJ TERMS'] == 0:
            has_zero = True
            continue

        num_vpc = float(row['NUM VPC'])
        num_successful_attempts += (num_vpc > 0)

        if df_status_by_depth.loc[inst, int(curr_index)] == DEFAULT_STATUS:
            curr_time = float(curr_df.loc[curr_index,col])
            if curr_time > MAX_TIME - EPS:
                df_status_by_depth.loc[inst, int(curr_index)] = map_rejection_reason_to_number['GUR_TIMEOUT']

    if not has_zero:
        raise ValueError(
            "*** ERROR: Instance {:d} -- {}: has no bb0 entry.".format(
                i, inst, curr_index
            )
        )        
    
    # if num_successful_attempts == 0 and not skip_instance:
    #     if PRINT_SKIP_REASON:
    #         print("Skipping instance {:d} -- {}: no VPCs generated successfully for any number of terms.".format(i, inst, num_attempts[i]))
    #     skip_instance = True
    #     skipped_instances_dict[inst] = i

    if not skip_instance:
        if num_successful_attempts == 6:
            all6_instances_dict[inst] = i
        #selected_time_instances_dict[len(selected_time_instances_dict)] = inst
        selected_time_instances_dict[inst] = i

num_selected_time_instances = len(selected_time_instances_dict)
num_all6_instances = len(all6_instances_dict)
print("Total number of errors: {}".format(num_time_errors))
print("Total number of timeouts: {}".format(num_timeouts))
print("Total number of instances for time reporting: {}".format(num_selected_time_instances))
print("Total number of \"6 trees\" instances: {}".format(num_all6_instances))

Total number of errors: 0
Total number of timeouts: 28
Total number of instances for time reporting: 300
Total number of "6 trees" instances: 196


#### DEBUG (check which instances were selected but do not have all six runs)

In [18]:
## DEBUG (check which instances were selected but do not have all six runs)
not_all_6 = [key for key in selected_time_instances_dict.keys() if key not in all6_instances_dict.keys()]
not_all_6

['10teams_presolved',
 '30n20b8_presolved',
 'a1c1s1_presolved',
 'aligninq_presolved',
 'app3_presolved',
 'berlin_5_8_0_presolved',
 'bg512142_presolved',
 'bnatt400_presolved',
 'bppc8-02_presolved',
 'cod105_presolved',
 'cvs08r139-94_presolved',
 'cvs16r106-72_presolved',
 'cvs16r128-89_presolved',
 'cvs16r70-62_presolved',
 'cvs16r89-60_presolved',
 'danoint_presolved',
 'dg012142_presolved',
 'eilB101_presolved',
 'eild76_presolved',
 'f2gap801600_presolved',
 'go19_presolved',
 'graph20-20-1rand_presolved',
 'graphdraw-domain_presolved',
 'hgms-det_presolved',
 'ic97_potential_presolved',
 'ic97_tension_presolved',
 'icir97_tension_presolved',
 'lrn_presolved',
 'milo-v12-6-r2-40-1_presolved',
 'mkc1_presolved',
 'n2seq36f_presolved',
 'n4-3_presolved',
 'n6-3_presolved',
 'n7-3_presolved',
 'neos-1058477_presolved',
 'neos-1215259_presolved',
 'neos-1330346_presolved',
 'neos-1396125_presolved',
 'neos-1413153_presolved',
 'neos-1415183_presolved',
 'neos-1480121_presolved',
 

# Section 2: Gap closed tables

### Prepare gap short/long column names

In [19]:
col_gmic            = 'GMIC % GAP CLOSED'
col_best_disj       = 'BEST DISJ % GAP CLOSED'
col_vpc             = 'VPC % GAP CLOSED'
col_vpc_gmic        = 'VPC+GMIC % GAP CLOSED'
col_max_gmic_vpc    = 'MAX(GMIC,VPC) % GAP CLOSED'
col_first_ref_first = 'FIRST REF FIRST_CUT_PASS % GAP CLOSED'
col_first_ref       = 'BEST REF FIRST_CUT_PASS % GAP CLOSED'
col_first_ref_v     = 'REF+V FIRST_CUT_PASS % GAP CLOSED'
col_last_ref_first  = 'FIRST REF LAST_CUT_PASS % GAP CLOSED'
col_last_ref        = 'BEST REF LAST_CUT_PASS % GAP CLOSED'
col_last_ref_v      = 'REF+V LAST_CUT_PASS % GAP CLOSED'
col_num_vpcs        = 'NUM VPC'
col_num_gmic        = 'NUM GMIC'
gap_cols = [
    col_gmic,
    col_best_disj,
    col_vpc,
    col_vpc_gmic,
    col_max_gmic_vpc,
    col_first_ref_first,
    col_first_ref,
    col_first_ref_v,
    col_last_ref_first,
    col_last_ref,
    col_last_ref_v,
]

map_short_to_cols_gap = {
    'G'      : col_gmic,
    'DB'     : col_best_disj,
    'V'      : col_vpc,
    'V+G'    : col_vpc_gmic,
    'max(G,V)': col_max_gmic_vpc,
    'GurF'   : col_first_ref,
    'V+GurF' : col_first_ref_v,
    'GurL'   : col_last_ref,
    'V+GurL' : col_last_ref_v,
}
map_cols_to_short_gap = {v: k for k, v in map_short_to_cols_gap.items()}

gap_cols_short = list(map_short_to_cols_gap.keys())

### `calc_gap_closed` function

In [20]:
## Calculate gap closed for GMICs, Gurobi, and VPCs
def calc_gap_closed(gap_df, col):
    return np.where(
        gap_df[col] > EPS, # condition
        100. * (gap_df[col] - gap_df["LP OBJ"]) / (gap_df["IP OBJ"] - gap_df["LP OBJ"]), # if condition is true
        0.0 # if condition is false
    )


def calc_gap_closed2(gap_df, col):
    conditions = gap_df[col] > EPS & np.isfinite(gap_df[col])
    choices = 100. * (gap_df[col] - gap_df["LP OBJ"]) / (gap_df["IP OBJ"] - gap_df["LP OBJ"])
    return np.select(conditions, choices, default=0.0)



### `gap_df`: Calculate gap closed for GMICs, Gurobi, and VPCs

In [21]:
# Create subset of dataframe relevant to gap closed
gap_df = df.loc[:, 
                [
                    'NUM DISJ TERMS',
                    'ROWS',
                    'COLS',
                    'LP OBJ',
                    'BEST DISJ OBJ',
                    'WORST DISJ OBJ',
                    'IP OBJ',
                    'GMIC OBJ',
                    'VPC OBJ',
                    'VPC+GMIC OBJ',
                    'FIRST REF FIRST_CUT_PASS',
                    'BEST REF FIRST_CUT_PASS',
                    'FIRST REF+V FIRST_CUT_PASS',
                    'FIRST REF LAST_CUT_PASS',
                    'BEST REF LAST_CUT_PASS',
                    'FIRST REF+V LAST_CUT_PASS',
                    'NUM GMIC',
                    'NUM VPC',
                    'NUM OBJ',
                    'ExitReason']
               ]

# Calculate some missing % gap closed columns
# gap closed = 100 * (post_cut_opt_val - lp_opt_val) / (ip_opt_val - lp_opt_val)
cut_type = "GMIC"
col = cut_type + " OBJ"
gap_df[cut_type + " % GAP CLOSED"] = calc_gap_closed(gap_df, col)

cut_type = "BEST DISJ"
col = cut_type + " OBJ"
gap_df[cut_type + " % GAP CLOSED"] = calc_gap_closed(gap_df, col)

cut_type = "VPC"
col = cut_type + " OBJ"
gap_df[cut_type + " % GAP CLOSED"] = calc_gap_closed(gap_df, col)

cut_type = "VPC+GMIC"
col = cut_type + " OBJ"
gap_df[cut_type + " % GAP CLOSED"] = calc_gap_closed(gap_df, col)

# Add max(G,V) column
gap_df["MAX(GMIC,VPC) % GAP CLOSED"] = np.maximum(gap_df["GMIC % GAP CLOSED"], gap_df["VPC % GAP CLOSED"])

# Compare against Gurobi
col = "REF FIRST_CUT_PASS"
gap_df["FIRST " + col + " % GAP CLOSED"] = calc_gap_closed(gap_df, "FIRST " + col)
gap_df["BEST " + col + " % GAP CLOSED"] = calc_gap_closed(gap_df, "BEST " + col)
col = "REF+V FIRST_CUT_PASS"
gap_df[col + " % GAP CLOSED"] = calc_gap_closed(gap_df, "FIRST " + col)
col = "REF LAST_CUT_PASS"
gap_df["FIRST " + col + " % GAP CLOSED"] = calc_gap_closed(gap_df, "FIRST " + col)
gap_df["BEST " + col + " % GAP CLOSED"] = calc_gap_closed(gap_df, "BEST " + col)
col = "REF+V LAST_CUT_PASS"
gap_df[col + " % GAP CLOSED"] = calc_gap_closed(gap_df, "FIRST " + col)

display(gap_df.loc['bm23_presolved'])
display(gap_df.loc[("bm23_presolved",2)])

,NUM DISJ TERMS,ROWS,COLS,LP OBJ,BEST DISJ OBJ,WORST DISJ OBJ,IP OBJ,GMIC OBJ,VPC OBJ,VPC+GMIC OBJ,...,BEST DISJ % GAP CLOSED,VPC % GAP CLOSED,VPC+GMIC % GAP CLOSED,"MAX(GMIC,VPC) % GAP CLOSED",FIRST REF FIRST_CUT_PASS % GAP CLOSED,BEST REF FIRST_CUT_PASS % GAP CLOSED,REF+V FIRST_CUT_PASS % GAP CLOSED,FIRST REF LAST_CUT_PASS % GAP CLOSED,BEST REF LAST_CUT_PASS % GAP CLOSED,REF+V LAST_CUT_PASS % GAP CLOSED
disj_terms,,,,,,,,,,,,,,,,,,,,,
0,0,20,27,20.570922,-inf,-inf,34.0,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,20.867452,22.152648,0.000000,37.49489,41.913096,0.000000
2,2,20,27,20.570922,21.483725,27.267238,34.0,22.828302,21.483725,23.184302,...,6.797215,6.797215,19.460606,16.809643,20.867452,22.152648,21.009686,37.49489,41.913096,39.732568
4,4,20,27,20.570922,22.537880,29.935573,34.0,22.828302,22.537880,23.250549,...,14.647006,14.647006,19.953919,16.809643,20.867452,22.152648,21.731716,37.49489,41.913096,42.053122
8,8,20,27,20.570922,22.936206,35.613171,34.0,22.828302,22.936206,23.250033,...,17.613150,17.613150,19.950078,17.613150,20.867452,22.152648,21.920039,37.49489,41.913096,43.499387
16,16,20,27,20.570922,25.881188,35.613171,34.0,22.828302,25.800104,25.800854,...,39.543044,38.939251,38.944835,38.939251,20.867452,22.152648,38.980155,37.49489,41.913096,48.176815
32,32,20,27,20.570922,28.161871,44.566830,34.0,22.828302,28.161871,28.161871,...,56.526208,56.526208,56.526208,56.526208,20.867452,22.152648,57.046658,37.49489,41.913096,61.787616
64,64,20,27,20.570922,30.168224,56.839578,34.0,22.828302,29.923215,29.923215,...,71.466577,69.642109,69.642109,69.642109,20.867452,22.152648,66.200094,37.49489,41.913096,71.190662


NUM DISJ TERMS                                   2
ROWS                                            20
COLS                                            27
LP OBJ                                   20.570922
BEST DISJ OBJ                            21.483725
WORST DISJ OBJ                           27.267238
IP OBJ                                        34.0
GMIC OBJ                                 22.828302
VPC OBJ                                  21.483725
VPC+GMIC OBJ                             23.184302
FIRST REF FIRST_CUT_PASS                 23.373228
BEST REF FIRST_CUT_PASS                  23.545818
FIRST REF+V FIRST_CUT_PASS               23.392329
FIRST REF LAST_CUT_PASS                   25.60614
BEST REF LAST_CUT_PASS                   26.199464
FIRST REF+V LAST_CUT_PASS                25.906639
NUM GMIC                                         6
NUM VPC                                          6
NUM OBJ                                          8
ExitReason                     

### `selected_gap_df`: Gap closed for selected instances, adding 0-row that has best for `V+` cols

In [22]:
## `selected_gap_df`: Gap closed for selected instances, adding 0-row that has best for `V+` cols
## Show the instances that have been selected (and their original index)
## and then set the selected_gap_df as the selected instances from gap_df
## We also set the '0' row to contain the best result for each method
## (including the option of not using VPCs at all)
## and we replace any runs with no VPCs with the values obtained without them
selected_gap_df = gap_df.loc[selected_gap_instances_dict.keys()]

# From https://pandas.pydata.org/docs/user_guide/advanced.html#defined-levels
# "The MultiIndex keeps all the defined levels of an index, even if they are not actually used.
# When slicing an index, you may notice this."
# Even without using remove_unused_levels, index was correct with selected_gap_df.index.get_level_values(0).unique()
selected_gap_df.index = selected_gap_df.index.remove_unused_levels()

#display(selected_gap_df.index.difference(gap_df.index))
#selected_gap_df.drop(['22433_presolved'])

# # Check what the selected_gap_df contains for bm23
# inst = "bm23_presolved"
# display(selected_gap_df.loc[inst])

#inst = "10teams_presolved"
# inst = '22433_presolved'
# curr_df = selected_gap_df.loc[inst]
# display(curr_df)
# # for i in curr_df.index:
# #     display(curr_df.loc[i])

#display(selected_gap_df.index.get_level_values(0).unique())

# Do we update the value of the "best" in each column when no VPCs are generated for a run and we use the "no-VPCs" data?
# This may cause the stats in the "best" row to improve
# For example, we replace V+GurF with GurF when no VPCs are generated, since that is what would occur without VPCs
# But if GurF is better than any V+GurF when VPCs are produced, then the average in the max-row is inflated
SHOULD_UPDATE_MAX_WHEN_NO_VPCS = False

# inst_set = selected_gap_df.index.get_level_values(0).unique()
inst_set = selected_gap_df.index.levels[0]
num_inst = len(inst_set)
for curr_inst_ind, inst in enumerate(inst_set):
    print("{}/{}".format(curr_inst_ind+1,num_inst), end='\r', flush=True)
    curr_df = selected_gap_df.loc[inst].copy() # copy needed to not throw SettingWithCopyWarning

    # Set 0-row to have max values across all rows for this instance
    max_vals = curr_df[gap_cols].max()
    selected_gap_df.loc[(inst,0),gap_cols] = max_vals

    for ind in curr_df.index:
        if ind == 0:
            continue

        # Propogate GurF and GurL down
        sel_gap = [col_first_ref_first, col_first_ref, col_last_ref_first, col_last_ref]
        selected_gap_df.loc[(inst,ind),sel_gap] = curr_df.loc[0,sel_gap]

        # If no VPCs produced, the values for V+GurF and V+GurL have not been provided
        # We replace these by GurF and GurL
        # Currently disabled: update max for that column too (if disabled, we instead keep max as the value among those that generated VPCs)
        num_vpc = curr_df.loc[ind,col_num_vpcs]
        if num_vpc == 0:
            # print("Zero cuts for inst {} at depth {:d}".format(inst, ind))
            ref_gap = [col_first_ref_first, col_last_ref_first] # this is where we pull info from
            refinds = [gap_cols.index(colname) for colname in ref_gap] 
            sel_gap = [col_first_ref_v, col_last_ref_v] # this is where we put the info
            selected_gap_df.loc[(inst,ind),sel_gap] = curr_df.loc[0,ref_gap].to_numpy()

            if SHOULD_UPDATE_MAX_WHEN_NO_VPCS:
                for i in refinds:
                    if curr_df.loc[0,gap_cols[i]] > selected_gap_df.loc[(inst,0),gap_cols[i+1]]:
                        # if curr_df.loc[0,gap_cols[i]] > 0:
                            # print("DEBUG: Updating {} for inst {} from {:f} to {:f}".format(
                            #     gap_cols[i+1], 
                            #     inst, 
                            #     selected_gap_df.loc[(inst,0),gap_cols[i+1]], 
                            #     curr_df.loc[0,gap_cols[i]]))
                        selected_gap_df.loc[(inst,0),gap_cols[i+1]] = curr_df.loc[0,gap_cols[i]]

display(selected_gap_df.head(21).loc[:,[col_num_vpcs]+gap_cols])

NUM VPC  GMIC % GAP CLOSED  \
INSTANCE          disj_terms                               
10teams_presolved 0                 0         100.000000   
                  2                77         100.000000   
                  4                 1         100.000000   
                  8                51         100.000000   
                  16                5         100.000000   
                  32                0         100.000000   
                  64                0         100.000000   
23588_presolved   0                 0           5.772831   
                  2                11           5.772831   
                  4                75           5.772831   
                  8                75           5.772831   
                  16               75           5.772831   
                  32               75           5.772831   
                  64               75           5.772831   
30n20b8_presolved 0                 0          11.099235   
                  2               190          11.099235   
                  4               190          11.099235   
                  8                 0          11.099235   
                  16                0          11.099235   
                  32                0          11.099235   
                  64                0          11.099235   

                              BEST DISJ % GAP CLOSED  VPC % GAP CLOSED  \
INSTANCE          disj_terms                                             
10teams_presolved 0                     8.120488e-12      9.371044e-10   
                  2                    -4.872293e-12     -1.689062e-10   
                  4                     8.120488e-12     -2.744725e-10   
                  8                    -1.624098e-11      9.371044e-10   
                  16                   -9.744586e-12     -2.760966e-11   
                  32                   -9.744586e-12      0.000000e+00   
                  64                   -9.744586e-12      0.000000e+00   
23588_presolved   0                     7.218238e+01      7.161482e+01   
                  2                     2.188689e+01      1.745981e+01   
                  4                     3.409109e+01      2.749366e+01   
                  8                     4.357635e+01      3.475065e+01   
                  16                    4.952913e+01      4.821609e+01   
                  32                    6.209834e+01      6.059920e+01   
                  64                    7.218238e+01      7.161482e+01   
30n20b8_presolved 0                     1.564622e+00      3.089354e-02   
                  2                     4.865507e-02      7.780367e-03   
                  4                     8.645315e-02      3.089354e-02   
                  8                     8.707174e-02      0.000000e+00   
                  16                    3.210749e-01      0.000000e+00   
                  32                    4.434385e-01      0.000000e+00   
                  64                    1.564622e+00      0.000000e+00   

                              VPC+GMIC % GAP CLOSED  \
INSTANCE          disj_terms                          
10teams_presolved 0                      100.000000   
                  2                      100.000000   
                  4                      100.000000   
                  8                      100.000000   
                  16                     100.000000   
                  32                     100.000000   
                  64                     100.000000   
23588_presolved   0                       71.624582   
                  2                       18.670949   
                  4                       27.493660   
                  8                       34.750650   
                  16                      48.241773   
                  32                      60.621962   
                  64                      71.624582   
30n20b8_presolved 0                       11.099235   
           

#### DEBUG: Why REF+V is less than REF

In [23]:
### DEBUG
# Why REF+V < REF

# inst = 'f2gap801600_presolved'
inst = 'neos-1112787_presolved'

tmp_df = gap_df.loc[inst,['NUM VPC']+['BEST REF FIRST_CUT_PASS']+['FIRST REF+V FIRST_CUT_PASS']+gap_cols+['LP OBJ','IP OBJ']]

# display(tmp_df)
# display(gap_df.loc[inst,['NUM VPC']+['BEST REF FIRST_CUT_PASS']+['FIRST REF+V FIRST_CUT_PASS']+gap_cols])

# display(selected_gap_df.loc[inst,['NUM VPC']+['BEST REF FIRST_CUT_PASS']+['FIRST REF+V FIRST_CUT_PASS']+gap_cols])

col = "REF+V FIRST_CUT_PASS"
tmp_df[col + " % GAP CLOSED"] = calc_gap_closed(tmp_df, "FIRST " + col)
display(tmp_df)

,NUM VPC,BEST REF FIRST_CUT_PASS,FIRST REF+V FIRST_CUT_PASS,GMIC % GAP CLOSED,BEST DISJ % GAP CLOSED,VPC % GAP CLOSED,VPC+GMIC % GAP CLOSED,"MAX(GMIC,VPC) % GAP CLOSED",FIRST REF FIRST_CUT_PASS % GAP CLOSED,BEST REF FIRST_CUT_PASS % GAP CLOSED,REF+V FIRST_CUT_PASS % GAP CLOSED,FIRST REF LAST_CUT_PASS % GAP CLOSED,BEST REF LAST_CUT_PASS % GAP CLOSED,REF+V LAST_CUT_PASS % GAP CLOSED,LP OBJ,IP OBJ
disj_terms,,,,,,,,,,,,,,,,
0,0,5.092058e+11,0.0,0.000000,0.000000,0.0,0.000000,0.000000,14.212392,14.212392,0.0,51.656914,51.860079,0.0,5.000000e+11,5.647730e+11
2,0,0.000000e+00,0.0,23.423909,1.183228,0.0,23.423909,23.423909,0.000000,0.000000,0.0,0.000000,0.000000,0.0,5.000000e+11,5.647730e+11
4,0,0.000000e+00,0.0,23.423909,3.188024,0.0,23.423909,23.423909,0.000000,0.000000,0.0,0.000000,0.000000,0.0,5.000000e+11,5.647730e+11
8,0,0.000000e+00,0.0,23.423909,5.099750,0.0,23.423909,23.423909,0.000000,0.000000,0.0,0.000000,0.000000,0.0,5.000000e+11,5.647730e+11
16,0,0.000000e+00,0.0,23.423909,9.931782,0.0,23.423909,23.423909,0.000000,0.000000,0.0,0.000000,0.000000,0.0,5.000000e+11,5.647730e+11
32,0,0.000000e+00,0.0,23.423909,23.324585,0.0,23.423909,23.423909,0.000000,0.000000,0.0,0.000000,0.000000,0.0,5.000000e+11,5.647730e+11
64,0,0.000000e+00,0.0,23.423909,75.121684,0.0,23.423909,23.423909,0.000000,0.000000,0.0,0.000000,0.000000,0.0,5.000000e+11,5.647730e+11


In [24]:
# inst = 'neos-1112787_presolved'

# display(gap_df.loc[inst,['NUM VPC']+['BEST REF FIRST_CUT_PASS']+['FIRST REF+V FIRST_CUT_PASS']+gap_cols])

# display(selected_gap_df.loc[inst,['NUM VPC']+['BEST REF FIRST_CUT_PASS']+['FIRST REF+V FIRST_CUT_PASS']+gap_cols])

### `best_gap_df`: For each instance, what the best gap closed is (and how that was obtained)

In [25]:
## Create best df = for each instance, what the best gap closed is (and how that was obtained)

# inst_set = selected_gap_instances_dict.keys()
inst_set = selected_gap_df.index.levels[0]
# inst_set = ['neos22_presolved']
# inst_set = ['usAbbrv-8-25_70_presolved']

best_gap_df = pd.DataFrame(
    columns = gap_cols_short+[
        'BEST VPC DISJ',
        'BEST GMIC+VPC DISJ',
        'BEST V+GurF DISJ',
        'BEST V+GurL DISJ',
        'NUM VPC',
        'NUM GMIC',
    ],
    index = inst_set,
    dtype = float,
)

num_inst = len(inst_set)
for i, inst in enumerate(inst_set):
    print("{}/{}".format(i+1,num_inst), end='\r', flush=True)
    # print("Processing instance {:d} with name {}.".format(i, inst))
    best_vpc = -1.
    best_vpc_disj = -1
    best_vpcgmic = -1.
    best_vpcgmic_disj = -1
    best_max_gmic_vpc = -1.
    best_VGurF = -1.
    best_VGurF_disj = -1
    best_VGurL = -1.
    best_VGurL_disj = -1
    best_num_vpc = -1
    best_num_gmic = -1
    
    curr_df = selected_gap_df.loc[inst]
    
    # Get info for GurF and GurL from the no-VPC row
    row = curr_df.loc[0]
    GurF_gap = float(row[col_first_ref])
    GurL_gap = float(row[col_last_ref])
    gmic_gap = float(row[col_gmic])
    disj_gap = float(row[col_best_disj])

    for index, row in curr_df.iterrows():
        num_disj_terms = int(row['NUM DISJ TERMS'])
        # num_obj_tried  = float(row['NUM OBJ'])
        num_vpc        = float(row[col_num_vpcs])
        if num_disj_terms <= 0 or num_vpc == 0:
            continue
            
        # print("Index {:d}: Processing instance {} with {:d} disj terms.".format(index, inst, num_disj_terms))
        vpc_gap     = float(row[col_vpc])
        vpcgmic_gap = float(row[col_vpc_gmic])
        VGurF_gap   = float(row[col_first_ref_v])
        VGurL_gap   = float(row[col_last_ref_v])
        # num_vpc     = float(row['NUM VPC'])
        num_gmic    = float(row[col_num_gmic])
        
        if (best_vpc < vpc_gap): #or (is_val(best_vpc, vpc_gap) and best_num_vpc == 0):
            best_vpc = vpc_gap
            best_vpc_disj = index
            best_num_vpc = num_vpc
            best_num_gmic = num_gmic
        if best_vpcgmic < vpcgmic_gap:
            best_vpcgmic = vpcgmic_gap
            best_vpcgmic_disj = index
        if best_max_gmic_vpc < max(vpc_gap, gmic_gap):
            best_max_gmic_vpc = max(vpc_gap, gmic_gap)
        if best_VGurF < VGurF_gap:
            best_VGurF = VGurF_gap
            best_VGurF_disj = index
        if best_VGurL < VGurL_gap:
            best_VGurL = VGurL_gap
            best_VGurL_disj = index

    best_gap_df.iloc[i] = [
        gmic_gap if gmic_gap >= EPS else 0.,
        disj_gap if disj_gap >= EPS else 0.,
        best_vpc if best_vpc >= EPS else 0.,
        best_vpcgmic if best_vpcgmic >= EPS else 0.,
        best_max_gmic_vpc if best_max_gmic_vpc >= EPS else 0.,
        GurF_gap if GurF_gap >= EPS else 0.,
        best_VGurF if best_VGurF >= EPS else 0.,
        GurL_gap if GurL_gap >= EPS else 0.,
        best_VGurL if best_VGurL >= EPS else 0.,
        best_vpc_disj,
        best_vpcgmic_disj,
        best_VGurF_disj,
        best_VGurL_disj,
        best_num_vpc,
        best_num_gmic,
    ]

col_list = ['BEST VPC DISJ', 'BEST GMIC+VPC DISJ', 'BEST V+GurF DISJ', 'BEST V+GurL DISJ', 'NUM VPC', 'NUM GMIC']
for col in col_list:
    best_gap_df[col] = best_gap_df[col].astype(np.int64)

display(best_gap_df)

,G,DB,V,V+G,"max(G,V)",GurF,V+GurF,GurL,V+GurL,BEST VPC DISJ,BEST GMIC+VPC DISJ,BEST V+GurF DISJ,BEST V+GurL DISJ,NUM VPC,NUM GMIC
INSTANCE,,,,,,,,,,,,,,,
10teams_presolved,100.000000,0.000000,0.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,8,2,16,2,51,153
23588_presolved,5.772831,72.182376,71.614817,71.624582,71.614817,16.055009,68.920337,26.063355,71.826753,64,64,64,64,75,74
30n20b8_presolved,11.099235,1.564622,0.030894,11.099235,11.099235,1.167537,1.198632,22.412056,19.692774,4,2,2,2,190,184
50v-10_presolved,45.753596,18.008191,11.184101,45.823184,45.753596,51.601082,49.881430,74.129495,74.179502,64,16,2,64,29,29
a1c1s1_presolved,25.105518,4.895611,1.094331,25.382401,25.105518,45.978556,46.016169,89.709365,89.135817,8,2,4,16,3,154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
umts_presolved,0.973181,0.209040,0.092781,0.973181,0.973181,1.700438,1.379851,5.359690,5.534309,32,2,2,2,235,275
usAbbrv-8-25_70_presolved,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8,8,8,8,4,722
vpm1_presolved,16.930266,7.788162,4.672897,16.930266,16.930266,57.476636,50.934579,100.000000,50.934579,32,2,32,32,10,16


#### DEBUG: In `best_gap_df`, can get V > V+G due to numerical issues

In [26]:
## DEBUG: You can get V > V+G due to numerical issues

col1 = best_gap_df['V']
col2 = best_gap_df['V+G']

display(best_gap_df[(col1 > col2 + EPS) == True])

df.loc['neos-1058477_presolved'] #.to_csv("neos-1058477_presolved_data.csv")

,G,DB,V,V+G,"max(G,V)",GurF,V+GurF,GurL,V+GurL,BEST VPC DISJ,BEST GMIC+VPC DISJ,BEST V+GurF DISJ,BEST V+GurL DISJ,NUM VPC,NUM GMIC
INSTANCE,,,,,,,,,,,,,,,


,cutlimit,gomory,mode,partial_bb_strategy,partial_bb_keep_pruned_nodes,partial_bb_num_strong,preprocess,prlp_flip_beta,rounds,strengthen,...,gurobi_version,cplex_version,ExitReason,end_time_string,time elapsed,instname,Unnamed: 286,Unnamed: 280,IS PURE BINARY,IS MIXED BINARY
disj_terms,,,,,,,,,,,,,,,,,,,,,
0,-1,0,0,4,0,5,0,0,1,1,...,10.01,NaN,NO_DISJUNCTION,Thu Jun 15 20:09:42 2023,1,neos-1058477_presolved,NaN,DONE,False,True
2,-1,-1,0,4,0,5,0,0,1,1,...,10.01,NaN,CUT_LIMIT,Wed Jun 28 04:23:17 2023,4,neos-1058477_presolved,DONE,NaN,False,True
4,-1,-1,0,4,0,5,0,0,1,1,...,10.01,NaN,SUCCESS,Mon Jun 26 20:59:45 2023,3,neos-1058477_presolved,DONE,NaN,False,True
8,-1,-1,0,4,0,5,0,0,1,1,...,10.01,NaN,CUT_LIMIT,Sat Jul 1 12:29:20 2023,33,neos-1058477_presolved,DONE,NaN,False,True
16,-1,-1,0,4,0,5,0,0,1,1,...,10.01,NaN,PRLP_INFEASIBLE,Tue Jun 27 10:51:50 2023,25,neos-1058477_presolved,DONE,NaN,False,True
32,-1,-1,0,4,0,5,0,0,1,1,...,10.01,NaN,PRLP_TIME_LIMIT,Tue Jun 27 07:02:27 2023,77,neos-1058477_presolved,DONE,NaN,False,True
64,-1,-1,0,4,0,5,0,0,1,1,...,10.01,NaN,PRLP_TIME_LIMIT,Fri Jun 30 03:17:23 2023,132,neos-1058477_presolved,DONE,NaN,False,True


#### DEBUG: Find instances in which V+GurF max does not match up

In [27]:
## DEBUG: Find instances in which V+GurF max does not match up
# This causes the value in Table 2 'Best' row to not match Table 1 'All'

# For instance f2gap801600_presolved, the gap closed at the end of the root node is 0% whenever VPCs are used,
# but without VPCs, the gap closed is 50%
# In `best_gap_df`, for an instance in which no VPCs were generated,
# we use the value of GurF/GurL for V+GurF/V+GurL
# In `selected_gap_df`, the "zero" row contains

num_inst = len(best_gap_df.index)
col = 'V+GurF'
origcol = map_short_to_cols_gap[col]
num_errors = 0
avg1 = 0
avg2 = 0
for inst in best_gap_df.index:
    val1 = best_gap_df.loc[inst,col]
    val2 = selected_gap_df.loc[(inst,0),origcol]
    if abs(val1-val2) > EPS:
        print("{} has best_gap_df = {:f} and selected_gap_df = {:f} for col {} (diff = {:e})".format(inst,val1,val2,col,abs(val1-val2)))
        num_errors += 1
    avg1 += val1 / num_inst
    avg2 += val2 / num_inst

print("Average from best_gap_df = {}".format(avg1))
print("Average from selected_gap_df = {}".format(avg2))
print("Total # of errors =", num_errors, flush=True)

Average from best_gap_df = 29.92671280500642
Average from selected_gap_df = 29.926712805008744
Total # of errors = 0


#### DEBUG: Print relevant info from `selected_gap_df` and `best_gap_df` to further debug

In [28]:
## DEBUG
# inst = 'f2gap801600_presolved'
# inst = 'neos22_presolved'
# inst = 'neos-1112787_presolved'
# display(best_gap_df.loc[inst])
# display(selected_gap_df.loc[inst,[col_num_vpcs]+gap_cols])

In [29]:
# ## DEBUG
# gap_cols = [
#     'GMIC % GAP CLOSED',
#     'BEST DISJ % GAP CLOSED',
#     'VPC % GAP CLOSED',
#     'VPC+GMIC % GAP CLOSED',
#     'REF FIRST_CUT_PASS % GAP CLOSED',
#     'REF+V FIRST_CUT_PASS % GAP CLOSED',
#     'REF LAST_CUT_PASS % GAP CLOSED',
#     'REF+V LAST_CUT_PASS % GAP CLOSED',
# ]
# col_num_vpcs = 'NUM VPC'

inst = 'f2gap801600_presolved'
tmp_selected_gap_df = gap_df.loc[selected_gap_instances_dict.keys()]
# Check if inst is in tmp_selected_gap_df
if inst not in tmp_selected_gap_df.index.get_level_values(0).unique():
    print(ValueError("Instance {} is not in tmp_selected_gap_df".format(inst)))
else:
    curr_df = tmp_selected_gap_df.loc[inst].copy() # copy needed to not throw SettingWithCopyWarning

    # Set 0-row to have max values across all rows for this instance
    max_vals = curr_df[gap_cols].max()
    # selected_gap_df.loc[(inst,0),gap_cols] = max_vals

    display(tmp_selected_gap_df.loc[inst])
    display(max_vals)

,NUM DISJ TERMS,ROWS,COLS,LP OBJ,BEST DISJ OBJ,WORST DISJ OBJ,IP OBJ,GMIC OBJ,VPC OBJ,VPC+GMIC OBJ,...,BEST DISJ % GAP CLOSED,VPC % GAP CLOSED,VPC+GMIC % GAP CLOSED,"MAX(GMIC,VPC) % GAP CLOSED",FIRST REF FIRST_CUT_PASS % GAP CLOSED,BEST REF FIRST_CUT_PASS % GAP CLOSED,REF+V FIRST_CUT_PASS % GAP CLOSED,FIRST REF LAST_CUT_PASS % GAP CLOSED,BEST REF LAST_CUT_PASS % GAP CLOSED,REF+V LAST_CUT_PASS % GAP CLOSED
disj_terms,,,,,,,,,,,,,,,,,,,,,
0,0,80,1600,86570.120867,-inf,-inf,86679.0,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,49.794634,49.794634,0.000000,49.794634,49.794634,0.000000
2,2,80,1600,86570.120867,8.657127e+04,8.657845e+04,86679.0,86655.691018,86571.026383,86655.691018,...,1.056924,0.831671,78.591874,78.591874,49.794634,49.794634,95.407752,49.794634,49.794634,95.407752
4,4,80,1600,86570.120867,8.657302e+04,8.658275e+04,86679.0,86655.691018,86571.188056,86655.691018,...,2.660687,0.980160,78.591874,78.591874,49.794634,49.794634,95.407752,49.794634,49.794634,95.407752
8,8,80,1600,86570.120867,8.657410e+04,8.661955e+04,86679.0,86655.691018,86571.326821,86655.707800,...,3.653145,1.107609,78.607287,78.591874,49.794634,49.794634,95.407752,49.794634,49.794634,95.407752
16,16,80,1600,86570.120867,8.657505e+04,8.661955e+04,86679.0,86655.691018,NaN,86655.691018,...,4.524866,0.000000,78.591874,78.591874,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
32,32,80,1600,86570.120867,8.657588e+04,8.661955e+04,86679.0,86655.691018,86571.271636,86655.691018,...,5.286915,1.056924,78.591874,78.591874,49.794634,49.794634,95.407752,49.794634,49.794634,95.407752
64,64,80,1600,86570.120867,8.657675e+04,8.662223e+04,86679.0,86655.691018,NaN,86655.691018,...,6.086314,0.000000,78.591874,78.591874,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


GMIC % GAP CLOSED                        78.591874
BEST DISJ % GAP CLOSED                    6.086314
VPC % GAP CLOSED                          1.107609
VPC+GMIC % GAP CLOSED                    78.607287
MAX(GMIC,VPC) % GAP CLOSED               78.591874
FIRST REF FIRST_CUT_PASS % GAP CLOSED    49.794634
BEST REF FIRST_CUT_PASS % GAP CLOSED     49.794634
REF+V FIRST_CUT_PASS % GAP CLOSED        95.407752
FIRST REF LAST_CUT_PASS % GAP CLOSED     49.794634
BEST REF LAST_CUT_PASS % GAP CLOSED      49.794634
REF+V LAST_CUT_PASS % GAP CLOSED         95.407752
dtype: float64

### Table 1: `avg_gap_df`: average percent gap closed across different combinations of cuts

In [30]:
## TABLE 1: average percent gap closed across different combinations of cuts
## Create avg_gap_df = average gap closed across instances
all_set_name = 'All'
good_vpc_set_name = tex_escape('≥10%')
avg_row_name = tex_escape('Avg (%)')
wins_row_name = 'Wins'

idx = pd.MultiIndex.from_product(
    [ [all_set_name, good_vpc_set_name], [avg_row_name, wins_row_name] ],
    names = ['Set', '']
)
    
ncols = len(best_gap_df.columns)
nrows = len(idx)

col = best_gap_df['V'].astype(float)
good_vpc_df = best_gap_df[col >= 10.]

data = np.zeros((nrows, ncols), dtype=float)
data[0,:] = [best_gap_df[col].mean() for col in best_gap_df.columns]
data[2,:] = [good_vpc_df[col].mean() for col in best_gap_df.columns]

# display(best_gap_df.head())
avg_gap_df = pd.DataFrame(
    data,
    columns = best_gap_df.columns,
    index = idx,
    dtype = object
)

inst_col_name = '# inst'
avg_gap_df[inst_col_name] = [len(best_gap_df), 0, len(good_vpc_df), 0]

avg_gap_df.iloc[1] = ["" for i in range(ncols+1)]
avg_gap_df.iloc[3] = ["" for i in range(ncols+1)]

display(avg_gap_df)

G         DB          V        V+G   max(G,V)  \
Set                                                                         
All       Avg (\%)  13.446463  15.705382   9.513701  19.260231  18.369989   
          Wins                                                              
$\ge$10\% Avg (\%)  15.531816  37.912082  27.780148  33.140178  31.144471   
          Wins                                                              

                         GurF     V+GurF       GurL     V+GurL BEST VPC DISJ  \
Set                                                                            
All       Avg (\%)  26.653322  29.926713  45.828516  47.889972      31.97561   
          Wins                                                                 
$\ge$10\% Avg (\%)  27.495584  37.802223  49.145302  56.698233     50.666667   
          Wins                                                                 

                   BEST GMIC+VPC DISJ BEST V+GurF DISJ BEST V+GurL DISJ  \
Set                                                                       
All       Avg (\%)          28.164634        19.030488        18.957317   
          Wins                                                            
$\ge$10\% Avg (\%)            47.6875        34.166667          33.4375   
          Wins                                                            

                      NUM VPC    NUM GMIC # inst  
Set                                               
All       Avg (\%)  55.628049  173.097561    328  
          Wins                                    
$\ge$10\% Avg (\%)  52.260417     79.4375     96  
          Wins

### `wins_df`: num wins

In [31]:
## Create num wins df
# x wins over y for an instance if x > y + EPS
#shortcols = avg_gap_df.columns[0:-1]
wins_df = pd.DataFrame(
    np.zeros((len(gap_cols_short), len(gap_cols_short)), dtype=int),
    columns = gap_cols_short,
    index = gap_cols_short,
    dtype = int,
)

from itertools import permutations
for (ind1, ind2) in permutations(range(len(gap_cols_short)), 2):
    wins_df.at[gap_cols_short[ind1],gap_cols_short[ind2]] =\
        int(sum(best_gap_df[gap_cols_short[ind1]] > best_gap_df[gap_cols_short[ind2]] + EPS))
    wins_df.at[gap_cols_short[ind2],gap_cols_short[ind1]] =\
        int(sum(best_gap_df[gap_cols_short[ind2]] > best_gap_df[gap_cols_short[ind1]] + EPS))

# Sets we are considering
# all_set = 'Wins (All)'
# good_vpc_set = 'Wins (V ≥ 10%)'
all_set = (all_set_name,wins_row_name)
good_vpc_set = (good_vpc_set_name,wins_row_name)

# "G" are wins relative to "V"
shortrefcol = 'V'
#refcol = 'VPC % GAP CLOSED'
#refcol = map_short_to_cols[shortrefcol]
refcol = shortrefcol
shortdestcol = 'G'
#col = 'GMIC % GAP CLOSED'
#col = map_short_to_cols[shortcol]
destcol = shortdestcol
avg_gap_df.at[all_set,shortdestcol] = wins_df.at[shortdestcol,shortrefcol]
avg_gap_df.at[good_vpc_set,shortdestcol] = sum(good_vpc_df[destcol] > good_vpc_df[refcol] + EPS)

# "DB", "V", "V+G": wins are relative to "G"
shortrefcol = 'G'
#refcol = 'GMIC % GAP CLOSED'
#refcol = map_short_to_cols[shortrefcol]
refcol = shortrefcol
shortdestcol = 'DB'
#col = 'BEST DISJ % GAP CLOSED'
#col = map_short_to_cols[shortcol]
destcol = shortdestcol
avg_gap_df.at[all_set,shortdestcol] = wins_df.at[shortdestcol,shortrefcol]
avg_gap_df.at[good_vpc_set,shortdestcol] = sum(good_vpc_df[destcol] > good_vpc_df[refcol] + EPS)

shortdestcol = 'V'
#col = 'VPC % GAP CLOSED'
#col = map_short_to_cols[shortcol]
destcol = shortdestcol
avg_gap_df.at[all_set,shortdestcol] = wins_df.at[shortdestcol,shortrefcol]
avg_gap_df.at[good_vpc_set,shortdestcol] = sum(good_vpc_df[destcol] > good_vpc_df[refcol] + EPS)

shortdestcol = 'V+G'
#col = 'VPC+GMIC % GAP CLOSED'
#col = map_short_to_cols[shortcol]
destcol = shortdestcol
avg_gap_df.at[all_set,shortdestcol] = wins_df.at[shortdestcol,shortrefcol]
avg_gap_df.at[good_vpc_set,shortdestcol] = sum(good_vpc_df[destcol] > good_vpc_df[refcol] + EPS)

# "V+GurF" are wins relative to "GurF"
shortrefcol = 'GurF'
refcol = shortrefcol
shortdestcol = 'V+GurF'
destcol = shortdestcol
#col = map_short_to_cols[shortcol]
avg_gap_df.at[all_set,shortdestcol] = wins_df.at[shortdestcol,shortrefcol]
avg_gap_df.at[good_vpc_set,shortdestcol] = sum(good_vpc_df[destcol] > good_vpc_df[refcol] + EPS)

# "V+GurL" are wins relative to "GurL"
shortrefcol = 'GurL'
refcol = shortrefcol
shortdestcol = 'V+GurL'
destcol = shortdestcol
wins_df.at[shortdestcol,shortrefcol] = int(sum(best_gap_df[destcol] > best_gap_df[refcol] + EPS))
wins_df.at[shortrefcol,shortdestcol] = int(sum(best_gap_df[refcol] > best_gap_df[destcol] + EPS))
avg_gap_df.at[all_set,shortdestcol] = wins_df.at[shortdestcol,shortrefcol]
avg_gap_df.at[good_vpc_set,shortdestcol] = sum(good_vpc_df[destcol] > good_vpc_df[refcol] + EPS)

# Count number of instances that have V+G > 0
shortdestcol = inst_col_name
#col = 'V+GurL'
destcol = 'V+G'
avg_gap_df.at[all_set,shortdestcol] = sum(best_gap_df[destcol] > EPS)
avg_gap_df.at[good_vpc_set,shortdestcol] = sum(good_vpc_df[destcol] > EPS)

display(avg_gap_df)
display(wins_df)

G         DB          V        V+G   max(G,V)  \
Set                                                                         
All       Avg (\%)  13.446463  15.705382   9.513701  19.260231  18.369989   
          Wins            140        155        113        192              
$\ge$10\% Avg (\%)  15.531816  37.912082  27.780148  33.140178  31.144471   
          Wins             20         86         76         93              

                         GurF     V+GurF       GurL     V+GurL BEST VPC DISJ  \
Set                                                                            
All       Avg (\%)  26.653322  29.926713  45.828516  47.889972      31.97561   
          Wins                       175                   124                 
$\ge$10\% Avg (\%)  27.495584  37.802223  49.145302  56.698233     50.666667   
          Wins                        79                    61                 

                   BEST GMIC+VPC DISJ BEST V+GurF DISJ BEST V+GurL DISJ  \
Set                                                                       
All       Avg (\%)          28.164634        19.030488        18.957317   
          Wins                                                            
$\ge$10\% Avg (\%)            47.6875        34.166667          33.4375   
          Wins                                                            

                      NUM VPC    NUM GMIC # inst  
Set                                               
All       Avg (\%)  55.628049  173.097561    328  
          Wins                               253  
$\ge$10\% Avg (\%)  52.260417     79.4375     96  
          Wins                                96

,G,DB,V,V+G,"max(G,V)",GurF,V+GurF,GurL,V+GurL
G,0,102,140,0,0,51,27,11,6
DB,155,0,225,131,142,103,94,61,47
V,113,0,0,0,0,70,45,45,6
V+G,192,115,214,0,153,103,79,53,16
"max(G,V)",113,102,140,0,0,98,67,52,11
GurF,200,158,191,154,158,0,58,0,3
V+GurF,229,160,206,171,182,175,0,40,0
GurL,250,206,223,211,212,247,218,0,111
V+GurL,255,211,253,239,244,249,242,124,0


### Analyze instances in which DB > G but V <= G

In [32]:
col1 = 'DB'
col2 = 'G'
tmp_df = best_gap_df.loc[best_gap_df[col1] > best_gap_df[col2] + EPS]

col1 = 'V'
tmp_df = tmp_df[tmp_df[col1] <= tmp_df[col2] + EPS]
display(tmp_df.head())

# inst_set = tmp_df.index
inst_depth_set = [(inst,tmp_df.at[inst,'BEST VPC DISJ']) for inst in tmp_df.index]

print("Total num inst with DB > G >= V is {:d}".format(len(tmp_df)))
print("Num times hit cut limit = {:d}".format(sum(df.loc[inst_depth_set,'ExitReason'] == 'CUT_LIMIT')))

# display(df.loc[inst_depth_set])


,G,DB,V,V+G,"max(G,V)",GurF,V+GurF,GurL,V+GurL,BEST VPC DISJ,BEST GMIC+VPC DISJ,BEST V+GurF DISJ,BEST V+GurL DISJ,NUM VPC,NUM GMIC
INSTANCE,,,,,,,,,,,,,,,
app3_presolved,20.407763,55.666020,15.523885,26.668549,20.407763,32.104990,34.623929,84.971740,84.902346,32,32,32,4,15,18
beasleyC3_presolved,1.296774,2.810011,1.236898,1.796111,1.296774,58.444201,63.682504,98.560735,97.951900,16,32,2,2,124,124
bppc8-09_presolved,3.078078,3.301803,0.593229,3.078078,3.078078,2.435273,0.725151,4.687396,2.756221,4,32,4,2,30,30
eilB101_presolved,2.465774,12.019007,1.078716,2.885857,2.465774,12.032046,12.032046,45.034822,44.066223,16,16,2,2,89,71
eild76_presolved,1.085382,9.275935,0.565928,1.132339,1.085382,10.508184,10.527491,52.399319,54.313587,16,16,8,8,63,55


Total num inst with DB > G >= V is 42
Num times hit cut limit = 25


### Analyze instances in which V+G <= G

In [33]:
col1 = 'V+G'
col2 = 'G'
tmp_df = best_gap_df.loc[best_gap_df[col1] <= best_gap_df[col2] + EPS]

display(tmp_df)

inst_depth_set = [(inst,tmp_df.at[inst,'BEST VPC DISJ']) for inst in tmp_df.index]

print("Total num inst with V+G <= G is {:d}".format(len(tmp_df)))
print("Num times with G = 100% gap closed = {:d}".format(sum(tmp_df['G'] == 100.)))
print("Num times with V+G = 0% gap closed = {:d}".format(sum(tmp_df['V+G'] == 0.)))
print("Num times hit cut limit = {:d}".format(sum(df.loc[inst_depth_set,'ExitReason'] == 'CUT_LIMIT')))

# display(df.loc[inst_depth_set])

,G,DB,V,V+G,"max(G,V)",GurF,V+GurF,GurL,V+GurL,BEST VPC DISJ,BEST GMIC+VPC DISJ,BEST V+GurF DISJ,BEST V+GurL DISJ,NUM VPC,NUM GMIC
INSTANCE,,,,,,,,,,,,,,,
10teams_presolved,100.000000,0.000000,0.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,8,2,16,2,51,153
30n20b8_presolved,11.099235,1.564622,0.030894,11.099235,11.099235,1.167537,1.198632,22.412056,19.692774,4,2,2,2,190,184
a2c1s1_presolved,24.372660,3.337449,0.461882,24.372660,24.372660,42.891045,44.309222,92.388157,92.110315,2,4,64,16,18,157
b1c1s1_presolved,23.005942,3.738399,0.044877,23.005942,23.005942,22.510555,27.761022,76.892805,78.065195,8,2,64,16,4,240
b2c1s1_presolved,19.812503,2.007490,0.065546,19.812503,19.812503,23.193264,23.246604,73.285439,73.571818,32,4,64,64,2,238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tr12-30_presolved,58.360211,1.743446,0.477979,58.360211,58.360211,60.343792,60.423920,98.775186,99.051261,64,2,2,2,2,321
traininstance6_presolved,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2,2,2,2,2,21
umts_presolved,0.973181,0.209040,0.092781,0.973181,0.973181,1.700438,1.379851,5.359690,5.534309,32,2,2,2,235,275


Total num inst with V+G <= G is 136
Num times with G = 100% gap closed = 0
Num times with V+G = 0% gap closed = 75
Num times hit cut limit = 26


### Analyze when G > V

In [34]:
col1 = 'G'
col2 = 'V'
tmp_df = best_gap_df.loc[best_gap_df[col1] > best_gap_df[col2] + EPS]

display(tmp_df)

inst_depth_set = [(inst,tmp_df.at[inst,'BEST VPC DISJ']) for inst in tmp_df.index]

print("Total num inst with G > V is {:d}".format(len(tmp_df)))
print("Num times with #V < 10 is {:d}".format(len(tmp_df[(tmp_df['NUM VPC'] < 10)])))
print("Num times with #V < 10 while #G > 10 is {:d}".format(len(tmp_df[(tmp_df['NUM VPC'] < 10) & (tmp_df['NUM GMIC'] > 10)])))
# print("Num times with #V < 10 is {:d}".format(sum(tmp_df['NUM VPC'] < 10)))

# print("Num times with V+G = 0% gap closed = {:d}".format(sum(tmp_df['V+G'] == 0.)))
print("Num times hit cut limit = {:d}".format(sum(df.loc[inst_depth_set,'ExitReason'] == 'CUT_LIMIT')))

tmp_inst_set = tmp_df[(tmp_df['NUM VPC'] < 10) & (tmp_df['NUM GMIC'] > 10)].index
tmp_inst_depth_set = [(inst,tmp_df.at[inst,'BEST VPC DISJ']) for inst in tmp_inst_set]
print("Num times hit cut limit when #G > #V = {:d} (should be 0)".format(sum(df.loc[tmp_inst_depth_set,'ExitReason'] == 'CUT_LIMIT')))

# display(df.loc[inst_depth_set])

,G,DB,V,V+G,"max(G,V)",GurF,V+GurF,GurL,V+GurL,BEST VPC DISJ,BEST GMIC+VPC DISJ,BEST V+GurF DISJ,BEST V+GurL DISJ,NUM VPC,NUM GMIC
INSTANCE,,,,,,,,,,,,,,,
10teams_presolved,100.000000,0.000000,0.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,8,2,16,2,51,153
30n20b8_presolved,11.099235,1.564622,0.030894,11.099235,11.099235,1.167537,1.198632,22.412056,19.692774,4,2,2,2,190,184
50v-10_presolved,45.753596,18.008191,11.184101,45.823184,45.753596,51.601082,49.881430,74.129495,74.179502,64,16,2,64,29,29
a1c1s1_presolved,25.105518,4.895611,1.094331,25.382401,25.105518,45.978556,46.016169,89.709365,89.135817,8,2,4,16,3,154
a2c1s1_presolved,24.372660,3.337449,0.461882,24.372660,24.372660,42.891045,44.309222,92.388157,92.110315,2,4,64,16,18,157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tr12-30_presolved,58.360211,1.743446,0.477979,58.360211,58.360211,60.343792,60.423920,98.775186,99.051261,64,2,2,2,2,321
umts_presolved,0.973181,0.209040,0.092781,0.973181,0.973181,1.700438,1.379851,5.359690,5.534309,32,2,2,2,235,275
vpm1_presolved,16.930266,7.788162,4.672897,16.930266,16.930266,57.476636,50.934579,100.000000,50.934579,32,2,32,32,10,16


Total num inst with G > V is 140
Num times with #V < 10 is 59
Num times with #V < 10 while #G > 10 is 55
Num times hit cut limit = 52
Num times hit cut limit when #G > #V = 0 (should be 0)


### Analyze when DB % gap closed nontrivial

In [35]:
# Select instances in selected_gap_df for which value in col_best_disj is at least MIN_DISJ_GAP
MIN_DISJ_GAP = 0.
tmp_df = selected_gap_df.loc[selected_gap_df[col_best_disj] >= MIN_DISJ_GAP]

# Collect instance names with best disj gap >= MIN_DISJ_GAP
tmp_df.index = tmp_df.index.remove_unused_levels()
inst_set_db = tmp_df.index.get_level_values(0).unique()
num_inst_db = len(inst_set_db)
inst_set_orig = selected_gap_df.index.levels[0]
num_inst_orig = len(inst_set)
print("Total num inst with best disj gap >= {:f} is {:d} (out of {:d} total instances).".format(MIN_DISJ_GAP, num_inst_db, num_inst_orig))

# Report average in each column broken down by depth
tmp_df_grouped = tmp_df.groupby(level='disj_terms').mean(numeric_only=True)
display(tmp_df_grouped[gap_cols])

# Repeat with MIN_DISJ_GAP = 1.0
MIN_DISJ_GAP = 1.
tmp_df = selected_gap_df.loc[selected_gap_df[col_best_disj] >= MIN_DISJ_GAP]

# Collect instance names with best disj gap >= MIN_DISJ_GAP
tmp_df.index = tmp_df.index.remove_unused_levels()
inst_set_db = tmp_df.index.get_level_values(0).unique()
num_inst_db = len(inst_set_db)
inst_set_orig = selected_gap_df.index.levels[0]
num_inst_orig = len(inst_set)
print("Total num inst with best disj gap >= {:f} is {:d} (out of {:d} total instances).".format(MIN_DISJ_GAP, num_inst_db, num_inst_orig))

# Report average in each column broken down by depth
tmp_df_grouped = tmp_df.groupby(level='disj_terms').mean(numeric_only=True)
display(tmp_df_grouped[gap_cols])

# Repeat with MIN_DISJ_GAP = 10.
MIN_DISJ_GAP = 10.
tmp_df = selected_gap_df.loc[selected_gap_df[col_best_disj] >= MIN_DISJ_GAP]

# Collect instance names with best disj gap >= MIN_DISJ_GAP
tmp_df.index = tmp_df.index.remove_unused_levels()
inst_set_db = tmp_df.index.get_level_values(0).unique()
num_inst_db = len(inst_set_db)
inst_set_orig = selected_gap_df.index.levels[0]
num_inst_orig = len(inst_set)
print("Total num inst with best disj gap >= {:f} is {:d} (out of {:d} total instances).".format(MIN_DISJ_GAP, num_inst_db, num_inst_orig))

# Report average in each column broken down by depth
tmp_df_grouped = tmp_df.groupby(level='disj_terms').mean(numeric_only=True)
display(tmp_df_grouped[gap_cols])


Total num inst with best disj gap >= 0.000000 is 328 (out of 328 total instances).


,GMIC % GAP CLOSED,BEST DISJ % GAP CLOSED,VPC % GAP CLOSED,VPC+GMIC % GAP CLOSED,"MAX(GMIC,VPC) % GAP CLOSED",FIRST REF FIRST_CUT_PASS % GAP CLOSED,BEST REF FIRST_CUT_PASS % GAP CLOSED,REF+V FIRST_CUT_PASS % GAP CLOSED,FIRST REF LAST_CUT_PASS % GAP CLOSED,BEST REF LAST_CUT_PASS % GAP CLOSED,REF+V LAST_CUT_PASS % GAP CLOSED
disj_terms,,,,,,,,,,,
0,13.446463,15.705382,9.513701,19.260231,18.369989,24.939044,26.653322,29.926713,43.249463,45.828516,47.889972
2,13.402795,2.546211,1.873728,14.077991,13.681678,25.153761,26.886351,27.158841,43.696634,46.359151,44.470210
4,13.590678,4.580387,3.094776,14.765579,14.234946,25.504243,27.184067,27.689167,44.292979,46.623875,45.378825
8,13.311424,7.044397,4.118763,15.150096,14.472347,25.349827,26.982470,27.892486,44.335410,46.500552,45.660185
16,13.339735,9.794206,5.241589,16.161291,15.508492,25.045033,26.650050,28.200160,43.913833,46.049438,45.298861
32,13.521727,13.254301,7.159369,17.753057,16.950253,25.197021,26.612354,29.289129,43.630742,45.506034,45.831328
64,13.415370,16.617307,8.104504,18.544392,17.831941,25.069386,26.421368,29.537853,43.560105,45.383411,46.546395


Total num inst with best disj gap >= 1.000000 is 221 (out of 328 total instances).


,GMIC % GAP CLOSED,BEST DISJ % GAP CLOSED,VPC % GAP CLOSED,VPC+GMIC % GAP CLOSED,"MAX(GMIC,VPC) % GAP CLOSED",FIRST REF FIRST_CUT_PASS % GAP CLOSED,BEST REF FIRST_CUT_PASS % GAP CLOSED,REF+V FIRST_CUT_PASS % GAP CLOSED,FIRST REF LAST_CUT_PASS % GAP CLOSED,BEST REF LAST_CUT_PASS % GAP CLOSED,REF+V LAST_CUT_PASS % GAP CLOSED
disj_terms,,,,,,,,,,,
0,17.172556,23.271665,14.099284,25.795527,24.476662,30.103202,31.702710,37.216060,51.399537,53.841190,57.586499
2,18.034060,6.228002,4.563816,19.627171,18.738590,31.095840,32.003560,34.641253,51.209488,53.372441,52.514526
4,18.051421,8.630777,5.840959,20.274593,19.276213,31.518635,33.053257,34.889742,52.659490,54.922798,54.561769
8,18.194450,11.950722,6.992612,21.318665,20.170391,31.532043,33.038711,35.296449,53.028577,55.258639,55.422604
16,17.926488,15.746611,8.426757,22.468787,21.421448,31.037246,32.628626,35.598749,52.460304,54.687251,54.975654
32,17.894799,19.773264,10.689081,24.214943,23.019039,30.969205,32.681493,36.686272,52.310946,54.690438,55.751332
64,17.172556,23.271665,11.351049,24.361770,23.364537,30.103202,31.702710,36.143602,51.399537,53.841190,55.642255


Total num inst with best disj gap >= 10.000000 is 147 (out of 328 total instances).


,GMIC % GAP CLOSED,BEST DISJ % GAP CLOSED,VPC % GAP CLOSED,VPC+GMIC % GAP CLOSED,"MAX(GMIC,VPC) % GAP CLOSED",FIRST REF FIRST_CUT_PASS % GAP CLOSED,BEST REF FIRST_CUT_PASS % GAP CLOSED,REF+V FIRST_CUT_PASS % GAP CLOSED,FIRST REF LAST_CUT_PASS % GAP CLOSED,BEST REF LAST_CUT_PASS % GAP CLOSED,REF+V LAST_CUT_PASS % GAP CLOSED
disj_terms,,,,,,,,,,,
0,16.185303,32.341262,19.888672,28.513512,26.776269,28.441482,30.295679,36.990677,50.047107,52.667710,57.569994
2,25.773618,24.196163,16.979095,32.410542,30.236838,42.113317,42.392293,47.628771,67.368810,70.224810,66.589222
4,20.745589,20.630246,14.632706,26.604431,24.341525,34.283017,35.236081,37.813359,57.457392,60.658103,60.710604
8,18.131760,22.905260,13.820889,24.538896,22.452878,31.367259,32.743361,35.965035,55.096282,58.416241,58.012090
16,16.820844,25.072231,13.690085,24.436280,22.853130,29.400623,31.438551,35.217747,52.472035,55.715132,55.607735
32,16.029797,28.888945,15.842426,25.662639,23.973513,28.034758,29.928782,35.051899,50.264727,53.182478,54.693305
64,16.185303,32.341262,15.903204,26.413892,25.122821,28.441482,30.295679,35.766689,50.047107,52.667710,55.468102


### Table 2: `gap_by_size_df`: gap closed by num leaves

In [36]:
## TABLE 2: gap closed by num leaves
## Note that ``best'' can be worse than for a single row
## because when no VPCs are generated, we assume the "no VPCs" results hold for Gurobi,
## but we do not count that for the ``best'' calculation,
## since otherwise there is potential bias, as sometimes Gurobi does better without VPCs
shortcols = [
        'DB',
        'V',
        'V+G',
        'max(G,V)',
        #'GurF',
        'V+GurF',
        #'GurL',
        'V+GurL',
    ]

gap_by_size_df = pd.DataFrame(
    columns = shortcols,
    index = [0] + sizes + ['Best'],
    # index = [str(size) + " leaves" for size in sizes]+['Best'],
    dtype = float,
)
zero_row_name = 0

# `grouped_df` will collect gap closed across instances, grouped by num terms
grouped_df = selected_gap_df.groupby(level='disj_terms').mean()
ungrouped_df = best_gap_df.mean()

# For each of the columns (in shortcols),
# save the average value for each size
# (this will put in the right place as the index is based on sizes for both)
for col in shortcols:
    orig_col = map_short_to_cols_gap[col]
    #gap_by_size_df.loc[2]['DB'] = best_gap_df[orig_col].mean()
    gap_by_size_df[col] = grouped_df[orig_col]

# Fill in the 'Best' row, since that is currently stored in `gap_by_size_df` in the "0" row
gap_by_size_df.loc['Best'] = gap_by_size_df.loc[zero_row_name]

# Now update the zero row with correct values
col = 'DB'
gap_by_size_df[col][zero_row_name] = 0.

col = 'V'
gap_by_size_df[col][zero_row_name] = 0.

stubs = ['G', 'GurF', 'GurL']
for stub in stubs:
    col = 'V+'+stub
    # orig_col = map_short_to_cols[stub]
    gap_by_size_df[col][0] = ungrouped_df[stub]

# Also replace the 0-row of the "max(G,V)" column with the value of G, since that corresponds to no VPCs
gap_by_size_df['max(G,V)'][0] = gap_by_size_df['V+G'][0]

# Reindex to add "leaves" to index
idx = ['VPCs disabled']+[str(size) + " leaves" for size in sizes]+['Best']
reidx = {old_id : new_id for old_id, new_id in zip(gap_by_size_df.index,idx)}
gap_by_size_df.rename(reidx, inplace=True)

# display(grouped_df[gap_cols])
display(ungrouped_df)
display(gap_by_size_df)

# Create new df with additional columns:
# (1) the ratio 'V'/'DB'
# (2) the ratio 'max(G,V)'/'V+G'
gap_by_size_df_new = gap_by_size_df.copy()
gap_by_size_df_new['V/DB'] = gap_by_size_df_new['V'] / gap_by_size_df_new['DB']
gap_by_size_df_new['max(G,V)/V+G'] = gap_by_size_df_new['max(G,V)'] / gap_by_size_df_new['V+G']
display(gap_by_size_df_new)

/var/folders/_8/bgx4yr0n6f1dm56pnvsqqb1s91dzml/T/ipykernel_17114/3267952728.py:26: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped_df = selected_gap_df.groupby(level='disj_terms').mean()


G                      13.446463
DB                     15.705382
V                       9.513701
V+G                    19.260231
max(G,V)               18.369989
GurF                   26.653322
V+GurF                 29.926713
GurL                   45.828516
V+GurL                 47.889972
BEST VPC DISJ          31.975610
BEST GMIC+VPC DISJ     28.164634
BEST V+GurF DISJ       19.030488
BEST V+GurL DISJ       18.957317
NUM VPC                55.628049
NUM GMIC              173.097561
dtype: float64

,DB,V,V+G,"max(G,V)",V+GurF,V+GurL
VPCs disabled,0.000000,0.000000,13.446463,13.446463,26.653322,45.828516
2 leaves,2.453057,1.805177,14.098190,13.715143,26.905517,43.985345
4 leaves,4.440741,3.000423,14.585543,14.071088,27.058672,44.346485
8 leaves,6.700767,3.917847,15.195443,14.550755,27.358989,44.737419
16 leaves,9.346300,5.001882,16.138983,15.516038,27.949882,44.797036
32 leaves,12.446112,6.722822,17.419784,16.665932,28.783325,45.553851
64 leaves,15.705382,7.659745,18.294014,17.620661,29.179875,46.268580
Best,15.705382,9.513701,19.260231,18.369989,29.926713,47.889972


,DB,V,V+G,"max(G,V)",V+GurF,V+GurL,V/DB,"max(G,V)/V+G"
VPCs disabled,0.000000,0.000000,13.446463,13.446463,26.653322,45.828516,NaN,1.000000
2 leaves,2.453057,1.805177,14.098190,13.715143,26.905517,43.985345,0.735889,0.972830
4 leaves,4.440741,3.000423,14.585543,14.071088,27.058672,44.346485,0.675658,0.964728
8 leaves,6.700767,3.917847,15.195443,14.550755,27.358989,44.737419,0.584686,0.957574
16 leaves,9.346300,5.001882,16.138983,15.516038,27.949882,44.797036,0.535172,0.961401
32 leaves,12.446112,6.722822,17.419784,16.665932,28.783325,45.553851,0.540154,0.956724
64 leaves,15.705382,7.659745,18.294014,17.620661,29.179875,46.268580,0.487715,0.963193
Best,15.705382,9.513701,19.260231,18.369989,29.926713,47.889972,0.605761,0.953778


### Table 5: `all_gap_results_df`: complete gap closed results

In [37]:
inst_set = selected_gap_df.index.levels[0]
inst_set.set_names("Instance",inplace=True)

col_idx = pd.MultiIndex.from_arrays(
    [
        ['', '', '# cuts', '# cuts'] + ['% gap closed']*len(gap_cols_short),
        ['Rows', 'Cols', 'G', 'V'] + gap_cols_short
    ],
)

all_gap_results_df = pd.DataFrame(
    columns = col_idx,
    index = inst_set,
    dtype = object,
)

# Enter number of rows and cols
tmp_df = df.xs(0, level='disj_terms').loc[inst_set,['ROWS','COLS']]
tmp_df.columns = pd.MultiIndex.from_product([[''],['Rows','Cols']])
all_gap_results_df.loc[:,tmp_df.columns] = tmp_df

# Enter number of cuts
# tmp_df = best_gap_df.xs(0, level='disj_terms').loc[inst_set,['NUM GMIC', 'NUM VPC']]
tmp_df = best_gap_df.loc[inst_set, ['NUM GMIC', 'NUM VPC']]
tmp_df.columns = pd.MultiIndex.from_product([['# cuts'],['G','V']])
all_gap_results_df.loc[:,tmp_df.columns] = tmp_df

# Enter gap closed
tmp_df = best_gap_df.loc[inst_set, gap_cols_short]
tmp_df.columns = pd.MultiIndex.from_product([['% gap closed'],gap_cols_short])
all_gap_results_df.loc[:,tmp_df.columns] = tmp_df

# Add average row
all_gap_results_df.loc["Average"] = all_gap_results_df.loc[:,('% gap closed',gap_cols_short)].mean()

# Now convert the % gap closed columns to objects so we can add an int row
all_gap_results_df.loc[:,('% gap closed',gap_cols_short)] = all_gap_results_df.loc[:,('% gap closed',gap_cols_short)].astype(object)

# Add wins row
win_gap_cols_short = ['DB', 'V', 'V+G', 'V+GurF', 'V+GurL']
all_gap_results_df.loc['Wins',('% gap closed',win_gap_cols_short)] = avg_gap_df.loc[all_set,win_gap_cols_short].values.tolist()
# all_gap_results_df.loc['Wins',('% gap closed',win_gap_cols_short)] = avg_gap_df.loc[all_set,gap_cols_short].astype(np.int64).values.tolist()
# all_gap_results_df.loc["Wins"] = avg_gap_df.loc[all_set,gap_cols_short]
# wins_df.at[cols[ind1],cols[ind2]] = int(sum(best_gap_df[cols[ind1]] > best_gap_df[cols[ind2]] + EPS))

# Replace missing entries with empty string
all_gap_results_df = all_gap_results_df.fillna('',downcast=False)

# Convert rows, cols, # cuts to int values
tmp_cols = pd.MultiIndex.from_product([[''],['Rows','Cols']])
all_gap_results_df.loc[inst_set,tmp_cols] = all_gap_results_df.loc[inst_set,tmp_cols].astype(np.int64)
tmp_cols = pd.MultiIndex.from_product([['# cuts'],['G','V']])
all_gap_results_df.loc[inst_set,tmp_cols] = all_gap_results_df.loc[inst_set,tmp_cols].astype(np.int64)

display(all_gap_results_df.tail())

print("Num instances =",len(all_gap_results_df)-2)

/var/folders/_8/bgx4yr0n6f1dm56pnvsqqb1s91dzml/T/ipykernel_17114/2696136982.py:20: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  all_gap_results_df.loc[:,tmp_df.columns] = tmp_df
/var/folders/_8/bgx4yr0n6f1dm56pnvsqqb1s91dzml/T/ipykernel_17114/2696136982.py:26: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  all_gap_results_df.loc[:,tmp_df.columns] = tmp_df
/var/folders/_8/bgx4yr0n6f1dm56pnvsqqb1s91dzml/T/ipykernel_17114/2696136982.py:31: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace

# cuts     % gap closed                       \
                     Rows  Cols      G   V            G         DB         V   
Instance                                                                       
vpm1_presolved        128   188     16  10    16.930266   7.788162  4.672897   
vpm2_presolved        127   187     25  25    17.849671  14.293216  7.834301   
zib54-UUE_presolved  1114  3726     56  56    10.696523  17.642953  7.938383   
Average                                       13.446463  15.705382  9.513701   
Wins                                                           155       113   

                                                                            \
                           V+G   max(G,V)       GurF     V+GurF       GurL   
Instance                                                                     
vpm1_presolved       16.930266  16.930266  57.476636  50.934579      100.0   
vpm2_presolved       19.708285  17.849671  47.349883  52.793701  74.022681   
zib54-UUE_presolved   15.53831  10.696523  46.359509  54.154746  69.739045   
Average              19.260231  18.369989  26.653322  29.926713  45.828516   
Wins                       192                              175              

                                
                        V+GurL  
Instance                        
vpm1_presolved       50.934579  
vpm2_presolved       72.616451  
zib54-UUE_presolved  68.124432  
Average              47.889972  
Wins                       124

Num instances = 328


# Section 3: Time tables

## `time_df`: Create subset of dataframe relevant to time

In [71]:
## Create subset of dataframe relevant to time
time_df = df.loc[:, 
                [
                    'NUM DISJ TERMS',
                    'ROWS',
                    'COLS',
                    'LP OBJ',
                    'IP OBJ',
                    'FIRST REF OBJ',
                    'AVG REF OBJ',
                    'BEST REF OBJ',
                    'FIRST REF+V OBJ',
                    'AVG REF+V OBJ',
                    'FIRST REF BOUND',
                    'AVG REF BOUND',
                    'BEST REF BOUND',
                    'FIRST REF+V BOUND',
                    'AVG REF+V BOUND',
                    'FIRST REF ITERS',
                    'AVG REF ITERS',
                    'BEST REF ITERS',
                    'FIRST REF+V ITERS',
                    'AVG REF+V ITERS',
                    'FIRST REF NODES',
                    'AVG REF NODES',
                    'BEST REF NODES',
                    'FIRST REF+V NODES',
                    'AVG REF+V NODES',
                    'FIRST REF TIME',
                    'AVG REF TIME',
                    'BEST REF TIME',
                    'FIRST REF+V TIME',
                    'AVG REF+V TIME',
                    'VPC_GEN_TIME',
                    'NUM GMIC',
                    'NUM VPC',
                    'NUM OBJ',
                    'ALL REF TIME',
                    'ALL REF+V TIME',
                    'ExitReason']
               ]
#display(time_df.loc[("bm23_presolved",2)])

## Prepare short/long column names for time dfs
1. First run of Gurobi without VPCs
2. Best among 7 runs of Gurobi without VPCs
3. First run of Gurobi with VPCs for each disjunction size
4. First run of Gurobi with VPCs for each disjunction size, adding cut generation time
5. Best run across first Gurobi without VPCs and first Gurobi with VPCs (across all terms)

In [74]:
col_num_vpcs = 'NUM VPC'
col_vpc_gen_time = 'VPC_GEN_TIME'

# Gur1/Gur7 names
gur1_col_stub = 'FIRST REF' # Should we change to AVG?
gur7_col_stub = 'BEST REF'
gur1v_col_stub = gur1_col_stub + '+V'
gur1v_w_cut_col_stub = gur1v_col_stub + ' W/CUTGEN'

# gur1time: first run of Gurobi without VPCs
gur1time_col = gur1_col_stub + ' TIME'
gur1nodes_col = gur1_col_stub + ' NODES'

# gur7time: best among 7 runs of Gurobi without VPCs
gur7time_col = gur7_col_stub + ' TIME'
gur7nodes_col = gur7_col_stub + ' NODES'

# gur1vtime: first run of Gurobi w/VPCs for each disj size
gur1vtime_col = gur1v_col_stub + ' TIME'
gur1vnodes_col = gur1v_col_stub + ' NODES'

# gur1v_w_cut_time: first run of Gurobi w/VPCs for each disj size, counting cut generation time
gur1v_w_cut_time_col = gur1v_w_cut_col_stub + ' TIME'

# Track best disjunction used in 0-row
gurv_disj_col = gur1v_col_stub + ' DISJ'
gurv_w_cut_disj_col = gur1v_w_cut_col_stub + ' DISJ'

# Best Gurobi run across the first without VPCs and first w/VPCs for each disj size
mintime_col       = 'MIN BB TIME'
mintime_w_cut_col = 'MIN BB W/CUTGEN TIME'
mintime_disj_col  = 'MIN BB TIME DISJ'
minnodes_col      = 'MIN BB NODES'

map_cols_to_short_time = {
    gur1time_col         : 'Gur1',
    gur7time_col         : 'Gur7',
    gur1vtime_col        : 'V',
    gur1v_w_cut_time_col : 'Total',
    mintime_col          : 'V7',
    mintime_w_cut_col    : 'Total7',
}

map_cols_to_short_nodes = {
    gur1nodes_col        : 'Gur1',
    gur7nodes_col        : 'Gur7',
    gur1vnodes_col       : 'V',
    minnodes_col         : 'V7',
}

map_short_to_cols_time = {v: k for k, v in map_cols_to_short_time.items()}
map_short_to_cols_nodes = {v: k for k, v in map_cols_to_short_nodes.items()}

time_cols_short = list(map_short_to_cols_time.keys())
node_cols_short = list(map_short_to_cols_nodes.keys())
# display(time_cols, node_cols)

# Select a subset of columns for the "long" list used when updating the 0-row
time_cols_long = [map_short_to_cols_time[col] for col in time_cols_short]
node_cols_long = [map_short_to_cols_nodes[col] for col in node_cols_short]

# # Update list of columns with mintime cols
# newshortcol1 = 'V7'
# newshortcol2 = 'Total7'
# newshortcol3 = 'V7'
# map_cols_to_short_time [mintime_col]       = newshortcol1
# map_cols_to_short_time [mintime_w_cut_col] = newshortcol2
# map_cols_to_short_nodes[minnodes_col]      = newshortcol3

# map_short_to_cols_time [newshortcol1]      = mintime_col
# map_short_to_cols_time [newshortcol2]      = mintime_w_cut_col
# map_short_to_cols_nodes[newshortcol3]      = minnodes_col

# time_cols_short.append(newshortcol1)
# time_cols_short.append(newshortcol2)
# node_cols_short.append(newshortcol3)

## Add total time for running solver + generating cuts

In [75]:
# Add total time for running solver + generating cuts
time_df[gur1v_w_cut_time_col] = time_df[gur1vtime_col] + time_df[col_vpc_gen_time]

display(time_df.loc['bm23_presolved'])

,NUM DISJ TERMS,ROWS,COLS,LP OBJ,IP OBJ,FIRST REF OBJ,AVG REF OBJ,BEST REF OBJ,FIRST REF+V OBJ,AVG REF+V OBJ,...,AVG REF+V TIME,VPC_GEN_TIME,NUM GMIC,NUM VPC,NUM OBJ,ALL REF TIME,ALL REF+V TIME,ExitReason,AVG REF+V W/CUTGEN TIME,FIRST REF+V W/CUTGEN TIME
disj_terms,,,,,,,,,,,,,,,,,,,,,
0,0,20,27,20.570922,34.0,34.0,34.0,34.0,0.0,0.0,...,0.000,0.00,0,0,0,0.120737;0.116043;0.120618;0.158696;0.126145;0...,NaN,NO_DISJUNCTION,0.000,0.000
2,2,20,27,20.570922,34.0,34.0,34.0,34.0,34.0,34.0,...,0.132,0.01,6,6,8,0.124761;0.122699;0.122670;0.162981;0.129027;0...,0.147789;0.105460;0.116325;0.140183;0.137159;0...,CUT_LIMIT,0.142,0.158
4,4,20,27,20.570922,34.0,34.0,34.0,34.0,34.0,34.0,...,0.127,0.02,6,6,7,0.122179;0.116280;0.115538;0.158093;0.128733;0...,0.114969;0.133110;0.125814;0.131684;0.121659;0...,CUT_LIMIT,0.147,0.135
8,8,20,27,20.570922,34.0,34.0,34.0,34.0,34.0,34.0,...,0.146,0.03,6,6,7,0.129148;0.122269;0.125794;0.165218;0.130844;0...,0.153219;0.134925;0.150829;0.121706;0.158747;0...,CUT_LIMIT,0.176,0.183
16,16,20,27,20.570922,34.0,34.0,34.0,34.0,34.0,34.0,...,0.128,0.05,6,6,7,0.126258;0.121518;0.123165;0.163783;0.129714;0...,0.137835;0.119083;0.129786;0.125912;0.128590;0...,CUT_LIMIT,0.178,0.188
32,32,20,27,20.570922,34.0,34.0,34.0,34.0,34.0,34.0,...,0.114,0.10,6,6,7,0.123828;0.119528;0.122219;0.160673;0.128640;0...,0.108789;0.110813;0.112637;0.118277;0.118973;0...,CUT_LIMIT,0.214,0.209
64,64,20,27,20.570922,34.0,34.0,34.0,34.0,34.0,34.0,...,0.123,0.21,6,6,7,0.122517;0.122204;0.121509;0.163164;0.131323;0...,0.122178;0.136256;0.119202;0.135599;0.116280;0...,CUT_LIMIT,0.333,0.332


## `selected_time_df`: Solving and cut-generation time for instances selected for time reporting; 0-row with min values across all rows

In [76]:
## Solving and cut-generation time for instances selected for time reporting
selected_time_df = time_df.loc[selected_time_instances_dict.keys()]
selected_time_df.index = selected_time_df.index.remove_unused_levels()
selected_time_df[minnodes_col] = 0

## Fill in 0-row with min values across all rows
## Also fill in gur1 values (present only in 0 row currently) for all disj terms
comparison_time_cols = [gur1vtime_col, gur1v_w_cut_time_col]
comparison_node_cols = [gur1vnodes_col]
cols_to_display = [col_num_vpcs]+[gur1time_col,gur1vtime_col]+[gur1nodes_col,gur1vnodes_col]+[mintime_col,mintime_w_cut_col,minnodes_col,gurv_disj_col,gurv_w_cut_disj_col,mintime_disj_col]
inst_set = selected_time_df.index.levels[0]
# tmp_inst = '23588_presolved'
# inst_set = ['10teams_presolved',tmp_inst]
for i, inst in enumerate(inst_set):
    print("{}/{}".format(i+1,len(inst_set)), end='\r', flush=True)
    curr_df = selected_time_df.loc[inst].copy() # copy needed to not throw SettingWithCopyWarning
    
    # Select only the rows in which VPCs were generated
    curr_df_with_vpcs = curr_df[curr_df[col_num_vpcs] > 0]
    
    # display(inst)
    # display(curr_df_with_vpcs[[col_num_vpcs]+[gur1time_col,gur1vtime_col]])

    # Set 0-row to have min time values across all (non-0-vpc) rows for this instance
    # best_vals = curr_df_with_vpcs[comparison_time_cols].min()
    # selected_time_df.loc[(inst,0),comparison_time_cols] = best_vals
    best_vals_idx = curr_df_with_vpcs[comparison_time_cols].idxmin()
    for curr_col, curr_disj_id in zip(comparison_time_cols, best_vals_idx):
        selected_time_df.at[(inst,0),curr_col] = curr_df_with_vpcs.at[curr_disj_id, curr_col]
    
    # display(best_vals_idx)
    # print("selected_time_df.at[('{}',0),gur1vtime_col] = {}".format(inst,selected_time_df.at[(inst,0),gur1vtime_col]))
    # display(selected_time_df[[col_num_vpcs]+[gur1time_col,gur1vtime_col]].head(14))
    # print("selected_time_df.at[('{}',0),gur1vtime_col] = {}".format(inst,selected_time_df.at[(inst,0),gur1vtime_col]))


    # Also add id of the best disj to the 0-row
    selected_time_df.at[(inst,0),gurv_disj_col]       = int(best_vals_idx[0])
    selected_time_df.at[(inst,0),gurv_w_cut_disj_col] = int(best_vals_idx[1])

    # Update 0-row of mintime (V7) entries
    curr_gur1time       = selected_time_df.at[(inst,0),gur1time_col]
    curr_gur1vtime      = selected_time_df.at[(inst,0),gur1vtime_col]
    curr_gur1vcuts_time = selected_time_df.at[(inst,0),gur1v_w_cut_time_col]
    
    curr_vals = [curr_gur1time, curr_gur1vtime]
    min_id = np.argmin(curr_vals)

    # If min_id is 0, then no cuts are used and we report the gur1 time
    # If min_id is 1, then gur1v < gur1 and we can report the number of cuts used
    selected_time_df.at[(inst,0),mintime_col] = curr_vals[min_id]

    # Add num cuts from mintime disj into num vpc col
    best_disj_size = 0 if min_id == 0 else best_vals_idx[0]
    selected_time_df.at[(inst,0),mintime_disj_col] = best_disj_size
    best_num_cuts = selected_time_df.at[(inst,best_disj_size),col_num_vpcs]
    selected_time_df.at[(inst,0),col_num_vpcs] = best_num_cuts

    # Update with cuts into Total7 column
    curr_vals = [curr_gur1time, curr_gur1vcuts_time]
    selected_time_df.at[(inst,0),mintime_w_cut_col] = min(curr_vals)

    # Repeat for nodes
    best_vals = curr_df_with_vpcs[comparison_node_cols].min()
    selected_time_df.loc[(inst,0),comparison_node_cols] = best_vals
    # selected_time_df.at[(inst,0),minnodes_col] = int(selected_time_df.loc[(inst,0),[gur1nodes_col,gur1vnodes_col]].min())

    curr_gur1nodes       = selected_time_df.at[(inst,0),gur1nodes_col]
    curr_gur1vnodes      = selected_time_df.at[(inst,0),gur1vnodes_col]
    curr_vals = [curr_gur1nodes, curr_gur1vnodes]
    min_id = np.argmin(curr_vals)
    selected_time_df.at[(inst,0),minnodes_col] = int(curr_vals[min_id])

    # Propogate down 0-row values for gur1 columns
    selected_time_df.loc[inst, gur1time_col] = curr_gur1time
    selected_time_df.loc[inst, gur1nodes_col] = curr_gur1nodes

    #### FOR SOME REASON, THE BELOW ZEROES OUT selected_time_df.loc[[(inst,0)]][gur1vtime_col]
    # display(selected_time_df.loc[(inst,0),[gur1nodes_col,gur1vnodes_col]])

    ## OLD CODE BELOW
    # best_vals_idx = curr_df_with_vpcs[comparison_node_cols].idxmin()
    # for curr_col, curr_disj_id in zip(comparison_node_cols, best_vals_idx):
    #     selected_time_df.at[(inst,0),curr_col] = curr_df_with_vpcs.loc[curr_disj_id, curr_col]

    # # Also add id of the best disj to the 0-row
    # selected_time_df.at[(inst,0),gurv_disj_col + ' (NODES)'] = int(best_vals_idx[0])
    # selected_time_df.at[(inst,0),gurv_w_cut_disj_col+ ' (NODES)'] = int(best_vals_idx[1])

    # for ind in curr_df.index:
    #     if ind == 0:
    #         continue

    #     # Propogate GurF and GurL down
    #     subinds = [4,6]
    #     sel_gap = [gap_cols[i] for i in subinds]
    #     selected_gap_df.loc[(inst,ind),sel_gap] = curr_df.loc[0,sel_gap]

    #     # If no VPCs produced, the values for V+GurF and V+GurL have not been provided
    #     # We replace these by GurF and GurL
    #     # Currently disabled: update max for that column too (if disabled, we instead keep max as the value among those that generated VPCs)
    #     num_vpc = curr_df.loc[ind,col_num_vpcs]
    #     if num_vpc == 0:
    #         # print("Zero cuts for inst {} at depth {:d}".format(inst, ind))
    #         subinds = [5,7]
    #         refinds = [4,6]
    #         sel_gap = [gap_cols[i] for i in subinds]
    #         selected_gap_df.loc[(inst,ind),sel_gap] = curr_df.loc[0,[gap_cols[i] for i in refinds]].to_numpy()

    #         # for i in refinds:
    #         #     if curr_df.loc[0,gap_cols[i]] > selected_gap_df.loc[(inst,0),gap_cols[i+1]]:
    #         #         if curr_df.loc[0,gap_cols[i]] > 0:
    #         #             # print("DEBUG: Updating {} for inst {} from {:f} to {:f}".format(
    #         #             #     gap_cols[i+1], 
    #         #             #     inst, 
    #         #             #     selected_gap_df.loc[(inst,0),gap_cols[i+1]], 
    #         #             #     curr_df.loc[0,gap_cols[i]]))
    #         #         selected_gap_df.loc[(inst,0),gap_cols[i+1]] = curr_df.loc[0,gap_cols[i]]

# Add minimum time when using cuts and when not using cuts
# selected_time_df[mintime_col] = selected_time_df[[gur1time_col, gur1vtime_col]].min(axis=1)
# selected_time_df[mintime_w_cut_col] = selected_time_df[[gur1time_col, gur1v_w_cut_time_col]].min(axis=1)
# selected_time_df[minnodes_col] = selected_time_df[[gur1nodes_col,gur1vnodes_col]].min(axis=1)

display(selected_time_df.head(35).loc[:,[col_num_vpcs]+[gur1time_col,gur1vtime_col]+[gur1nodes_col,gur1vnodes_col]+[mintime_col,mintime_w_cut_col,minnodes_col,gurv_disj_col,gurv_w_cut_disj_col,mintime_disj_col]])
# display(selected_time_df.loc['10teams_presolved',[col_num_vpcs]+[gur1time_col,gur1vtime_col]+[gur1nodes_col,gur1vnodes_col]+[mintime_col,mintime_w_cut_col,minnodes_col,gurv_disj_col,gurv_w_cut_disj_col,mintime_disj_col]])
# display(selected_time_df.loc[inst_set,cols_to_display])

NUM VPC  FIRST REF TIME  FIRST REF+V TIME  \
INSTANCE          disj_terms                                              
10teams_presolved 0                 0           0.924             2.482   
                  2                77           0.924             2.619   
                  4                 1           0.924             2.482   
                  8                51           0.924             2.667   
                  16                5           0.924            23.237   
                  32                0           0.924             0.000   
                  64                0           0.924             0.000   
23588_presolved   0                11           1.574             1.401   
                  2                11           1.574             1.401   
                  4                75           1.574             1.447   
                  8                75           1.574             1.405   
                  16               75           1.574             1.820   
                  32               75           1.574             3.367   
                  64               75           1.574             2.587   
30n20b8_presolved 0               190          35.451            29.150   
                  2               190          35.451            29.150   
                  4               190          35.451            30.717   
                  8                 0          35.451             0.000   
                  16                0          35.451             0.000   
                  32                0          35.451             0.000   
                  64                0          35.451             0.000   
a1c1s1_presolved  0                 4        2520.253          1624.189   
                  2                15        2520.253          1946.188   
                  4                 4        2520.253          1624.189   
                  8                 3        2520.253          1664.560   
                  16                1        2520.253          1720.260   
                  32                0        2520.253             0.000   
                  64                0        2520.253             0.000   
a2c1s1_presolved  0                18        1159.654           707.505   
                  2                18        1159.654           707.505   
                  4                 2        1159.654           812.401   
                  8                 2        1159.654           902.821   
                  16                2        1159.654           761.794   
                  32                3        1159.654           989.129   
                  64                2        1159.654           795.418   

                              FIRST REF NODES  FIRST REF+V NODES  MIN BB TIME  \
INSTANCE          disj_terms                                                    
10teams_presolved 0                         1                106        0.924   
                  2                         1                106          NaN   
                  4                         1                106          NaN   
                  8                         1                106          NaN   
                  16                        1                809          NaN   
                  32                        1                  0          NaN   
                  64                        1                  0          NaN   
23588_presolved   0                       654                998        1.401   
                  2                       654                998          NaN   
                  4                       654               1073          NaN   
                  8                       654               1223          NaN   
                  16                      654               1570          NaN   
                  32                      654               1518          NaN   
                  64                   

In [77]:
# ### DEBUGGING that first ref+v time gets zeroed out for some reason?
# tmp_df = selected_time_df[[col_num_vpcs]+[gur1time_col,gur1vtime_col]].head(14).copy(deep=True)
# display(tmp_df)

# print(tmp_df.loc[('23588_presolved',0),gur1vtime_col])
# display(tmp_df.loc[[('23588_presolved',0)]][gur1vtime_col])

# tmp_df = selected_time_df
# print(tmp_df.loc[('23588_presolved',0),gur1vtime_col])
# display(tmp_df.loc[[('23588_presolved',0)]][gur1vtime_col])
# display(tmp_df.loc['23588_presolved'])

## Table 3: `avg_bb_df`: average time/nodes taken

### Prepare variables for row/col names

In [78]:
## Prepare variables for row/col names

bb_classes = ['All', '6 trees']
num_bb_classes = len(bb_classes)

bucket_min = [0, 10, 100, 1000]
bucket_max = [3600, 3600, 3600, 3600]
num_buckets = len(bucket_min)
assert(len(bucket_max) == num_buckets)
bb_buckets = ['[' + str(bucket_min[j]) + ',' + str(bucket_max[j]) + ')' for j in range(num_buckets)]
# bucket_names = [classes[i] + ' [' + str(bucket_min[j]) + ',' + str(bucket_max[j]) + ')' for i in range(num_classes) for j in range(num_buckets)]
# display(bucket_names)

bb_metrics = ['Gmean', 'Wins1', 'Wins7']

time_col_header = 'Time (s)'
node_col_header = 'Nodes (\\#)'

### Set up empty `avg_bb_df`

In [79]:
## Prepare avg_bb_df

avg_bb_cols = pd.MultiIndex.from_arrays(
    [[time_col_header]*len(time_cols_short) + [node_col_header]*len(node_cols_short), time_cols_short + node_cols_short],
    names = ['criterion', 'type'])

#bb_row_names = pd.MultiIndex.from_product([bb_buckets, bb_row_names], names=['bucket', 'metric'])
bb_row_names = pd.MultiIndex.from_product(
    [bb_classes, bb_buckets, bb_metrics],
    names=['class', 'bucket', 'metric'])

avg_bb_df = pd.DataFrame(
    columns = avg_bb_cols,
    index = bb_row_names,
    dtype = float
)

display(avg_bb_df.loc[:,avg_bb_cols.get_level_values(0)==node_col_header].head(6))
#display(avg_bb_df.loc[(bb_classes[0], bb_buckets[1], bb_metrics[0]),:])
display(avg_bb_df.loc[(bb_classes[0], bb_buckets, bb_metrics[0]),:])

criterion              Nodes (\#)             
type                         Gur1 Gur7   V  V7
class bucket    metric                        
All   [0,3600)  Gmean         NaN  NaN NaN NaN
                Wins1         NaN  NaN NaN NaN
                Wins7         NaN  NaN NaN NaN
      [10,3600) Gmean         NaN  NaN NaN NaN
                Wins1         NaN  NaN NaN NaN
                Wins7         NaN  NaN NaN NaN

criterion                Time (s)                           Nodes (\#)       \
type                         Gur1 Gur7   V Total  V7 Total7       Gur1 Gur7   
class bucket      metric                                                      
All   [0,3600)    Gmean       NaN  NaN NaN   NaN NaN    NaN        NaN  NaN   
      [10,3600)   Gmean       NaN  NaN NaN   NaN NaN    NaN        NaN  NaN   
      [100,3600)  Gmean       NaN  NaN NaN   NaN NaN    NaN        NaN  NaN   
      [1000,3600) Gmean       NaN  NaN NaN   NaN NaN    NaN        NaN  NaN   

criterion                         
type                       V  V7  
class bucket      metric          
All   [0,3600)    Gmean  NaN NaN  
      [10,3600)   Gmean  NaN NaN  
      [100,3600)  Gmean  NaN NaN  
      [1000,3600) Gmean  NaN NaN

### `avg_bb_df`: shifted geometric mean of time taken across instances, in various buckets, and geomean of nodes too

In [80]:
## Create gmean_df
#   = shifted geometric mean of time taken across instances, in various buckets
#     and geomean of nodes too

# Custom functions for prior to python 3.8
# def geo_mean(iterable):
#     a = np.array(iterable)
#     return a.prod()**(1.0/len(a))
# def geo_mean_overflow(iterable):
#     return np.exp(np.log(iterable).mean())
from statistics import geometric_mean
SHIFT_TIME  = 60
SHIFT_NODES = 1000

num_inst = np.zeros(len(avg_bb_df),dtype = np.int64)
row_ind = 0

#avg_bb_df.loc[(bb_classes[0], bb_buckets, bb_metrics[0]),:] = \
shortcols_time = time_cols_short
cols_time = [map_short_to_cols_time[shortcol] for shortcol in shortcols_time]
shortcols_nodes = node_cols_short
cols_nodes = [map_short_to_cols_nodes[shortcol] for shortcol in shortcols_nodes]

cols = cols_time + cols_nodes
shortcols = shortcols_time + shortcols_nodes

# First calculate stats for "all" instances
curr_df = selected_time_df.loc[:,cols_time + cols_nodes]
curr_df = curr_df[curr_df.index.get_level_values(1) == 0] # take only best values

for i in range(num_buckets):
    curr_df = curr_df[curr_df[gur1time_col] > bucket_min[i]]
    
    avg_bb_df.loc[(bb_classes[0], bb_buckets[i], bb_metrics[0]),(time_col_header,shortcols_time)] = \
        [geometric_mean(curr_df[col] + SHIFT_TIME) - SHIFT_TIME for col in cols_time]
    avg_bb_df.loc[(bb_classes[0], bb_buckets[i], bb_metrics[0]),(node_col_header,shortcols_nodes)] = \
        [geometric_mean(curr_df[col] + SHIFT_NODES) - SHIFT_NODES for col in cols_nodes]
    
    print("row {:d}: {:d}".format(row_ind,len(curr_df)))
    
    num_inst[row_ind:row_ind+len(bb_metrics)] = len(bb_metrics)*[len(curr_df)]
    row_ind += len(bb_metrics)

# Now calculate stats for "6 trees" instances
curr_df = selected_time_df.loc[all6_instances_dict.keys(),cols_time + cols_nodes]
curr_df = curr_df[curr_df.index.get_level_values(1) == 0] # take only best values

for i in range(num_buckets):
    curr_df = curr_df[curr_df[gur1time_col] > bucket_min[i]]
    
    avg_bb_df.loc[(bb_classes[1], bb_buckets[i], bb_metrics[0]),(time_col_header,shortcols_time)] = \
        [geometric_mean(curr_df[col] + SHIFT_TIME) - SHIFT_TIME for col in cols_time]
    avg_bb_df.loc[(bb_classes[1], bb_buckets[i], bb_metrics[0]),(node_col_header,shortcols_nodes)] = \
        [geometric_mean(curr_df[col] + SHIFT_NODES) - SHIFT_NODES for col in cols_nodes]
    
    print("row {:d}: {:d}".format(row_ind,len(curr_df)))

    num_inst[row_ind:row_ind+len(bb_metrics)] = len(bb_metrics)*[len(curr_df)]
    row_ind += len(bb_metrics)

avg_bb_df[inst_col_name] = num_inst
# avg_bb_df['NUM INST'] = avg_bb_df['NUM INST'].astype(np.int64)

display(avg_bb_df.loc[(bb_classes, bb_buckets, bb_metrics[0]),:])

row 0: 300
row 3: 167
row 6: 104
row 9: 54
row 12: 196
row 15: 91
row 18: 52
row 21: 21


criterion                      Time (s)                            \
type                               Gur1         Gur7            V   
class   bucket      metric                                          
All     [0,3600)    Gmean    122.549348    92.121528   104.404089   
        [10,3600)   Gmean    372.160077   253.257370   298.963838   
        [100,3600)  Gmean   1013.883019   630.678814   739.745871   
        [1000,3600) Gmean   2735.052905  1806.126192  1948.602447   
6 trees [0,3600)    Gmean     70.495418    53.406876    57.905113   
        [10,3600)   Gmean    251.035865   171.073614   190.649739   
        [100,3600)  Gmean    694.578390   432.591238   488.148697   
        [1000,3600) Gmean   2041.368928  1261.348457  1501.703997   

criterion                                                          \
type                              Total           V7       Total7   
class   bucket      metric                                          
All     [0,3600)    Gmean    122.846078    97.418226   101.608059   
        [10,3600)   Gmean    360.938434   272.531900   287.632561   
        [100,3600)  Gmean    837.118637   694.637345   725.024131   
        [1000,3600) Gmean   2048.232664  1896.371598  1921.264700   
6 trees [0,3600)    Gmean     60.653710    55.637828    57.293347   
        [10,3600)   Gmean    200.240693   180.781730   187.507985   
        [100,3600)  Gmean    503.948396   469.135930   477.377836   
        [1000,3600) Gmean   1536.166586  1411.020662  1426.289646   

criterion                      Nodes (\#)                                \
type                                 Gur1           Gur7              V   
class   bucket      metric                                                
All     [0,3600)    Gmean     8046.919880    5629.881079    6345.310699   
        [10,3600)   Gmean    37722.190948   22564.554140   26284.072557   
        [100,3600)  Gmean   109107.844715   60877.502233   68723.786592   
        [1000,3600) Gmean   207373.658314  119223.080759  118938.819424   
6 trees [0,3600)    Gmean     6594.443364    4882.944212    5181.375266   
        [10,3600)   Gmean    50114.513149   30725.533951   33099.058886   
        [100,3600)  Gmean   170635.523904   95096.793799  102696.233500   
        [1000,3600) Gmean   375814.985276  196242.059804  209395.165889   

criterion                                 # inst  
type                                   V7         
class   bucket      metric                        
All     [0,3600)    Gmean     5978.223166    300  
        [10,3600)   Gmean    24239.645399    167  
        [100,3600)  Gmean    65572.205702    104  
        [1000,3600) Gmean   115682.750940     54  
6 trees [0,3600)    Gmean     5003.320856    196  
        [10,3600)   Gmean    31549.113326     91  
        [100,3600)  Gmean   100590.324913     52  
        [1000,3600) Gmean   201544.492402     21

### Update wins1 rows

In [82]:
## Update wins1 rows
# A win in terms of time is counted when the ``Gur1'' baseline seconds taken 
# is at least 10\% slower, to account for some variability in runtimes.
# A win in terms of nodes is when the ``Gur1'' baseline number of nodes is higher.

# Make all columns "object" type to allow for integer values
avg_bb_df.loc[:,(time_col_header,shortcols_time)] = avg_bb_df.loc[:,(time_col_header,shortcols_time)].astype(object)
avg_bb_df.loc[:,(node_col_header,shortcols_nodes)] = avg_bb_df.loc[:,(node_col_header,shortcols_nodes)].astype(object)

# First calculate stats for "all" instances
curr_df = selected_time_df.loc[:,cols_time + cols_nodes]
curr_df = curr_df[curr_df.index.get_level_values(1) == 0] # take only best values

for i in range(num_buckets):
    curr_df = curr_df[curr_df[gur1time_col] > bucket_min[i]]
    
    refcol = gur1time_col
    avg_bb_df.loc[(bb_classes[0], bb_buckets[i], bb_metrics[1]),(time_col_header,shortcols_time)] = \
        [ int(sum(curr_df[refcol] > 1.1*curr_df[col])) for col in cols_time ]

    refcol = gur1nodes_col
    avg_bb_df.loc[(bb_classes[0], bb_buckets[i], bb_metrics[1]),(node_col_header,shortcols_nodes)] = \
        [ int(sum(curr_df[refcol] > curr_df[col])) for col in cols_nodes ]

# Now calculate stats for "6 trees" instances
curr_df = selected_time_df.loc[all6_instances_dict.keys(),cols_time + cols_nodes]
curr_df = curr_df[curr_df.index.get_level_values(1) == 0] # take only best values

for i in range(num_buckets):
    curr_df = curr_df[curr_df[gur1time_col] > bucket_min[i]]
    
    refcol = gur1time_col
    avg_bb_df.loc[(bb_classes[1], bb_buckets[i], bb_metrics[1]),(time_col_header,shortcols_time)] = \
        [ int(sum(curr_df[refcol] > 1.1*curr_df[col])) for col in cols_time ]

    refcol = gur1nodes_col
    avg_bb_df.loc[(bb_classes[1], bb_buckets[i], bb_metrics[1]),(node_col_header,shortcols_nodes)] = \
        [ int(sum(curr_df[refcol] > curr_df[col])) for col in cols_nodes ]

### Update wins7 rows

In [83]:
## Update wins7 rows
# A win in terms of time is counted when the ``Gur1'' baseline seconds taken 
# is at least 10\% slower, to account for some variability in runtimes.
# A win in terms of nodes is when the ``Gur1'' baseline number of nodes is higher.

# First calculate stats for "all" instances
curr_df = selected_time_df.loc[:,cols_time + cols_nodes]
curr_df = curr_df[curr_df.index.get_level_values(1) == 0] # take only best values
for i in range(num_buckets):
    curr_df = curr_df[curr_df[gur1time_col] > bucket_min[i]]
    
    refcol = gur7time_col
    avg_bb_df.loc[(bb_classes[0], bb_buckets[i], bb_metrics[2]),(time_col_header,shortcols_time)] = \
        [ int(sum(curr_df[refcol] > 1.1*curr_df[col])) for col in cols_time ]

    refcol = gur7nodes_col
    avg_bb_df.loc[(bb_classes[0], bb_buckets[i], bb_metrics[2]),(node_col_header,shortcols_nodes)] = \
        [ int(sum(curr_df[refcol] > curr_df[col])) for col in cols_nodes ]

# Now calculate stats for "6 trees" instances
curr_df = selected_time_df.loc[all6_instances_dict.keys(),cols_time + cols_nodes]
curr_df = curr_df[curr_df.index.get_level_values(1) == 0] # take only best values

for i in range(num_buckets):
    curr_df = curr_df[curr_df[gur1time_col] > bucket_min[i]]
    
    refcol = gur7time_col
    avg_bb_df.loc[(bb_classes[1], bb_buckets[i], bb_metrics[2]),(time_col_header,shortcols_time)] = \
        [ int(sum(curr_df[refcol] > 1.1*curr_df[col])) for col in cols_time ]

    refcol = gur7nodes_col
    avg_bb_df.loc[(bb_classes[1], bb_buckets[i], bb_metrics[2]),(node_col_header,shortcols_nodes)] = \
        [ int(sum(curr_df[refcol] > curr_df[col])) for col in cols_nodes ]

In [84]:
# display(avg_bb_df.loc[:,cols.get_level_values(0)=='Nodes'].head(6))
display(avg_bb_df.loc[(bb_classes[0:2], bb_buckets, bb_metrics[0:3]),:])

criterion                      Time (s)                            \
type                               Gur1         Gur7            V   
class   bucket      metric                                          
All     [0,3600)    Gmean    122.549348    92.121528   104.404089   
                    Wins1             0          193          184   
                    Wins7             0            0           68   
        [10,3600)   Gmean    372.160077    253.25737   298.963838   
                    Wins1             0          113          106   
                    Wins7             0            0           35   
        [100,3600)  Gmean   1013.883019   630.678814   739.745871   
                    Wins1             0           69           60   
                    Wins7             0            0           17   
        [1000,3600) Gmean   2735.052905  1806.126192  1948.602447   
                    Wins1             0           25           20   
                    Wins7             0            0            7   
6 trees [0,3600)    Gmean     70.495418    53.406876    57.905113   
                    Wins1             0          132          127   
                    Wins7             0            0           50   
        [10,3600)   Gmean    251.035865   171.073614   190.649739   
                    Wins1             0           71           67   
                    Wins7             0            0           22   
        [100,3600)  Gmean     694.57839   432.591238   488.148697   
                    Wins1             0           42           38   
                    Wins7             0            0           10   
        [1000,3600) Gmean   2041.368928  1261.348457  1501.703997   
                    Wins1             0           15           12   
                    Wins7             0            0            4   

criterion                                                          \
type                              Total           V7       Total7   
class   bucket      metric                                          
All     [0,3600)    Gmean    122.846078    97.418226   101.608059   
                    Wins1            89          184           89   
                    Wins7            21           68           21   
        [10,3600)   Gmean    360.938434     272.5319   287.632561   
                    Wins1            76          106           76   
                    Wins7            19           35           19   
        [100,3600)  Gmean    837.118637   694.637345   725.024131   
                    Wins1            48           60           48   
                    Wins7            12           17           12   
        [1000,3600) Gmean   2048.232664  1896.371598    1921.2647   
                    Wins1            19           20           19   
                    Wins7             6            7            6   
6 trees [0,3600)    Gmean      60.65371    55.637828    57.293347   
                    Wins1            67          127           67   
                    Wins7            15           50           15   
        [10,3600)   Gmean    200.240693    180.78173   187.507985   
                    Wins1            57           67           57   
                    Wins7            14           22           14   
        [100,3600)  Gmean    503.948396    469.13593   477.377836   
                    Wins1            35           38           35   
                    Wins7             7           10            7   
        [1000,3600) Gmean   1536.166586  1411.020662  1426.289646   
                    Wins1            11           12           11   
                    Wins7             3            4            3   

criterion                      Nodes (\#)                                \
type                                 Gur1           Gur7              V   
class   bucket      metric                                                
All     [0,3600)    Gmean      8046.91988    56

## Table 6: `all_bb_results_df`: all time/nodes results

In [85]:
inst_set = selected_time_df.index.levels[0]
inst_set.set_names("Instance",inplace=True)
numcuts_col_header = '# cuts'

col_idx = pd.MultiIndex.from_arrays(
    [
        ['', '', numcuts_col_header] + [time_col_header]*len(time_cols_short) + [node_col_header]*len(node_cols_short),
        ['Rows', 'Cols', map_cols_to_short_time[gur1vtime_col]] + time_cols_short + node_cols_short
    ],
)

all_bb_results_df = pd.DataFrame(
    columns = col_idx,
    index = inst_set,
    dtype = object,
)

# Enter number of rows and cols
tmp_df = df.xs(0, level='disj_terms').loc[inst_set,['ROWS','COLS']]
tmp_df.columns = pd.MultiIndex.from_product([[''],['Rows','Cols']])
all_bb_results_df.loc[:,tmp_df.columns] = tmp_df

# Enter number of cuts
# tmp_df = selected_time_df.loc[(inst_set,0), ['NUM VPC']]
tmp_df = selected_time_df.xs(0, level='disj_terms')['NUM VPC']
tmp_df.columns = pd.MultiIndex.from_product([[numcuts_col_header],[map_cols_to_short_time[gur1vtime_col]]])
all_bb_results_df.loc[:,tmp_df.columns] = tmp_df

# Enter time
tmp_df = selected_time_df.xs(0, level='disj_terms')[time_cols_long]
tmp_df.columns = pd.MultiIndex.from_product([[time_col_header],time_cols_short])
all_bb_results_df.loc[:,tmp_df.columns] = tmp_df

# Enter nodes
tmp_df = selected_time_df.xs(0, level='disj_terms')[node_cols_long]
tmp_df.columns = pd.MultiIndex.from_product([[node_col_header],node_cols_short])
all_bb_results_df.loc[:,tmp_df.columns] = tmp_df

all_bb_results_df = all_bb_results_df.sort_values(by=[(time_col_header, map_cols_to_short_time[mintime_col])])

# Add average + wins rows
# Replace missing entries with empty string
tmp_df = avg_bb_df.xs((bb_classes[0],bb_buckets[0])).copy(deep=True)
tmp_df.drop(inst_col_name, axis=1, level=0, inplace=True)
all_bb_results_df = pd.concat([all_bb_results_df, tmp_df]).fillna('',downcast=False)

# Remove unnecessary entries
all_bb_results_df.loc['Wins1',[
        (time_col_header,map_cols_to_short_time[gur1time_col]),
        (node_col_header,map_cols_to_short_nodes[gur1nodes_col])
    ]] = ""
# all_bb_results_df.loc['Wins1',([time_col_header,node_col_header],'Gur1')] = ""
all_bb_results_df.loc['Wins7',[
        (time_col_header,map_cols_to_short_time[gur1time_col]),
        (time_col_header,map_cols_to_short_time[gur7time_col]),
        (node_col_header,map_cols_to_short_nodes[gur1nodes_col]),
        (node_col_header,map_cols_to_short_nodes[gur7nodes_col]),
    ]] = ""
# all_bb_results_df.loc['Wins7',([time_col_header,node_col_header],['Gur1','Gur7'])] = ""
# all_bb_results_df = all_bb_results_df.fillna('',downcast=False)

# Convert rows, cols, # cuts to int values
tmp_cols = pd.MultiIndex.from_product([[''],['Rows','Cols']])
all_bb_results_df.loc[inst_set,tmp_cols] = all_bb_results_df.loc[inst_set,tmp_cols].astype(np.int64)
tmp_cols = pd.MultiIndex.from_product([[numcuts_col_header],[map_cols_to_short_time[gur1vtime_col]]])
all_bb_results_df.loc[inst_set,tmp_cols] = all_bb_results_df.loc[inst_set,tmp_cols].astype(np.int64)

# Rename inst col back to inst_row_name
all_bb_results_df.index.set_names("Instance",inplace=True)

display(all_bb_results_df.head(15))
display(all_bb_results_df.tail(10))

/var/folders/_8/bgx4yr0n6f1dm56pnvsqqb1s91dzml/T/ipykernel_17114/3348511383.py:21: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  all_bb_results_df.loc[:,tmp_df.columns] = tmp_df
/var/folders/_8/bgx4yr0n6f1dm56pnvsqqb1s91dzml/T/ipykernel_17114/3348511383.py:27: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  all_bb_results_df.loc[:,tmp_df.columns] = tmp_df
/var/folders/_8/bgx4yr0n6f1dm56pnvsqqb1s91dzml/T/ipykernel_17114/3348511383.py:32: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace i

# cuts Time (s)                       \
                         Rows  Cols      V     Gur1   Gur7      V  Total   
Instance                                                                   
neos-796608_presolved      64   104      2    0.004  0.002  0.003  0.063   
neos-501453_presolved      13    52      0    0.006  0.003  0.006  0.006   
vpm1_presolved            128   188     11    0.013  0.013  0.011  0.041   
gt2_presolved              28   173      1    0.013  0.007  0.012  0.032   
set1cl_presolved          431   651      2    0.021  0.018  0.019   0.12   
nexp-50-20-1-1_presolved  267   443      0    0.021  0.021  0.021  0.081   
sp150x300d_presolved      269   419      2    0.034  0.027  0.027  0.067   
p0548_presolved           119   371      3    0.032  0.032   0.03  0.132   
pipex_presolved            25    48      6    0.055  0.044  0.031  0.051   
roy_presolved             147   139      0    0.032  0.032  0.032  0.072   
set1al_presolved          432   652      3    0.053   0.05  0.036  0.161   
haprp_presolved           694   756      3    0.056  0.027  0.044  0.404   
f2gap201600_presolved      20  1600     16    0.062   0.06   0.05   0.84   
f2gap401600_presolved      40  1600     29    0.072  0.069  0.053  1.194   
f2gap801600_presolved      80  1600      0    0.062   0.06  0.072  1.526   

                                       Nodes (\#)             
                             V7 Total7       Gur1 Gur7  V V7  
Instance                                                      
neos-796608_presolved     0.003  0.004          1    1  1  1  
neos-501453_presolved     0.006  0.006          1    1  1  1  
vpm1_presolved            0.011  0.013          1    1  1  1  
gt2_presolved             0.012  0.013          1    1  1  1  
set1cl_presolved          0.019  0.021          1    1  1  1  
nexp-50-20-1-1_presolved  0.021  0.021          1    1  1  1  
sp150x300d_presolved      0.027  0.034          1    1  1  1  
p0548_presolved            0.03  0.032          1    1  1  1  
pipex_presolved           0.031  0.051         12    1  1  1  
roy_presolved             0.032  0.032          1    1  1  1  
set1al_presolved          0.036  0.053          2    2  2  2  
haprp_presolved           0.044  0.056          1    1  1  1  
f2gap201600_presolved      0.05  0.062          1    1  1  1  
f2gap401600_presolved     0.053  0.072          1    1  1  1  
f2gap801600_presolved     0.062  0.062          1    1  1  1

# cuts    Time (s)             \
                             Rows  Cols      V        Gur1       Gur7   
Instance                                                                
hgms-det_presolved           4599   950      0    3600.001   3600.001   
ic97_potential_presolved      998   726      2    3600.003   3600.001   
neos-3024952-loue_presolved  3633  3218      9    3600.003     3600.0   
queens-30_presolved           900   900    597    3600.004   3600.001   
seymour_presolved            4369   893      0    3600.001     3600.0   
cod105_presolved             1024  1024    401    3600.022     3600.0   
cvs16r70-62_presolved        3278  2112    276    3600.005   3600.001   
Gmean                                           122.549348  92.121528   
Wins1                                                             193   
Wins7                                                                   

                                                                            \
                                      V       Total         V7      Total7   
Instance                                                                     
hgms-det_presolved             3600.001    3607.181   3600.001    3600.001   
ic97_potential_presolved       3600.001    3601.002   3600.001    3600.003   
neos-3024952-loue_presolved    3600.001    3603.722   3600.001    3600.003   
queens-30_presolved            3600.001    3846.484   3600.001    3600.004   
seymour_presolved              3600.001    3610.742   3600.001    3600.001   
cod105_presolved               3600.002    4588.402   3600.002    3600.022   
cvs16r70-62_presolved          3600.004    4241.964   3600.004    3600.005   
Gmean                        104.404089  122.846078  97.418226  101.608059   
Wins1                               184          89        184          89   
Wins7                                68          21         68          21   

                             Nodes (\#)                                         
                                   Gur1         Gur7            V           V7  
Instance                                                                        
hgms-det_presolved               156058       156058       221399       156058  
ic97_potential_presolved         496930       454616       497504       496930  
neos-3024952-loue_presolved      599384       469842       561129       561129  
queens-30_presolved               39184        23053        24324        24324  
seymour_presolved                 42230        39814        40012        40012  
cod105_presolved                   2000         1417         1642         1642  
cvs16r70-62_presolved             14224        12594        11967        11967  
Gmean                        8046.91988  5629.881079  6345.310699  5978.223166  
Wins1                                            227          207          207  
Wins7                                                          95           95

## Table 7: ``all6_bb_results_df``: 6-trees time/nodes results

In [86]:
inst_set = all6_instances_dict.keys()
all6_bb_results_df = all_bb_results_df.loc[inst_set]

all6_bb_results_df = all6_bb_results_df.sort_values(by=[(time_col_header, map_cols_to_short_time[mintime_col])])

# Add average + wins rows
# Replace missing entries with empty string
tmp_df = avg_bb_df.xs((bb_classes[1],bb_buckets[0])).copy(deep=True)
tmp_df.drop(inst_col_name, axis=1, level=0, inplace=True)
all6_bb_results_df = pd.concat([all6_bb_results_df, tmp_df]).fillna('',downcast=False)

# Remove unnecessary entries
all6_bb_results_df.loc['Wins1',[
        (time_col_header,map_cols_to_short_time[gur1time_col]),
        (node_col_header,map_cols_to_short_nodes[gur1nodes_col])
    ]] = ""
all6_bb_results_df.loc['Wins7',[
        (time_col_header,map_cols_to_short_time[gur1time_col]),
        (time_col_header,map_cols_to_short_time[gur7time_col]),
        (node_col_header,map_cols_to_short_nodes[gur1nodes_col]),
        (node_col_header,map_cols_to_short_nodes[gur7nodes_col]),
    ]] = ""

# Convert rows, cols, # cuts to int values
tmp_cols = pd.MultiIndex.from_product([[''],['Rows','Cols']])
all6_bb_results_df.loc[inst_set,tmp_cols] = all6_bb_results_df.loc[inst_set,tmp_cols].astype(np.int64)
tmp_cols = pd.MultiIndex.from_product([[numcuts_col_header],[map_cols_to_short_time[gur1vtime_col]]])
all6_bb_results_df.loc[inst_set,tmp_cols] = all6_bb_results_df.loc[inst_set,tmp_cols].astype(np.int64)

# Rename inst col back to inst_row_name
all6_bb_results_df.index.set_names("Instance",inplace=True)

display(all6_bb_results_df.head(15))
display(all6_bb_results_df.tail(10))

/var/folders/_8/bgx4yr0n6f1dm56pnvsqqb1s91dzml/T/ipykernel_17114/1995339031.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  tmp_df = avg_bb_df.xs((bb_classes[1],bb_buckets[0])).copy(deep=True)


# cuts Time (s)                       \
                         Rows  Cols      V     Gur1   Gur7      V  Total   
Instance                                                                   
neos-501453_presolved      13    52      0    0.006  0.003  0.006  0.006   
vpm1_presolved            128   188     11    0.013  0.013  0.011  0.041   
gt2_presolved              28   173      1    0.013  0.007  0.012  0.032   
set1cl_presolved          431   651      2    0.021  0.018  0.019   0.12   
nexp-50-20-1-1_presolved  267   443      0    0.021  0.021  0.021  0.081   
sp150x300d_presolved      269   419      2    0.034  0.027  0.027  0.067   
p0548_presolved           119   371      3    0.032  0.032   0.03  0.132   
pipex_presolved            25    48      6    0.055  0.044  0.031  0.051   
roy_presolved             147   139      0    0.032  0.032  0.032  0.072   
set1al_presolved          432   652      3    0.053   0.05  0.036  0.161   
haprp_presolved           694   756      3    0.056  0.027  0.044  0.404   
f2gap201600_presolved      20  1600     16    0.062   0.06   0.05   0.84   
f2gap401600_presolved      40  1600     29    0.072  0.069  0.053  1.194   
p0282_presolved           157   199     24    0.099  0.097   0.07  0.113   
mod008_presolved            6   319      6    0.076  0.069  0.071  0.199   

                                       Nodes (\#)             
                             V7 Total7       Gur1 Gur7  V V7  
Instance                                                      
neos-501453_presolved     0.006  0.006          1    1  1  1  
vpm1_presolved            0.011  0.013          1    1  1  1  
gt2_presolved             0.012  0.013          1    1  1  1  
set1cl_presolved          0.019  0.021          1    1  1  1  
nexp-50-20-1-1_presolved  0.021  0.021          1    1  1  1  
sp150x300d_presolved      0.027  0.034          1    1  1  1  
p0548_presolved            0.03  0.032          1    1  1  1  
pipex_presolved           0.031  0.051         12    1  1  1  
roy_presolved             0.032  0.032          1    1  1  1  
set1al_presolved          0.036  0.053          2    2  2  2  
haprp_presolved           0.044  0.056          1    1  1  1  
f2gap201600_presolved      0.05  0.062          1    1  1  1  
f2gap401600_presolved     0.053  0.072          1    1  1  1  
p0282_presolved            0.07  0.099          1    1  1  1  
mod008_presolved          0.071  0.076          4    1  1  1

# cuts   Time (s)             \
                              Rows  Cols      V       Gur1       Gur7   
Instance                                                                
nobel-eu-DBE_presolved         726  3460      0   1973.599   1973.599   
neos-1616732_presolved        1026   200     59   2907.053   2518.274   
iis-bupa-cov_presolved        4796   337     91   3600.001   3213.521   
cost266-UUE_presolved         1302  3882     34   3600.003   3600.001   
neos-3762025-ognon_presolved  2636  4507      1   3600.001     3600.0   
neos-3373491-avoca_presolved  1267  2152      0     3600.0    674.521   
maxgasflow_presolved          3935  4125      4   3600.021   2876.013   
Gmean                                            70.495418  53.406876   
Wins1                                                             132   
Wins7                                                                   

                                                                         \
                                      V     Total         V7     Total7   
Instance                                                                  
nobel-eu-DBE_presolved         2735.961  2737.721   1973.599   1973.599   
neos-1616732_presolved         2352.922  2354.522   2352.922   2354.522   
iis-bupa-cov_presolved         2979.352  2987.282   2979.352   2987.282   
cost266-UUE_presolved          3026.609  3045.189   3026.609   3045.189   
neos-3762025-ognon_presolved     3600.0   3602.58     3600.0   3600.001   
neos-3373491-avoca_presolved     3600.0   3600.56     3600.0     3600.0   
maxgasflow_presolved           3600.001  3602.213   3600.001   3600.021   
Gmean                         57.905113  60.65371  55.637828  57.293347   
Wins1                               127        67        127         67   
Wins7                                50        15         50         15   

                               Nodes (\#)                            \
                                     Gur1         Gur7            V   
Instance                                                              
nobel-eu-DBE_presolved             221308       221308       345286   
neos-1616732_presolved            1208038      1017810       945072   
iis-bupa-cov_presolved             207837       179734       130889   
cost266-UUE_presolved               73229        55668        62683   
neos-3762025-ognon_presolved       920777       620044       630254   
neos-3373491-avoca_presolved      5146164       510074      2728532   
maxgasflow_presolved               134333       117350       125176   
Gmean                         6594.443364  4882.944212  5181.375266   
Wins1                                              144          136   
Wins7                                                            68   

                                           
                                       V7  
Instance                                   
nobel-eu-DBE_presolved             221308  
neos-1616732_presolved             945072  
iis-bupa-cov_presolved             130889  
cost266-UUE_presolved               62683  
neos-3762025-ognon_presolved       630254  
neos-3373491-avoca_presolved      2728532  
maxgasflow_presolved               125176  
Gmean                         5003.320856  
Wins1                                 136  
Wins7                                  68

## Table 8: `avg_bb_by_depth_df`: average time/nodes by depth

In [87]:
## Prepare avg_bb_by_depth_df
## Prepare variables for row/col names

bb_classes_by_depth = [str(t) + ' leaves' for t in sizes]
num_bb_classes_by_depth = len(bb_classes_by_depth)

bb_buckets_by_depth = bb_buckets
bb_metrics_by_depth = bb_metrics[0:2]

cols_time_by_depth       = [gur1time_col, gur1vtime_col, gur1v_w_cut_time_col]
shortcols_time_by_depth  = [map_cols_to_short_time[col] for col in cols_time_by_depth]
cols_nodes_by_depth      = [gur1nodes_col, gur1vnodes_col]
shortcols_nodes_by_depth = [map_cols_to_short_nodes[col] for col in cols_nodes_by_depth]

avg_bb_cols_by_depth = pd.MultiIndex.from_arrays(
    [[time_col_header]*len(shortcols_time_by_depth) + 
     [node_col_header]*len(shortcols_nodes_by_depth), 
     shortcols_time_by_depth + shortcols_nodes_by_depth],
    names = ['criterion', 'type'])

# bucket_min = [0, 10, 100, 1000]
# bucket_max = [3600, 3600, 3600, 3600]
# num_buckets = len(bucket_min)
# assert(len(bucket_max) == num_buckets)
# bb_buckets = ['[' + str(bucket_min[j]) + ',' + str(bucket_max[j]) + ')' for j in range(num_buckets)]
# # bucket_names = [classes[i] + ' [' + str(bucket_min[j]) + ',' + str(bucket_max[j]) + ')' for i in range(num_classes) for j in range(num_buckets)]
# # display(bucket_names)

# bb_metrics = ['Gmean', 'Wins1', 'Wins7']

# time_col_header = 'Time (s)'
# node_col_header = 'Nodes (\\#)'

#bb_row_names = pd.MultiIndex.from_product([bb_buckets, bb_row_names], names=['bucket', 'metric'])
bb_row_names_by_depth = pd.MultiIndex.from_product(
    [bb_classes_by_depth, bb_buckets_by_depth, bb_metrics_by_depth],
    names=['class', 'bucket', 'metric'])

avg_bb_by_depth_df = pd.DataFrame(
    columns = avg_bb_cols_by_depth,
    index = bb_row_names_by_depth,
    dtype = float
)

# Fill in values for Gur1 from avg_bb_df
# display(
#     avg_bb_df.loc[
#         (bb_classes[1], bb_buckets, bb_metrics[0:2]),
#         [(time_col_header,map_cols_to_short_time[gur1time_col]),
#         (node_col_header,map_cols_to_short_nodes[gur1nodes_col])]
#     ]
# )

# Make all columns "object" type to allow for integer values
avg_bb_by_depth_df.loc[:,(time_col_header,shortcols_time_by_depth)] = avg_bb_by_depth_df.loc[:,(time_col_header,shortcols_time_by_depth)].astype(object)
avg_bb_by_depth_df.loc[:,(node_col_header,shortcols_nodes_by_depth)] = avg_bb_by_depth_df.loc[:,(node_col_header,shortcols_nodes_by_depth)].astype(object)

## Create gmean_df by depth
#   = shifted geometric mean of time taken across instances, in various buckets
#     and geomean of nodes too

num_inst_by_depth = np.zeros(len(avg_bb_by_depth_df),dtype = np.int64)
row_ind = 0

cols = cols_time_by_depth + cols_nodes_by_depth
shortcols = shortcols_time_by_depth + shortcols_nodes_by_depth

# Calculate stats for 6 trees instances by depth
curr_df = selected_time_df.loc[all6_instances_dict.keys(),cols]
for curr_size_ind in range(0,len(bb_classes_by_depth)):
    # print("{}".format(bb_classes_by_depth[curr_size_ind]))
    curr_by_depth_df = curr_df[curr_df.index.get_level_values(1) == sizes[curr_size_ind]] # take only best values

    for i in range(num_buckets):
        curr_by_depth_df = curr_by_depth_df[curr_by_depth_df[gur1time_col] > bucket_min[i]]
        avg_bb_by_depth_df.loc[
                (bb_classes_by_depth[curr_size_ind], bb_buckets_by_depth[i], bb_metrics_by_depth[0]),
                (time_col_header,shortcols_time_by_depth)] = \
            [geometric_mean(curr_by_depth_df[col] + SHIFT_TIME) - SHIFT_TIME for col in cols_time_by_depth]

        # display(avg_bb_by_depth_df.loc[
        #         (bb_classes_by_depth[curr_size_ind], bb_buckets_by_depth[i], bb_metrics_by_depth[0]),
        #         (time_col_header,shortcols_time_by_depth)].head())
        avg_bb_by_depth_df.loc[
                (bb_classes_by_depth[curr_size_ind], bb_buckets_by_depth[i], bb_metrics_by_depth[0]),
                (node_col_header,shortcols_nodes_by_depth)] = \
            [geometric_mean(curr_by_depth_df[col] + SHIFT_NODES) - SHIFT_NODES for col in cols_nodes_by_depth]
        
        # print("row {:d}: {:d}".format(row_ind,len(curr_by_depth_df)))

        num_inst_by_depth[row_ind:row_ind+len(bb_metrics_by_depth)] = len(bb_metrics_by_depth)*[len(curr_by_depth_df)]
        row_ind += len(bb_metrics_by_depth)

        ## Update wins1 rows
        # A win in terms of time is counted when the ``Gur1'' baseline seconds taken 
        # is at least 10\% slower, to account for some variability in runtimes.
        # A win in terms of nodes is when the ``Gur1'' baseline number of nodes is higher.
        refcol = gur1time_col
        avg_bb_by_depth_df.loc[
                (bb_classes_by_depth[curr_size_ind], bb_buckets_by_depth[i], bb_metrics_by_depth[1]),
                (time_col_header,shortcols_time_by_depth)] = \
            [ int(sum(curr_by_depth_df[refcol] > 1.1*curr_by_depth_df[col])) for col in cols_time_by_depth ]

        refcol = gur1nodes_col
        avg_bb_by_depth_df.loc[
                (bb_classes_by_depth[curr_size_ind], bb_buckets_by_depth[i], bb_metrics_by_depth[1]),
                (node_col_header,shortcols_nodes_by_depth)] = \
            [ int(sum(curr_by_depth_df[refcol] > curr_by_depth_df[col])) for col in cols_nodes_by_depth ]

avg_bb_by_depth_df[inst_col_name] = num_inst_by_depth

# for i in range(num_buckets):
#     curr_df = curr_df[curr_df[gur1time_col] > bucket_min[i]]
    
display(avg_bb_by_depth_df.loc[(bb_classes_by_depth, bb_buckets_by_depth, bb_metrics_by_depth),:])

criterion                        Time (s)                            \
type                                 Gur1            V        Total   
class     bucket      metric                                          
2 leaves  [0,3600)    Gmean     70.495418    73.232937    74.060773   
                      Wins1             0           54           35   
          [10,3600)   Gmean    251.035865   263.698515   265.379685   
                      Wins1             0           32           30   
          [100,3600)  Gmean     694.57839   660.934387   662.759412   
                      Wins1             0           19           19   
          [1000,3600) Gmean   2041.368928  2077.617323  2081.249353   
                      Wins1             0            5            5   
4 leaves  [0,3600)    Gmean     70.495418    73.272412    75.193851   
                      Wins1             0           65           37   
          [10,3600)   Gmean    251.035865   264.086386   267.453861   
                      Wins1             0           36           31   
          [100,3600)  Gmean     694.57839    684.99047   689.257652   
                      Wins1             0           20           20   
          [1000,3600) Gmean   2041.368928  2167.600435  2175.859921   
                      Wins1             0            5            5   
8 leaves  [0,3600)    Gmean     70.495418    74.040279    79.588816   
                      Wins1             0           54           24   
          [10,3600)   Gmean    251.035865   267.833528    279.65738   
                      Wins1             0           29           22   
          [100,3600)  Gmean     694.57839   716.810547   733.012739   
                      Wins1             0           18           17   
          [1000,3600) Gmean   2041.368928  2128.875978  2149.715901   
                      Wins1             0            4            4   
16 leaves [0,3600)    Gmean     70.495418    72.260878    87.859955   
                      Wins1             0           53           26   
          [10,3600)   Gmean    251.035865   257.861238   291.311439   
                      Wins1             0           29           25   
          [100,3600)  Gmean     694.57839   695.452193   747.041811   
                      Wins1             0           16           15   
          [1000,3600) Gmean   2041.368928  1822.614251  1883.842152   
                      Wins1             0            6            5   
32 leaves [0,3600)    Gmean     70.495418    77.128721   116.610659   
                      Wins1             0           57           16   
          [10,3600)   Gmean    251.035865   283.113817   375.420646   
                      Wins1             0           28           16   
          [100,3600)  Gmean     694.57839   797.666791   948.796649   
                      Wins1             0           15           10   
          [1000,3600) Gmean   2041.368928  2507.588443  2712.556708   
                      Wins1             0            3            1   
64 leaves [0,3600)    Gmean     70.495418    77.310769   157.981786   
                      Wins1             0           44           10   
          [10,3600)   Gmean    251.035865    284.16288     477.4119   
                      Wins1             0           26           10   
          [100,3600)  Gmean     694.57839   810.639657  1152.755444   
                      Wins1             0           13            8   
          [1000,3600) Gmean   2041.368928  2345.895524  2816.911002   
                      Wins1             0            3            2   

criterion                        Nodes (\#)                # inst  
type                                   Gur1              V         
class     bucket      metric                                       
2 leaves  [0,3600)    Gmean     6594.443364    6586.432231    196  
                      Wins1               0             86    196  
          [10,3600)   Gmean    50114.5

# Section 4: Objective and time analysis

## `obj_and_time_df`: objectives, successes, fails, and time per obj or cut

In [88]:
inst_set = best_gap_df.index
# inst_set = ['10teams_presolved', 'bm23_presolved', 'vpm1_presolved']

# Define rows to add
inst_depth_set = [(inst, best_gap_df.loc[inst, 'BEST VPC DISJ']) for inst in inst_set]

# Define columns to add
fail_rate_col_name = 'Fail rate (%)'
time_col_name = 'Time (s)'
sec_per_obj_col_name = '(s) / obj'
sec_per_cut_col_name = '(s) / cut'
obj_and_time_new_cols = [
    fail_rate_col_name,
    time_col_name,
    sec_per_obj_col_name,
    sec_per_cut_col_name,
]

obj_and_time_df = df.loc[inst_depth_set,['NUM OBJ', 'NUM CUTS', 'NUM FAILS']].copy(deep=True)
obj_and_time_df[fail_rate_col_name] = 100. * obj_and_time_df['NUM FAILS'] / obj_and_time_df['NUM OBJ']
obj_and_time_df[time_col_name] = df['VPC_GEN_TIME']
obj_and_time_df[sec_per_obj_col_name] = obj_and_time_df[time_col_name] / obj_and_time_df['NUM OBJ']
obj_and_time_df[sec_per_cut_col_name] = obj_and_time_df[time_col_name] / obj_and_time_df['NUM CUTS']

# Replace Fail rate = NaN when all cuts are one-sided cuts
SKIP_CHAR = '-'
obj_and_time_df.fillna(SKIP_CHAR, inplace = True)
obj_and_time_df.replace(np.inf, SKIP_CHAR, inplace = True)

# Add average row
# obj_and_time_df.loc['Average'] = 0
obj_and_time_df.loc['Average', obj_and_time_new_cols] =\
    [obj_and_time_df[obj_and_time_df[col] != SKIP_CHAR][col].mean() for col in obj_and_time_new_cols]
# for col in obj_and_time_new_cols:
#     obj_and_time_df.at[('Average',0),col] =\
#         obj_and_time_df[obj_and_time_df[col] != SKIP_CHAR][col].mean()

obj_and_time_df.loc['Average',['NUM OBJ', 'NUM CUTS', 'NUM FAILS']] = ""

display(obj_and_time_df)
# obj_and_time_df[obj_and_time_df['NUM CUTS'] == 0]
# obj_and_time_df[obj_and_time_df['(s) / obj'] > 100000]
# obj_and_time_df.loc['neos18_presolved']
# obj_and_time_df[obj_and_time_df['(s) / obj'] != SKIP_CHAR]['(s) / obj'].max()

,,NUM OBJ,NUM CUTS,NUM FAILS,Fail rate (%),Time (s),(s) / obj,(s) / cut
INSTANCE,disj_terms,,,,,,,
10teams_presolved,8,322.0,51.0,271.0,84.161491,1542.09000,4.789099,30.237059
23588_presolved,64,76.0,75.0,1.0,1.315789,125.63000,1.653026,1.675067
30n20b8_presolved,4,198.0,190.0,8.0,4.040404,638.67000,3.225606,3.361421
50v-10_presolved,64,30.0,29.0,1.0,3.333333,21.41000,0.713667,0.738276
a1c1s1_presolved,8,3.0,3.0,0.0,0.0,2.67000,0.89,0.890000
...,...,...,...,...,...,...,...,...
usAbbrv-8-25_70_presolved,8,4.0,4.0,0.0,0.0,2.80000,0.7,0.700000
vpm1_presolved,32,34.0,10.0,24.0,70.588235,0.29000,0.008529,0.029000
vpm2_presolved,64,36.0,25.0,11.0,30.555556,0.91000,0.025278,0.036400


In [89]:
### DEBUG finding max difference in time between TOTAL_TIME and sum of individual times
# cuts_cols = [col for col in df.columns if col.startswith('NUM CUTS')]
# time_cols = [
#     'INIT_SOLVE_TIME',
#     'VPC_GEN_TIME',
#     'VPC_APPLY_TIME',
#     'BB_TIME',
#     'TOTAL_TIME'
# ]
# display(df.loc['bell3b_presolved',['NUM OBJ', 'NUM FAILS'] + cuts_cols])

# obj_and_time_df = df.loc[inst_depth_set].copy(deep = True)['NUM OBJ', 'NUM CUTS', 'NUM FAILS', 'VPC_GEN_TIME']
# display(obj_and_time_df)

# max_diff_time = 0.
# max_diff_inst = ''
# for inst in best_gap_df.index:
#     depth = best_gap_df.loc[inst, 'BEST VPC DISJ']
#     curr_num_obj   = df.loc[(inst,depth)]['NUM OBJ']
#     curr_num_vpc   = df.loc[(inst,depth)]['NUM VPC']
#     curr_num_1side = df.loc[(inst,depth)]['NUM CUTS ONE_SIDED']
#     curr_num_fails = df.loc[(inst,depth)]['NUM FAILS']
#     if curr_num_vpc + curr_num_fails != curr_num_obj + curr_num_1side:
#         raise ValueError("{}: curr_num_vpc ({:d}) + curr_num_fails ({:d}) != curr_num_obj ({:d}) + curr_num_1side ({:d})".format(inst, curr_num_vpc, curr_num_fails, curr_num_obj, curr_num_1side))
    
#     curr_fail_pct = 100. * curr_num_fails / curr_num_obj
#     curr_init_solve = df.loc[(inst,depth)]['INIT_SOLVE_TIME']
#     curr_vpc_gen = df.loc[(inst,depth)]['VPC_GEN_TIME']
#     curr_vpc_apply = df.loc[(inst,depth)]['VPC_APPLY_TIME']
#     curr_bb_time = df.loc[(inst,depth)]['BB_TIME']
#     curr_total_time = df.loc[(inst,depth)]['TOTAL_TIME']

#     curr_diff_time = curr_total_time - (curr_init_solve + curr_vpc_gen + curr_vpc_apply + curr_bb_time)
#     if curr_diff_time < -EPS:
#         display(df.loc[inst,time_cols])
#         raise ValueError("{} (depth {:d}): curr_diff_time {} < 0.".format(inst,depth,curr_diff_time))
    
#     if max_diff_time < curr_diff_time:
#         max_diff_inst = inst
#         max_diff_time = curr_diff_time

# print("Max diff time = {} for inst {}".format(max_diff_time,max_diff_inst))
# display(df.loc[(max_diff_inst,best_gap_df.loc[max_diff_inst, 'BEST VPC DISJ']),time_cols])


## `best_disj_gap_df`: Number of times a particular depth achieves best result and beats baseline by at least EPS

In [90]:
long_cols_to_compare = {
    col_vpc:         col_gmic,
    col_vpc_gmic:    col_gmic,
    col_first_ref_v: col_first_ref,
    col_last_ref_v:  col_last_ref,
}
short_cols_to_compare = [map_cols_to_short_gap[col] for col in long_cols_to_compare.keys()]
row_no_improvement = 'No improvement'

best_disj_gap_df = pd.DataFrame(
    columns = short_cols_to_compare,
    index = [row_no_improvement] + [0] + sizes,
    dtype = int,
)

for curr_depth in [0] + sizes:
    curr_depth_df = selected_gap_df.xs(curr_depth,level='disj_terms')
    for col in long_cols_to_compare.keys():
        shortcol = map_cols_to_short_gap[col]
        refcol = long_cols_to_compare[col]
        
        # Calculate num times this depth yielded the best result
        curr_num_wins = sum(
            (curr_depth_df[col] == best_gap_df[shortcol]) & 
            (best_gap_df[shortcol] > best_gap_df[map_cols_to_short_gap[refcol]] + EPS)
        )
        best_disj_gap_df.at[curr_depth,shortcol] = curr_num_wins
        
# Add no improvement row
curr_depth = row_no_improvement
for col in long_cols_to_compare.keys():
    shortcol = map_cols_to_short_gap[col]
    refcol = long_cols_to_compare[col]

    # Calculate num times no improvement over the baseline
    curr_num_wins = sum(
            (best_gap_df[shortcol] <= best_gap_df[map_cols_to_short_gap[refcol]] + EPS)
        )
    best_disj_gap_df.at[curr_depth,shortcol] = curr_num_wins


# Reindex to add "leaves" to index
idx = [row_no_improvement] + ['Best'] + [str(size) + " leaves" for size in sizes]
reidx = {old_id : new_id for old_id, new_id in zip(best_disj_gap_df.index,idx)}
best_disj_gap_df.rename(reidx, inplace=True)

# Remove best row (it is good to verify this is the same as the relevant entries in win_df or Table 1)
# best_disj_gap_df.drop('Best', axis=0, inplace=True)

# Make sure all cols are int
best_disj_gap_df = best_disj_gap_df.astype(int)

best_disj_gap_df

,V,V+G,V+GurF,V+GurL
No improvement,215,136,153,204
Best,113,192,175,124
2 leaves,0,9,35,15
4 leaves,4,8,42,25
8 leaves,4,22,40,24
16 leaves,6,23,47,22
32 leaves,18,35,55,20
64 leaves,81,109,85,57


## `best_disj_time_df`: Number of times depth is best and improvement is at least 10%

In [91]:
row_no_improvement = 'No improvement'

best_disj_time_df = pd.DataFrame(
    columns = bb_classes,
    index = [row_no_improvement] + [0] + sizes,
    dtype = int,
)

col = gur1vtime_col
# shortcol = (time_col_header,map_cols_to_short_time[col])
# destcol = map_cols_to_short_time[col]
refcol = gur1time_col

# Calculate num times this depth yielded the best result
best_time_df = selected_time_df.xs(0,level='disj_terms')
for curr_depth in [0] + sizes:
    # First test for all instances
    curr_depth_df = selected_time_df.xs(curr_depth,level='disj_terms')
    
    curr_num_wins = sum(
        (curr_depth_df[col] == best_time_df[col]) & 
        (best_time_df[refcol] >= 1.1 * best_time_df[col] + EPS)
    )
    best_disj_time_df.at[curr_depth,bb_classes[0]] = curr_num_wins

# Add no improvement row
# Calculate num times no improvement over the baseline
curr_depth = row_no_improvement
curr_num_wins = sum(
        (best_time_df[refcol] < 1.1 * best_time_df[col])
    )
best_disj_time_df.at[curr_depth,bb_classes[0]] = curr_num_wins

# Now check only 6-tree instances
best_time_df = best_time_df.loc[all6_instances_dict.keys()]
for curr_depth in [0] + sizes:
    curr_depth_df = selected_time_df.xs(curr_depth,level='disj_terms').loc[all6_instances_dict.keys()]

    # Calculate num times this depth yielded the best result
    curr_num_wins = sum(
        (curr_depth_df[col] == best_time_df[col]) & 
        (best_time_df[refcol] >= 1.1 * best_time_df[col] + EPS)
    )
    best_disj_time_df.at[curr_depth,bb_classes[1]] = curr_num_wins

# Add no improvement row
# Calculate num times no improvement over the baseline
curr_depth = row_no_improvement
curr_num_wins = sum(
        (best_time_df[refcol] < 1.1 * best_time_df[col])
    )
best_disj_time_df.at[curr_depth,bb_classes[1]] = curr_num_wins

# Reindex to add "leaves" to index
idx = [row_no_improvement] + ['Best'] + [str(size) + " leaves" for size in sizes]
reidx = {old_id : new_id for old_id, new_id in zip(best_disj_time_df.index,idx)}
best_disj_time_df.rename(reidx, inplace=True)

# Remove best row (it is good to verify this is the same as the relevant entries in win_df or Table 1)
# best_disj_time_df.drop('Best', axis=0, inplace=True)

# Make sure all cols are int
best_disj_time_df = best_disj_time_df.astype(int)

best_disj_time_df

,All,6 trees
No improvement,116,69
Best,184,127
2 leaves,48,30
4 leaves,33,23
8 leaves,35,21
16 leaves,27,19
32 leaves,31,27
64 leaves,16,13


## `density_df`

In [115]:
rows = [
    '\# inst w/VPCs and time < 3600s',
    '\# wins by time',
    'Avg min cut density',
    'Avg max cut density',
    'Avg avg cut density',
    'Avg avg cut density (win by time)',
    'Avg avg cut density (non-win)',
]

columns = ['V ({:d})'.format(size) for size in sizes]

density_df = pd.DataFrame(
    columns = columns,
    index = rows,
    dtype = float
)

# Calculate stats for 6 trees instances by depth
# inst_set = all6_instances_dict.keys()
inst_set = selected_time_instances_dict.keys()
support_cols = [col for col in df.columns if "SUPPORT VPC" in col]
selected_cols = [gur1time_col,gur1vtime_col]+support_cols+['COLS']+['NUM VPC']
curr_df = df.loc[inst_set,selected_cols]
curr_df0 = curr_df.xs(0,level='disj_terms')

for curr_size_ind in range(0,len(sizes)):
    # Select only this depth
    # curr_by_depth_df = curr_df[curr_df.index.get_level_values(1) == sizes[curr_size_ind]]
    curr_by_depth_df = curr_df.xs(sizes[curr_size_ind], level='disj_terms')
    
    # Remove instances that take more than an hour
    INSTANCES_TO_KEEP = curr_by_depth_df[gur1time_col] < MAX_TIME
    curr_by_depth_df = curr_by_depth_df[INSTANCES_TO_KEEP]

    # Count number of instances having cuts
    curr_row_ind = 0
    density_df.iloc[curr_row_ind,curr_size_ind] = sum(curr_by_depth_df['NUM VPC'] > 0)
    
    # Mean of min, max, avg density
    curr_row_ind = 2
    for col_ind in range(len(support_cols)):
        curr_series = curr_by_depth_df[support_cols[col_ind]] / curr_by_depth_df['COLS']
        density_df.iloc[curr_row_ind,curr_size_ind] = curr_series.mean()
        curr_row_ind += 1

    ## Count wins1 (should be same as in avg_bb_by_depth_df)
    # A win in terms of time is counted when the ``Gur1'' baseline seconds taken 
    # is at least 10\% slower, to account for some variability in runtimes.
    # A win in terms of nodes is when the ``Gur1'' baseline number of nodes is higher.
    curr_wins_df = curr_by_depth_df[curr_df0.loc[INSTANCES_TO_KEEP,gur1time_col] > 1.1*curr_by_depth_df[gur1vtime_col]]
    curr_row_ind = 1
    density_df.iloc[curr_row_ind,curr_size_ind] = len(curr_wins_df)
    curr_row_ind = len(rows)-2
    density_df.iloc[curr_row_ind,curr_size_ind] = (curr_wins_df[support_cols[2]] / curr_wins_df['COLS']).mean()

    # curr_lose_df = curr_by_depth_df[1.1*curr_df0[gur1time_col] < curr_by_depth_df[gur1vtime_col]]
    curr_lose_df = curr_by_depth_df[curr_df0.loc[INSTANCES_TO_KEEP,gur1time_col] <= 1.1*curr_by_depth_df[gur1vtime_col]]
    curr_row_ind = len(rows)-1
    density_df.iloc[curr_row_ind,curr_size_ind] = (curr_lose_df[support_cols[2]] / curr_lose_df['COLS']).mean()

density_df

,V (2),V (4),V (8),V (16),V (32),V (64)
\# inst w/VPCs and time < 3600s,251.000000,251.000000,242.000000,241.000000,222.000000,203.000000
\# wins by time,100.000000,112.000000,111.000000,101.000000,134.000000,136.000000
Avg min cut density,0.203375,0.221461,0.236918,0.265540,0.264891,0.258883
Avg max cut density,0.321949,0.348038,0.353024,0.379395,0.365441,0.330684
Avg avg cut density,0.262402,0.285691,0.299081,0.327858,0.325140,0.303824
Avg avg cut density (win by time),0.239998,0.247006,0.236776,0.239216,0.199517,0.148246
Avg avg cut density (non-win),0.275352,0.312435,0.340003,0.379311,0.436620,0.439456


## `obj_fails_df`

In [116]:
inst_set = best_gap_df.index
# inst_set = ['10teams_presolved', 'bm23_presolved', 'vpm1_presolved']

# Define rows to add
inst_depth_set = [(inst, best_gap_df.loc[inst, 'BEST VPC DISJ']) for inst in inst_set]

rows = [
    '\# inst w/obj',
    '\# inst w/succ obj',
    '\# inst no obj',
    '\# inst all obj fail',
    '\# inst all obj succ',
    '\% obj fails',
    '\% fails dup',
    '\% fails unbdd',
    '\% fails tilim',
    '\% fails dyn',
    '\% fails all ones',
    '\% fails post-GMIC obj',
    '\% fails DB',
    '\# obj / cut',
    '(s) / obj',
    '(s) / cut',
]

columns = ['V ({:d})'.format(size) for size in sizes]

obj_fails_df = pd.DataFrame(
    columns = columns,
    index = rows,
    dtype = float
)

# Choose columns to pull
selected_cols = [
    'NUM OBJ',
    'NUM VPC',
    'NUM FAILS DUMMY_OBJ',
    'NUM FAILS ALL_ONES',
    'NUM FAILS CUT_VERTICES',
    'NUM FAILS ITER_BILINEAR',
    'NUM FAILS UNIT_VECTORS',
    'NUM FAILS DISJ_LB',
    'NUM FAILS TIGHT_POINTS',
    'NUM FAILS TIGHT_RAYS',
    'NUM FAILS TIGHT_POINTS2',
    'NUM FAILS TIGHT_RAYS2',
    'NUM FAILS USER',
    'NUM FAILS OBJ_CUT',
    'NUM FAILS ONE_SIDED',
    'NUM FAILS',
    'ABANDONED',
    'BAD_DYNAMISM',
    'BAD_SUPPORT',
    'BAD_VIOLATION',
    'CUT_LIMIT',
    'DUAL_INFEASIBLE',
    'DUPLICATE_SIC',
    'DUPLICATE_VPC',
    'ITERATION_LIMIT',
    'ORTHOGONALITY_SIC',
    'ORTHOGONALITY_VPC',
    'PRIMAL_INFEASIBLE',
    'TIME_LIMIT',
    'NUMERICAL_ISSUES_WARNING',
    'DLB_EQUALS_DUB_NO_OBJ',
    'DLB_EQUALS_LPOPT_NO_OBJ',
    'PRIMAL_INFEASIBLE_NO_OBJ',
    'NUMERICAL_ISSUES_NO_OBJ',
    'UNKNOWN',
    'VPC_GEN_TIME',
]

curr_df = df.loc[inst_set,selected_cols]
curr_df0 = curr_df.xs(0,level='disj_terms')

for curr_size_ind in range(0,len(sizes)):
    # Select only this depth
    # curr_by_depth_df = curr_df[curr_df.index.get_level_values(1) == sizes[curr_size_ind]]
    curr_by_depth_df = curr_df.xs(sizes[curr_size_ind], level='disj_terms')

    # Num inst with objectives tried
    obj_fails_df.iloc[0,curr_size_ind] =\
        sum(curr_by_depth_df['NUM OBJ'] > 0)

    # Num inst with successful objectives
    obj_fails_df.iloc[1,curr_size_ind] =\
        sum((curr_by_depth_df['NUM OBJ'] > 0) & (curr_by_depth_df['NUM VPC'] > 0))

    # Num inst with no objectives tried
    obj_fails_df.iloc[2,curr_size_ind] =\
        sum(curr_by_depth_df['NUM OBJ'] == 0)

    # Num inst with objectives tried but all failed
    obj_fails_df.iloc[3,curr_size_ind] =\
        sum((curr_by_depth_df['NUM OBJ'] > 0) & (curr_by_depth_df['NUM VPC'] == 0))

    # Num inst with objectives tried and all succeeded
    obj_fails_df.iloc[4,curr_size_ind] =\
        sum((curr_by_depth_df['NUM OBJ'] > 0) & (curr_by_depth_df['NUM OBJ'] == curr_by_depth_df['NUM VPC']))

    # Percent objective failures
    inst_w_obj_df = curr_by_depth_df[curr_by_depth_df['NUM OBJ'] > 0]
    obj_fails_df.iloc[5,curr_size_ind] =\
        (100. * inst_w_obj_df['NUM FAILS'] / inst_w_obj_df['NUM OBJ']).mean()

    ## Percent of failures caused by:
    inst_w_fails_df = curr_by_depth_df[curr_by_depth_df['NUM FAILS'] > 0]

    # duplicates
    obj_fails_df.iloc[6,curr_size_ind] =\
        (100. * (inst_w_fails_df['DUPLICATE_VPC']+inst_w_fails_df['DUPLICATE_SIC']) / inst_w_fails_df['NUM FAILS']).mean()

    # unbdd
    obj_fails_df.iloc[7,curr_size_ind] =\
        (100. * (inst_w_fails_df['DUAL_INFEASIBLE']) / inst_w_fails_df['NUM FAILS']).mean()

    # tilim
    obj_fails_df.iloc[8,curr_size_ind] =\
        (100. * (inst_w_fails_df['TIME_LIMIT']) / inst_w_fails_df['NUM FAILS']).mean()
    
    # dynamism
    obj_fails_df.iloc[9,curr_size_ind] =\
        (100. * (inst_w_fails_df['BAD_DYNAMISM']) / inst_w_fails_df['NUM FAILS']).mean()

    # all-ones
    obj_fails_df.iloc[10,curr_size_ind] =\
        (100. * (inst_w_fails_df['NUM FAILS ALL_ONES']) / inst_w_fails_df['NUM FAILS']).mean()

    # post-GMIC
    obj_fails_df.iloc[11,curr_size_ind] =\
        (100. * (inst_w_fails_df['NUM FAILS ITER_BILINEAR']) / inst_w_fails_df['NUM FAILS']).mean()

    # disj_lb
    obj_fails_df.iloc[12,curr_size_ind] =\
        (100. * (inst_w_fails_df['NUM FAILS DISJ_LB']) / inst_w_fails_df['NUM FAILS']).mean()
    
    # num obj / cut
    inst_w_cuts_df = curr_by_depth_df[curr_by_depth_df['NUM VPC'] > 0]
    obj_fails_df.iloc[13,curr_size_ind] = (inst_w_cuts_df['NUM OBJ'] / inst_w_cuts_df['NUM VPC']).mean()

    # (s) / obj
    obj_fails_df.iloc[14,curr_size_ind] = (inst_w_obj_df['VPC_GEN_TIME'] / inst_w_obj_df['NUM OBJ']).mean()

    # (s) / cut
    obj_fails_df.iloc[15,curr_size_ind] = (inst_w_cuts_df['VPC_GEN_TIME'] / inst_w_cuts_df['NUM VPC']).mean()

obj_fails_df

,V (2),V (4),V (8),V (16),V (32),V (64)
\# inst w/obj,307.000000,308.000000,293.000000,296.000000,270.000000,246.000000
\# inst w/succ obj,305.000000,304.000000,289.000000,292.000000,264.000000,238.000000
\# inst no obj,21.000000,20.000000,35.000000,32.000000,58.000000,82.000000
\# inst all obj fail,2.000000,4.000000,4.000000,4.000000,6.000000,8.000000
\# inst all obj succ,29.000000,38.000000,31.000000,30.000000,23.000000,19.000000
\% obj fails,28.144407,25.594210,26.186943,32.128437,32.879383,35.251373
\% fails dup,49.432658,39.327509,40.066823,46.314982,48.426960,52.899623
\% fails unbdd,38.698520,47.758558,44.086669,32.729394,32.300856,29.092011
\% fails tilim,3.698543,3.630131,6.209332,11.923470,11.085985,9.198959
\% fails dyn,7.926075,8.529425,8.849262,8.432706,7.418308,8.557499


## `active_cuts_df`: when generated cuts are active, by objective

In [117]:
inst_set = best_gap_df.index
# inst_set = ['10teams_presolved', 'bm23_presolved', 'vpm1_presolved']

# Define rows to add
inst_depth_set = [(inst, best_gap_df.loc[inst, 'BEST VPC DISJ']) for inst in inst_set]

rows = [
    '\% active GMIC',
    '\% active VPC',
    '\% cuts one-sided',
    '\% active one-sided',
    '\% cuts all ones',
    '\% active all ones',
    '\% cuts post-GMIC opt',
    '\% active post-GMIC opt',
    '\% cuts DB',
    '\% active DB',
]

columns = ['V+G ({:d})'.format(size) for size in sizes]

active_cuts_df = pd.DataFrame(
    columns = columns,
    index = rows,
    dtype = float
)

# Choose columns to pull
active_gmic_col = 'ACTIVE GMIC (all cuts)'
active_vpc_col = 'ACTIVE VPC (all cuts)'
selected_cols = [
    'NUM GMIC',
    'NUM VPC',
    active_gmic_col,
    active_vpc_col,
] + [col for col in df.columns if "NUM CUTS " in col] + [col for col in df.columns if "NUM ACTIVE" in col]

curr_df = df.loc[inst_set,selected_cols]

num_inst_with_one_sided_cuts = [0 for size in sizes]
for curr_size_ind in range(0,len(sizes)):
    # Select only this depth
    # curr_by_depth_df = curr_df[curr_df.index.get_level_values(1) == sizes[curr_size_ind]]
    curr_by_depth_df = curr_df.xs(sizes[curr_size_ind], level='disj_terms')

    # active gmic
    active_cuts_df.iloc[0,curr_size_ind] =\
        (100. * curr_by_depth_df[active_gmic_col] / curr_by_depth_df['NUM GMIC']).mean()

    # active vpc
    active_cuts_df.iloc[1,curr_size_ind] =\
        (100. * curr_by_depth_df[active_vpc_col] / curr_by_depth_df['NUM VPC']).mean()

    # percent of active cuts among those generated by a specific objective type
    obj_types = ['ONE_SIDED', 'ALL_ONES', 'ITER_BILINEAR', 'DISJ_LB']
    curr_row_index = 2
    inst_w_vpc = curr_by_depth_df[curr_by_depth_df['NUM VPC'] > 0]
    for obj in obj_types:
        active_cuts_df.iloc[curr_row_index,curr_size_ind] =\
            (100. * inst_w_vpc['NUM CUTS '+obj] / inst_w_vpc['NUM VPC']).mean()
        curr_row_index += 1

        inst_w_cuts = inst_w_vpc[inst_w_vpc['NUM CUTS '+obj] > 0]
        
        active_cuts_df.iloc[curr_row_index,curr_size_ind] =\
            (100. * inst_w_cuts['NUM ACTIVE '+obj] / inst_w_cuts['NUM CUTS '+obj]).mean()
        curr_row_index += 1
    
    # num one-sided cuts
    num_inst_with_one_sided_cuts[curr_size_ind] = sum(curr_by_depth_df['NUM CUTS ONE_SIDED'] > 0)

display(active_cuts_df)

print("Num inst with one-sided cuts (should be same across partial trees) =",num_inst_with_one_sided_cuts)
print("Total num one-sided cuts =", sum(curr_by_depth_df['NUM CUTS ONE_SIDED']))

,V+G (2),V+G (4),V+G (8),V+G (16),V+G (32),V+G (64)
\% active GMIC,44.305691,43.376751,42.289000,41.678714,41.120183,41.267641
\% active VPC,30.242083,31.611173,32.078155,35.635763,35.253731,34.802351
\% cuts one-sided,0.775203,0.748656,0.770219,1.145125,0.880503,0.875300
\% active one-sided,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
\% cuts all ones,11.759815,6.853341,7.931222,8.649239,7.966688,9.229225
\% active all ones,91.095890,84.210526,81.021898,73.793103,79.166667,79.577465
\% cuts post-GMIC opt,2.647351,3.615188,2.323705,2.589037,2.235267,2.999685
\% active post-GMIC opt,87.012987,71.875000,66.666667,69.230769,64.583333,58.928571
\% cuts DB,84.817631,88.782814,88.974854,87.616599,88.917542,86.895791
\% active DB,63.683123,58.931399,55.645813,53.327108,50.723329,45.428098


Num inst with one-sided cuts (should be same across partial trees) = [5, 5, 5, 5, 5, 5]
Total num one-sided cuts = 6


# Section 5: Export tables to LaTeX

## Format Table 1: gap closed and num wins

In [119]:
# Format Table 1: gap closed and num wins

# Create copy of table then remove values we do not want (wins for 'G)
# TABLE1 = avg_df.copy(deep=True)[[inst_col_name, 'G', 'DB', 'V', 'V+G', 'GurF', 'V+GurF', 'GurL', 'V+GurL']]
TABLE1 = avg_gap_df.copy(deep=True)[[inst_col_name]+gap_cols_short]

TABLE1['G'].loc[:,wins_row_name] = ""

# Process the column with # inst to only report number of instances for each set
TABLE1[inst_col_name].loc[:,wins_row_name] = ""
val = TABLE1[inst_col_name].loc[all_set_name,avg_row_name]
TABLE1[inst_col_name].loc[all_set_name,avg_row_name] = \
    create_multirow_string(str(val), extra_format=r"\tablenum[table-format=3]")
val = TABLE1[inst_col_name].loc[good_vpc_set_name,avg_row_name]
TABLE1[inst_col_name].loc[good_vpc_set_name,avg_row_name] = \
    create_multirow_string(str(val), extra_format=r"\tablenum[table-format=3]")

# Reset index to appear as cols
TABLE1.reset_index(inplace=True)

# Place column with # inst as second column
inst_col = TABLE1[inst_col_name]
TABLE1.drop(columns=[inst_col_name], inplace=True)
TABLE1.insert(loc=1, column=inst_col_name, value=inst_col)

# Set column should have multirow
setseries = TABLE1['Set']
format_col_as_multirow(setseries)

# for i in TABLE1.index:
#     curr_name = tex_escape(str(i))
#     print("Changing {} to {}".format(i, curr_name))
#     TABLE1.rename({i: curr_name}, inplace=True)
# print("")

# If we are not using the automatic tex-escaper, we need to do it ourselves
for col in TABLE1.columns:
    # curr_col = '{' + tex_escape(col) + '}'
    curr_col = tex_escape(str(col))
    TABLE1.rename({col: curr_col}, inplace=True, axis=1)

# Finally, apply the desired style
# styler.format({
#     ("Numeric", "Integers"): '\${}',
#     ("Numeric", "Floats"): '{:.3f}',
#     ("Non-Numeric", "Strings"): str.upper
# })
# styler.format_index(escape="latex", axis=0).format_index(escape="latex", axis=1)
# styler.hide(level=0,axis=0)
table1_str = TABLE1.style.\
    hide(axis=0).\
    format(formatter = int_format).\
    to_latex(
        #@{}l@{\hskip 5pt}
        column_format="""
        @{}l@{}
        S[table-format=2.0,table-auto-round,table-number-alignment=center]
        l
        *{1}{S[table-auto-round]}
        *{7}{S[table-auto-round]}
        @{}""",
        hrules = True,
        #clines = "skip-last;data",
        sparse_index = True,
        multirow_align = "c",
        # float_format="%.2f", 
        # escape=False, 
        siunitx=True,
        # index_names=False,
        #columns=['\# inst', 'G', 'DB', 'V', 'V+G', 'GurF', 'V+GurF', 'GurL', 'V+GurL']
        convert_css = True,
        environment = "table",
        position_float = "centering",
        label = "tab:gap-closed-summary",
        caption = """
            Summary statistics for percent gap closed by VPCs.
            The wins row reports how many instances close at least $\epsilon$ more gap using DB, V, V+G compared to G on its own, V+GurF compared to GurF, and V+GurL compared to GurL.
        """,
        )

# Add a midrule between the two sets; the "3" is hand-coded but can be automated
table1_str = add_midrule(table1_str, -3)

# Adjustbox environment sets width to pagewidth
table1_str = add_adjustbox_environment(table1_str)

# Set default siunitx options for this table
table1_str = add_sisetup(table1_str)

print(table1_str)


{
\sisetup{
    table-alignment-mode = format,
    table-number-alignment = center,
    table-format = 2.2,
}
\begin{table}
\centering
\caption{
            Summary statistics for percent gap closed by VPCs.
            The wins row reports how many instances close at least $\epsilon$ more gap using DB, V, V+G compared to G on its own, V+GurF compared to GurF, and V+GurL compared to GurL.
        }
\label{tab:gap-closed-summary}
\begin{adjustbox}{width=1\textwidth}
\begin{tabular}{@{}l@{}
        S[table-format=2.0,table-auto-round,table-number-alignment=center]
        l
        *{1}{S[table-auto-round]}
        *{7}{S[table-auto-round]}
        @{}}
\toprule
{Set} & {\# inst} & {} & {G} & {DB} & {V} & {V+G} & {max(G,V)} & {GurF} & {V+GurF} & {GurL} & {V+GurL} \\
\midrule
{\multirow[c]{2}{*}{All}} & {\multirow[c]{2}{*}{\tablenum[table-format=3]{328}}} & Avg (\%) & 13.446462510363968 & 15.705382040300913 & 9.513701102676414 & 19.260230535680012 & 18.36998860724214 & 26.653322267468695

## Format Table 2: depth x gap

In [120]:
# Format Table 2: percent gap closed by depth
TABLE2 = gap_by_size_df.copy(deep=True)

# If we are not using the automatic tex-escaper, we need to do it ourselves
for col in TABLE2.columns:
    # curr_col = '{' + tex_escape(col) + '}'
    curr_col = tex_escape(str(col))
    TABLE2.rename({col: curr_col}, inplace=True, axis=1)

# Finally, apply the desired style
table2_str = TABLE2.style.\
    format(formatter = int_format).\
    to_latex(
        column_format="""
        @{}l
        *{5}{S[table-auto-round]}
        @{}""",
        hrules = True,
        sparse_index = True,
        multirow_align = "c",
        siunitx=True,
        convert_css = True,
        environment = "table",
        position_float = "centering",
        label = "tab:depth",
        caption = """
            Average percent gap closed broken down by the number of leaf nodes used to construct the partial branch-and-bound tree,
            for VPCs with and without GMICs, as well as at the root by \Gurobi{} after the first and last round of cuts. 
            ``Best'' refers to the maximum gap closed across all partial tree sizes.
        """,
        )

# Adjustbox environment sets width to pagewidth
# table2_str = add_adjustbox_environment(table2_str)

# Set default siunitx options for this table
table2_str = add_sisetup(table2_str)

print(table2_str)


{
\sisetup{
    table-alignment-mode = format,
    table-number-alignment = center,
    table-format = 2.2,
}
\begin{table}
\centering
\caption{
            Average percent gap closed broken down by the number of leaf nodes used to construct the partial branch-and-bound tree,
            for VPCs with and without GMICs, as well as at the root by \Gurobi{} after the first and last round of cuts. 
            ``Best'' refers to the maximum gap closed across all partial tree sizes.
        }
\label{tab:depth}
\begin{tabular}{@{}l
        *{5}{S[table-auto-round]}
        @{}}
\toprule
{} & {DB} & {V} & {V+G} & {max(G,V)} & {V+GurF} & {V+GurL} \\
\midrule
VPCs disabled & 0.0 & 0.0 & 13.446462510363968 & 13.446462510363968 & 26.653322267468695 & 45.82851562691092 \\
2 leaves & 2.4530571843479825 & 1.8051768464473235 & 14.098189826297537 & 13.715142542836222 & 26.905517076184907 & 43.98534513458668 \\
4 leaves & 4.440741473392419 & 3.0004232266566735 & 14.585542682683085 & 14.07108808317449

## Format Table 3: summary of b&b results

In [121]:
# Format Table 3: summary of b&b results
TABLE3 = avg_bb_df.copy(deep=True)

# Remove unnecessary entries
TABLE3.loc[(slice(None), slice(None), bb_metrics[1:3]),([time_col_header,node_col_header],map_cols_to_short_time[gur1time_col])] = ""
TABLE3.loc[(slice(None), slice(None), bb_metrics[2]),([time_col_header,node_col_header],map_cols_to_short_time[gur7time_col])] = ""

# Process the column with # inst to only report number of instances for each set
TABLE3.loc[(slice(None), slice(None), bb_metrics[1:3]), inst_col_name] = ""

for curr_class in bb_classes:
    for curr_bucket in bb_buckets:
        curr_name = (curr_class, curr_bucket, bb_metrics[0])
        val = TABLE3.loc[curr_name, inst_col_name]
        TABLE3.loc[curr_name, inst_col_name] = \
            create_multirow_string(str(val), num_rows = 3, extra_format=r"\tablenum[table-format=3]")

# Set num wins in int format or enclose in braces (center)
# tmp_df = TABLE3.loc[(slice(None), slice(None), bb_metrics[1:3]),time_col_header].applymap(int_format, num_digits=6)
tmp_df = TABLE3.loc[(slice(None), slice(None), bb_metrics[1:3]),time_col_header].applymap(int_format, num_digits=4, add_phantom=True)
# tmp_df = TABLE3.loc[(slice(None), slice(None), bb_metrics[1:3]),time_col_header].applymap(enclose_in_braces)
tmp_df.columns = pd.MultiIndex.from_product([[time_col_header],tmp_df.columns])
TABLE3.loc[(slice(None), slice(None), bb_metrics[1:3]),time_col_header] = tmp_df

# tmp_df = TABLE3.loc[(slice(None), slice(None), bb_metrics[1:3]),node_col_header].applymap(int_format, num_digits=6)
tmp_df = TABLE3.loc[(slice(None), slice(None), bb_metrics[1:3]),node_col_header].applymap(int_format, num_digits=6, add_phantom=False)
# tmp_df = TABLE3.loc[(slice(None), slice(None), bb_metrics[1:3]),node_col_header].applymap(enclose_in_braces)
tmp_df.columns = pd.MultiIndex.from_product([[node_col_header],tmp_df.columns])
TABLE3.loc[(slice(None), slice(None), bb_metrics[1:3]),node_col_header] = tmp_df

# Reset index to appear as cols
TABLE3.reset_index(inplace=True)

# Add new col combining class and bucket in one
class_bucket_col = "\multirow{3}{*}{\shortstack[l]{" + TABLE3['class'] + "\\\\\\relax " + TABLE3['bucket'] + "}}"
for i in range(len(class_bucket_col)):
    if i%3!=0:
        class_bucket_col[i] = ""
TABLE3.drop(columns = ['class', 'bucket'], inplace = True, level = 0)
TABLE3.insert(loc=0, column="Set", value=class_bucket_col)

# Place column with # inst as second column
inst_col = TABLE3[inst_col_name]
TABLE3.drop(columns=[inst_col_name], inplace=True, level=0)
TABLE3.insert(loc=1, column=inst_col_name, value=inst_col)

# If we are not using the automatic tex-escaper, we need to do it ourselves
for col in TABLE3.columns:
    if isinstance(col, tuple):
        for lvl_ind, lvl_col in enumerate(col):
            curr_col = tex_escape(str(lvl_col))
            TABLE3.rename({lvl_col: curr_col}, inplace=True, axis=1, level=lvl_ind)
    else:
        # curr_col = '{' + tex_escape(col) + '}'
        curr_col = tex_escape(str(col))
        TABLE3.rename({col: curr_col}, inplace=True, axis=1)

# Finally, apply the desired style
    # format(formatter = int_format).\
table3_str = TABLE3.style.\
    hide(axis=0).\
    to_latex(
        column_format="""
        @{}l    % set
        c       % inst
        l       % stat
        *{2}{S[table-auto-round,table-format=4.2]}
        *{2}{H}
        *{2}{S[table-auto-round,table-format=4.2]}        
        *{2}{S[table-auto-round,table-format=6.0]}
        *{1}{H}
        *{1}{S[table-auto-round,table-format=6.0]}
        @{}}""",
        hrules = True,
        sparse_index = True,
        multirow_align = "c",
        siunitx=True,
        convert_css = True,
        environment = "table",
        position_float = "centering",
        label = "tab:bb-summary",
        caption = """
            Summary statistics for time to solve instances with branch-and-bound.
        """,
        )

# Add a midrule between the two sets; the "9" is hand-coded but can be automated
table3_str = add_midrule(table3_str, -13)

# Adjustbox environment sets width to pagewidth
table3_str = add_adjustbox_environment(table3_str)

# Set default siunitx options for this table
table3_str = add_sisetup(table3_str, table_format="4.2")

print(table3_str)


{
\sisetup{
    table-alignment-mode = format,
    table-number-alignment = center,
    table-format = 4.2,
}
\begin{table}
\centering
\caption{
            Summary statistics for time to solve instances with branch-and-bound.
        }
\label{tab:bb-summary}
\begin{adjustbox}{width=1\textwidth}
\begin{tabular}{@{}l    % set
        c       % inst
        l       % stat
        *{2}{S[table-auto-round,table-format=4.2]}
        *{2}{H}
        *{2}{S[table-auto-round,table-format=4.2]}        
        *{2}{S[table-auto-round,table-format=6.0]}
        *{1}{H}
        *{1}{S[table-auto-round,table-format=6.0]}
        @{}}}
\toprule
{Set} & {\# inst} & {metric} & \multicolumn{6}{r}{Time (s)} & \multicolumn{4}{r}{Nodes (\textbackslash{}\#)} \\
{} & {} & {} & {Gur1} & {Gur7} & {V} & {Total} & {V7} & {Total7} & {Gur1} & {Gur7} & {V} & {V7} \\
\midrule
\multirow{3}{*}{\shortstack[l]{All\\\relax [0,3600)}} & {\multirow[c]{3}{*}{\tablenum[table-format=3]{300}}} & Gmean & 122.549348 & 92.1215

/var/folders/_8/bgx4yr0n6f1dm56pnvsqqb1s91dzml/T/ipykernel_17114/1685802363.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  TABLE3.loc[curr_name, inst_col_name] = \


## Prepare Table 4: rejected instances

#### Verbose version

In [122]:
## *Verbose version*: For each instance that was not selected, print the reason
df_rejection_reason_rejected = df_rejection_reason[df_rejection_reason['SELECTED_GAP'] == False]
rejected_instance_list = df_rejection_reason_rejected.index
rejected_instance_list.name = 'Instance'
cols = ['Set', 'Reason']
df_rejected_instances = pd.DataFrame(columns=cols, index=rejected_instance_list)
df_rejected_instances['Set'] = df_ipopt.loc[rejected_instance_list,'SET']
df_rejected_instances.loc[df_rejection_reason_rejected['OPTIMAL_SOLUTION_FOUND'] > 0, 
                            'Reason'] = "Integer-optimal solution found constructing partial tree"
df_rejected_instances.loc[(df_rejection_reason_rejected['OPTIMAL_SOLUTION_FOUND'] == 0) 
                                & (df_rejection_reason_rejected['LP=DLB=DUB'] == 6), 
                            'Reason'] = "Max leaf value = LP value"
df_rejected_instances.loc[(df_rejection_reason_rejected['OPTIMAL_SOLUTION_FOUND'] == 0) 
                                & (df_rejection_reason_rejected['LP=DLB=DUB'] < 6) 
                                & (df_rejection_reason_rejected['LP=DLB=DUB'] + df_rejection_reason_rejected['PRLP_INFEASIBLE'] == 6), 
                            'Reason'] = "Max leaf value = LP value or PRLP primal infeasible"
df_rejected_instances.loc[(df_rejection_reason_rejected['OPTIMAL_SOLUTION_FOUND'] == 0)
                                & (df_rejection_reason_rejected['LP=DLB=DUB'] < 6) 
                                & (df_rejection_reason_rejected['LP=DLB=DUB'] + df_rejection_reason_rejected['PRLP_INFEASIBLE'] < 6)
                                & (df_rejection_reason_rejected['LP=DLB=DUB'] + df_rejection_reason_rejected['PRLP_INFEASIBLE'] + df_rejection_reason_rejected['PRLP_TIME_LIMIT'] == 6), 
                            'Reason'] = "Max leaf value = LP value or PRLP primal infeasible / hits time limit"
df_rejected_instances.loc[df_rejection_reason_rejected['<7_ATTEMPTS'] > 0, 
                            'Reason'] = "Numerical issues"
display(df_rejected_instances.head())
col_format = """@{}*{2}{l}X@{}"""

tmp_df_remaining_rejected_instances = df_rejection_reason.loc[df_rejected_instances[df_rejected_instances['Reason'].isna()].index]
if len(tmp_df_remaining_rejected_instances) > 0:
    display(tmp_df_remaining_rejected_instances)

,Set,Reason
Instance,,
22433_presolved,miplib2017,Integer-optimal solution found constructing pa...
air01_presolved,miplib2,Integer-optimal solution found constructing pa...
app1-1_presolved,miplib2017,Integer-optimal solution found constructing pa...
b-ball_presolved,miplib2017,Max leaf value = LP value or PRLP primal infea...
bppc4-08_presolved,miplib2017,Max leaf value = LP value or PRLP primal infea...


,SELECTED_GAP,SELECTED_TIME,SELECTED_6TREES,NUM_WITH_OBJS,NUM_WITH_CUTS,IP_OPT_UNKNOWN,TOO_MANY_ROWS_OR_COLS,OPTIMAL_SOLUTION_FOUND,LP_OPT_IS_NOT_CUT,DLB=DUB,LP=DLB=DUB,PRLP_INFEASIBLE,PRLP_TIME_LIMIT,NO_CUTS,NO_GAP,GUR_TIMEOUT,<7_ATTEMPTS
Instance,,,,,,,,,,,,,,,,,
neos-1112782_presolved,False,True,True,5,0,False,False,0,0,0,0,0,1,True,False,False,False
neos-1112787_presolved,False,True,True,6,0,False,False,0,0,0,0,0,0,True,False,False,False


#### Succinct version

In [123]:
## *Succinct version*: For each instance that was not selected, print the reason
df_rejected_instances = df_status_by_depth.loc[df_rejection_reason[df_rejection_reason['SELECTED_GAP'] == False].index]
df_rejected_instances.insert(loc = 0, column = 'Set', value = df_ipopt.loc[rejected_instance_list,'SET'])
col_format="""@{}*{2}{l}*{6}{c}@{}"""

### Print Table 4

In [124]:
# Format Table 4: rejected instances reasons
TABLE4 = df_rejected_instances.copy(deep=True)
TABLE4.reset_index(inplace=True)

TABLE4["Instance"] = TABLE4["Instance"].apply(remove_presolved_from_name)
TABLE4["Instance"] = TABLE4["Instance"].apply(tex_escape)

# If we are not using the automatic tex-escaper, we need to do it ourselves
for col in TABLE4.columns:
    # curr_col = '{' + tex_escape(col) + '}'
    curr_col = tex_escape(str(col))
    TABLE4.rename({col: curr_col}, inplace=True, axis=1)

# Finally, apply the desired style
    # format_index(escape="latex", axis=0).\
table4_str = TABLE4.style.\
    hide(axis=0).\
    to_latex(
        column_format=col_format,
        hrules = True,
        sparse_index = True,
        multirow_align = "c",
        siunitx=False,
        convert_css = True,
        environment = "table",
        position_float = "centering",
        label = "app:tab:discarded-instances",
        caption = """
            Instances that were not considered with reason for being discarded.
        """,
        )
        
print(table4_str)

\begin{table}
\centering
\caption{
            Instances that were not considered with reason for being discarded.
        }
\label{app:tab:discarded-instances}
\begin{tabular}{@{}*{2}{l}*{6}{c}@{}}
\toprule
Instance & Set & 2 & 4 & 8 & 16 & 32 & 64 \\
\midrule
22433 & miplib2017 &  &  & \ref{selection-criterion:partial-tree-does-not-find-opt} & \ref{selection-criterion:partial-tree-does-not-find-opt} & \ref{selection-criterion:partial-tree-does-not-find-opt} & \ref{selection-criterion:partial-tree-does-not-find-opt} \\
air01 & miplib2 & \ref{selection-criterion:partial-tree-does-not-find-opt} & \ref{selection-criterion:partial-tree-does-not-find-opt} & \ref{selection-criterion:partial-tree-does-not-find-opt} & \ref{selection-criterion:partial-tree-does-not-find-opt} & \ref{selection-criterion:partial-tree-does-not-find-opt} & \ref{selection-criterion:partial-tree-does-not-find-opt} \\
app1-1 & miplib2017 &  &  &  &  & \ref{selection-criterion:partial-tree-does-not-find-opt} & \ref{sel

### DEBUG: Test Table 4 code and make sure "set" is properly identified

In [125]:
#### DEBUG
# df_rejection_reason[df_rejection_reason['NUM_WITH_OBJS'] != df_rejection_reason['NUM_WITH_CUTS']]
# df_rejection_reason[(df_rejection_reason['NUM_WITH_CUTS'] > 0) & (df_rejection_reason['DLB=DUB'] > 0) & (df_rejection_reason['OPTIMAL_SOLUTION_FOUND'] == 0)]
# df_rejection_reason[(df_rejection_reason['LP=DLB=DUB'] == 6)]

# inst = 'chromaticindex32-8_presolved'
# # df_rejection_reason.loc[inst]
# tmp = df_bb.loc[(inst,64)]
# tmp[25:50]

# len(df_rejection_reason[df_rejection_reason['SELECTED'] == True])
# inst = 'berlin_5_8_0_presolved'
# gap_df.loc[inst]
#df_rejection_reason.loc['bnatt400_presolved']

In [126]:
##### DEBUG: Verify "Set" col is correct
for inst in rejected_instance_list:
    curr_set = df_ipopt.loc[inst,'SET']
    has_error = False
    if isinstance(curr_set, pd.Series):
        # check that all sets are same, then just take first
        first_set = curr_set[0]
        for tmp_set in curr_set:
            if tmp_set != first_set:
                print("*** ERROR: not all sets are equal ({} != {})".format(first_set, tmp_set))
                has_error = True
                break
        curr_set = first_set
    ref_set = df_rejected_instances.loc[inst, 'Set']
    if ref_set != curr_set:
        print("*** ERROR: for inst {}, df_rej_inst set {} != df_ipopt set {}".format(inst, ref_set, curr_set))
        has_error = True
    
    if has_error:
        break

## Format Table 5: full gap closed results

In [127]:
# Format Table 5: full gap closed results
TABLE5 = all_gap_results_df.copy(deep=True)

# Set wins row to be integer valued
TABLE5.loc['Wins'] = TABLE5.loc['Wins'].apply(int_format)
# TABLE5.iloc[len(TABLE5)-1] = TABLE5.iloc[len(TABLE5)-1].apply(int_format)

# Move instance names into a column
TABLE5.reset_index(inplace=True, col_level=1)

TABLE5[('',"Instance")] = TABLE5[('',"Instance")].apply(remove_presolved_from_name)
TABLE5[('',"Instance")] = TABLE5[('',"Instance")].apply(tex_escape)

# If we are not using the automatic tex-escaper, we need to do it ourselves
for col in TABLE5.columns:
    # curr_col = '{' + tex_escape(col) + '}'
    curr_col = tex_escape(str(col))
    TABLE5.rename({col: curr_col}, inplace=True, axis=1)

# Finally, apply the desired style
    # format(formatter = int_format).\
table5_str = TABLE5.style.\
    hide(axis=0).\
    to_latex(
        column_format="""
	@{}l*{2}{S[table-format=4.0,table-auto-round,table-number-alignment=center]}
	*{2}{S[table-format=4.0,table-auto-round,table-number-alignment=center]}
	*{8}{S[table-auto-round]}
	@{}
        """,
        hrules = True,
        sparse_index = True,
        multirow_align = "c",
        siunitx=True,
        convert_css = True,
        environment = "table",
        position_float = "centering",
        label = "app:tab:gap-closed",
        caption = """
            Percent gap closed by instance for GMICs (G), VPCs (V), both VPCs and GMICs used together, 
            and the bound implied by the partial branch-and-bound tree with 64 leaf nodes (DB).
            Also shown are the sizes of the instances, the number of cuts added, and the percent gap closed by 
            \Gurobi{} at the root (after one round (GurF) and after the last round (GurL)). 
            Entries in which DB appears to be 0.00 are actually small strictly positive numbers.
        """,
        )

# Adjustbox environment sets width to pagewidth
# table2_str = add_adjustbox_environment(table2_str)

# Set default siunitx options for this table
table5_str = add_sisetup(table5_str)

# Add a midrule between the instances and 3 summary rows; the "5" is hand-coded but can be automated
table5_str = add_midrule(table5_str, -5)

print(table5_str)


{
\sisetup{
    table-alignment-mode = format,
    table-number-alignment = center,
    table-format = 2.2,
}
\begin{table}
\centering
\caption{
            Percent gap closed by instance for GMICs (G), VPCs (V), both VPCs and GMICs used together, 
            and the bound implied by the partial branch-and-bound tree with 64 leaf nodes (DB).
            Also shown are the sizes of the instances, the number of cuts added, and the percent gap closed by 
            \Gurobi{} at the root (after one round (GurF) and after the last round (GurL)). 
            Entries in which DB appears to be 0.00 are actually small strictly positive numbers.
        }
\label{app:tab:gap-closed}
\begin{tabular}{@{}l*{2}{S[table-format=4.0,table-auto-round,table-number-alignment=center]}
	*{2}{S[table-format=4.0,table-auto-round,table-number-alignment=center]}
	*{8}{S[table-auto-round]}
	@{}}
\toprule
\multicolumn{3}{r}{} & \multicolumn{2}{r}{# cuts} & \multicolumn{9}{r}{% gap closed} \\
{Instance} & {Rows

## Format Table 6: "all" time/nodes results

In [128]:
# Format Table 6: "all" time/nodes results
TABLE6 = all_bb_results_df.copy(deep=True)

# Rename summary rows to reflect the set
rename_metrics_all = {metric : metric + ' (All)' for metric in bb_metrics}
TABLE6.rename(rename_metrics_all, inplace=True)

# Add summary rows from 6 trees set
summary_metrics_6trees = all6_bb_results_df.tail(3).copy(deep=True)
rename_metrics_6trees = {metric : metric + ' (6 trees)' for metric in bb_metrics}
summary_metrics_6trees.rename(rename_metrics_6trees, inplace=True)

TABLE6 = pd.concat([TABLE6, summary_metrics_6trees])

# Drop rows, cols, (time,V7)
TABLE6.drop([('','Rows'),('','Cols'),(node_col_header,map_cols_to_short_time[mintime_col])], axis=1, inplace=True)

# Set wins row to be integer valued
TABLE6.loc['Wins1 (All)'] = TABLE6.loc['Wins1 (All)'].apply(int_format)
TABLE6.loc['Wins7 (All)'] = TABLE6.loc['Wins7 (All)'].apply(int_format)
TABLE6.loc['Wins1 (6 trees)'] = TABLE6.loc['Wins1 (6 trees)'].apply(int_format)
TABLE6.loc['Wins7 (6 trees)'] = TABLE6.loc['Wins7 (6 trees)'].apply(int_format)
# TABLE6.iloc[len(TABLE6)-1] = TABLE6.iloc[len(TABLE6)-1].apply(int_format)

# Move instance names into a column
TABLE6.reset_index(inplace=True, col_level=1)

# Store indices of rows of 6-tree instances
six_trees_instances = list(all6_instances_dict.keys())
# mask = TABLE6[('','Instance')].isin(six_trees_instances)
# six_trees_indices = TABLE6.loc[mask, :].index.tolist()

# Remove presolved from name and escape
TABLE6[('',"Instance")] = TABLE6[('',"Instance")].apply(remove_presolved_from_name)
TABLE6[('',"Instance")] = TABLE6[('',"Instance")].apply(tex_escape)

# If we are not using the automatic tex-escaper, we need to do it ourselves
for col in TABLE6.columns:
    # curr_col = '{' + tex_escape(col) + '}'
    curr_col = tex_escape(str(col))
    TABLE6.rename({col: curr_col}, inplace=True, axis=1)

# Finally, apply the desired style
    # format(formatter = int_format).\
table6_str = TABLE6.style.\
    hide(axis=0).\
    to_latex(
        column_format="""
	@{}l % instance
	*{1}{S[table-format=4.0,table-auto-round,table-number-alignment=center]} % # cuts
	*{2}{S[table-format=4.2,table-auto-round]} % Gur1, Gur7
	*{2}{H} % V, Total
	*{2}{S[table-format=4.2,table-auto-round]} % V7, Total7
	*{3}{S[table-format=8.0,table-auto-round,table-number-alignment=center]} % Nodes
	@{}
        """,
        hrules = True,
        sparse_index = True,
        multirow_align = "c",
        siunitx=True,
        convert_css = True,
        environment = "table",
        position_float = "centering",
        label = "app:tab:bb",
        caption = """
            Time (in seconds) and number nodes taken to solve each instance.
            The table is sorted by column 4 (``V'' under ``Time (s)'').
            ``Gur1'' indicates \Gurobi{} run with one random seed.
            ``Gur7'' indicates the minimum from seven runs of \Gurobi{} with different random seeds.
        """,
        )

# Adjustbox environment sets width to pagewidth
# table6_str = add_adjustbox_environment(table6_str)

# Set default siunitx options for this table
table6_str = add_sisetup(table6_str)

# Add a midrule between the instances and 3 summary rows; the "6" is hand-coded but can be automated
table6_str = add_midrule(table6_str, -6)
table6_str = add_midrule(table6_str, -10)

# Add color to six tree instances
splitlines = table6_str.splitlines()
for i in range(len(splitlines)):
    line = splitlines[i]
    curr_line = line.split('&')
    if len(curr_line) > 0 and curr_line[0].strip()+'_presolved' in six_trees_instances:
        splitlines[i] = '\\rowcolor{lightgray!30} ' + line
table6_str = '\n'.join(splitlines).replace('NaN', '')

print(table6_str)


{
\sisetup{
    table-alignment-mode = format,
    table-number-alignment = center,
    table-format = 2.2,
}
\begin{table}
\centering
\caption{
            Time (in seconds) and number nodes taken to solve each instance.
            The table is sorted by column 4 (``V'' under ``Time (s)'').
            ``Gur1'' indicates \Gurobi{} run with one random seed.
            ``Gur7'' indicates the minimum from seven runs of \Gurobi{} with different random seeds.
        }
\label{app:tab:bb}
\begin{tabular}{@{}l % instance
	*{1}{S[table-format=4.0,table-auto-round,table-number-alignment=center]} % # cuts
	*{2}{S[table-format=4.2,table-auto-round]} % Gur1, Gur7
	*{2}{H} % V, Total
	*{2}{S[table-format=4.2,table-auto-round]} % V7, Total7
	*{3}{S[table-format=8.0,table-auto-round,table-number-alignment=center]} % Nodes
	@{}}
\toprule
{} & {# cuts} & \multicolumn{6}{r}{Time (s)} & \multicolumn{3}{r}{Nodes (\#)} \\
{Instance} & {V} & {Gur1} & {Gur7} & {V} & {Total} & {V7} & {Total7} & {Gur1} & {

## XXX Format Table X: "6 trees" time/nodes results

In [129]:
# Format Table 7: "6 trees" time/nodes results
TABLEX = all6_bb_results_df.copy(deep=True)

# Set wins row to be integer valued
TABLEX.loc['Wins1'] = TABLEX.loc['Wins1'].apply(int_format)
TABLEX.loc['Wins7'] = TABLEX.loc['Wins7'].apply(int_format)
# TABLEX.iloc[len(TABLEX)-1] = TABLEX.iloc[len(TABLEX)-1].apply(int_format)

# Move instance names into a column
TABLEX.reset_index(inplace=True, col_level=1)

TABLEX[('',"Instance")] = TABLEX[('',"Instance")].apply(remove_presolved_from_name)
TABLEX[('',"Instance")] = TABLEX[('',"Instance")].apply(tex_escape)

# If we are not using the automatic tex-escaper, we need to do it ourselves
for col in TABLEX.columns:
    # curr_col = '{' + tex_escape(col) + '}'
    curr_col = tex_escape(str(col))
    TABLEX.rename({col: curr_col}, inplace=True, axis=1)

# Finally, apply the desired style
    # format(formatter = int_format).\
tableX_str = TABLEX.style.\
    hide(axis=0).\
    to_latex(
        column_format="""@{}l*{2}{c}*{2}{c}H*{8}{c}@{}""",
        hrules = True,
        sparse_index = True,
        multirow_align = "c",
        siunitx=True,
        convert_css = True,
        environment = "table",
        position_float = "centering",
        label = "app:tab:bb-7trees",
        caption = """
  Time (in seconds) and number nodes taken to solve each of the instances for which all six branch-and-bound trees successfully yielded VPCs.
  %The columns with V1x are those in which we do not terminate the VPC computation as soon as the time exceeds \Gurobi{}'s time.  
  The table is sorted by column 4 (``V7'' under ``Time (s)'').
  ``Gur1'' indicates Gurobi run with one random seed.
  ``Gur7'' indicates the minimum from seven runs of Gurobi with different random seeds.
        """,
        )

# Adjustbox environment sets width to pagewidth
# tableX_str = add_adjustbox_environment(tableX_str)

# Set default siunitx options for this table
tableX_str = add_sisetup(tableX_str)

# Add a midrule between the instances and 3 summary rows; the "6" is hand-coded but can be automated
tableX_str = add_midrule(tableX_str, -6)

print(tableX_str)


{
\sisetup{
    table-alignment-mode = format,
    table-number-alignment = center,
    table-format = 2.2,
}
\begin{table}
\centering
\caption{
  Time (in seconds) and number nodes taken to solve each of the instances for which all six branch-and-bound trees successfully yielded VPCs.
  %The columns with V1x are those in which we do not terminate the VPC computation as soon as the time exceeds \Gurobi{}'s time.  
  The table is sorted by column 4 (``V7'' under ``Time (s)'').
  ``Gur1'' indicates Gurobi run with one random seed.
  ``Gur7'' indicates the minimum from seven runs of Gurobi with different random seeds.
        }
\label{app:tab:bb-7trees}
\begin{tabular}{@{}l*{2}{c}*{2}{c}H*{8}{c}@{}}
\toprule
\multicolumn{3}{r}{} & {# cuts} & \multicolumn{6}{r}{Time (s)} & \multicolumn{4}{r}{Nodes (\#)} \\
{Instance} & {Rows} & {Cols} & {V} & {Gur1} & {Gur7} & {V} & {Total} & {V7} & {Total7} & {Gur1} & {Gur7} & {V} & {V7} \\
\midrule
neos-501453 & 13 & 52 & 0 & 0.006000 & 0.003000 & 0.006

## Format Table 7: b&b summary by depth

In [130]:
# Format Table 7: summary of b&b results
TABLE7 = avg_bb_by_depth_df.copy(deep=True)

# Remove unnecessary entries
TABLE7.loc[(slice(None), slice(None), bb_metrics_by_depth[1:]),([time_col_header,node_col_header],map_cols_to_short_time[gur1time_col])] = ""
# TABLE7.loc[(slice(None), slice(None), bb_metrics_by_depth[2]),([time_col_header,node_col_header],map_cols_to_short_time[gur7time_col])] = ""

# Process the column with # inst to only report number of instances for each set
TABLE7.loc[(slice(None), slice(None), bb_metrics_by_depth[1:]), inst_col_name] = ""

for curr_class in bb_classes_by_depth:
    for curr_bucket in bb_buckets_by_depth:
        curr_name = (curr_class, curr_bucket, bb_metrics_by_depth[0])
        val = TABLE7.loc[curr_name, inst_col_name]
        TABLE7.loc[curr_name, inst_col_name] = \
            create_multirow_string(str(val), num_rows = 2, extra_format=r"\tablenum[table-format=3]")

# Set num wins in int format or enclose in braces (center)
# tmp_df = TABLE7.loc[(slice(None), slice(None), bb_metrics_by_depth[1:3]),time_col_header].applymap(int_format, num_digits=6)
tmp_df = TABLE7.loc[(slice(None), slice(None), bb_metrics_by_depth[1:3]),time_col_header].applymap(int_format, num_digits=4, add_phantom=True)
# tmp_df = TABLE7.loc[(slice(None), slice(None), bb_metrics_by_depth[1:3]),time_col_header].applymap(enclose_in_braces)
tmp_df.columns = pd.MultiIndex.from_product([[time_col_header],tmp_df.columns])
TABLE7.loc[(slice(None), slice(None), bb_metrics_by_depth[1:3]),time_col_header] = tmp_df

# tmp_df = TABLE7.loc[(slice(None), slice(None), bb_metrics_by_depth[1:3]),node_col_header].applymap(int_format, num_digits=6)
tmp_df = TABLE7.loc[(slice(None), slice(None), bb_metrics_by_depth[1:3]),node_col_header].applymap(int_format, num_digits=6, add_phantom=False)
# tmp_df = TABLE7.loc[(slice(None), slice(None), bb_metrics_by_depth[1:3]),node_col_header ].applymap(enclose_in_braces)
tmp_df.columns = pd.MultiIndex.from_product([[node_col_header],tmp_df.columns])
TABLE7.loc[(slice(None), slice(None), bb_metrics_by_depth[1:3]),node_col_header] = tmp_df

# Reset index to appear as cols
TABLE7.reset_index(inplace=True)

# Add new col combining class and bucket in one
class_bucket_col = "\multirow{2}{*}{\shortstack[l]{" + TABLE7['class'] + "\\\\\\relax " + TABLE7['bucket'] + "}}"
for i in range(len(class_bucket_col)):
    if i%len(bb_metrics_by_depth)!=0:
        class_bucket_col[i] = ""
TABLE7.drop(columns = ['class', 'bucket'], inplace = True, level = 0)
TABLE7.insert(loc=0, column="Set", value=class_bucket_col)

# Place column with # inst as second column
inst_col = TABLE7[inst_col_name]
TABLE7.drop(columns=[inst_col_name], inplace=True, level=0)
TABLE7.insert(loc=1, column=inst_col_name, value=inst_col)

# If we are not using the automatic tex-escaper, we need to do it ourselves
for col in TABLE7.columns:
    if isinstance(col, tuple):
        for lvl_ind, lvl_col in enumerate(col):
            curr_col = tex_escape(str(lvl_col))
            TABLE7.rename({lvl_col: curr_col}, inplace=True, axis=1, level=lvl_ind)
    else:
        # curr_col = '{' + tex_escape(col) + '}'
        curr_col = tex_escape(str(col))
        TABLE7.rename({col: curr_col}, inplace=True, axis=1)

# Finally, apply the desired style
    # format(formatter = int_format).\
table7_str = TABLE7.style.\
    hide(axis=0).\
    to_latex(
        column_format="""
        @{}l    % set
        c       % inst
        l       % stat
        *{3}{S[table-auto-round,table-format=4.2]}
        *{2}{S[table-auto-round,table-format=6.0]}
        @{}""",
        hrules = True,
        sparse_index = True,
        multirow_align = "c",
        siunitx=True,
        convert_css = True,
        environment = "table",
        position_float = "centering",
        label = "tab:bb-summary",
        caption = """
            Summary statistics for time to solve instances with branch-and-bound.
        """,
        )

# Add a midrule between the two sets; the "9" is hand-coded but can be automated
table7_str = add_midrule(table7_str, -41)
table7_str = add_midrule(table7_str, -33)
table7_str = add_midrule(table7_str, -25)
table7_str = add_midrule(table7_str, -17)
table7_str = add_midrule(table7_str, -9)

# Adjustbox environment sets width to pagewidth
table7_str = add_adjustbox_environment(table7_str)

# Set default siunitx options for this table
table7_str = add_sisetup(table7_str, table_format="4.2")

print(table7_str)


{
\sisetup{
    table-alignment-mode = format,
    table-number-alignment = center,
    table-format = 4.2,
}
\begin{table}
\centering
\caption{
            Summary statistics for time to solve instances with branch-and-bound.
        }
\label{tab:bb-summary}
\begin{adjustbox}{width=1\textwidth}
\begin{tabular}{@{}l    % set
        c       % inst
        l       % stat
        *{6}{S[table-auto-round,table-format=4.2]}
        *{4}{S[table-auto-round,table-format=6.0]}
        @{}}
\toprule
{Set} & {\# inst} & {metric} & \multicolumn{3}{r}{Time (s)} & \multicolumn{2}{r}{Nodes (\textbackslash{}\#)} \\
{} & {} & {} & {Gur1} & {V} & {Total} & {Gur1} & {V} \\
\midrule
\multirow{2}{*}{\shortstack[l]{2 leaves\\\relax [0,3600)}} & {\multirow[c]{2}{*}{\tablenum[table-format=3]{196}}} & Gmean & 70.495418 & 73.232937 & 74.060773 & 6594.443364 & 6586.432231 \\
 &  & Wins1 &  & {\tablenum[table-format=4.0]{54}\phantom{.00}} & {\tablenum[table-format=4.0]{35}\phantom{.00}} &  & {\tablenum[table-f

/var/folders/_8/bgx4yr0n6f1dm56pnvsqqb1s91dzml/T/ipykernel_17114/1880040719.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  TABLE7.loc[curr_name, inst_col_name] = \


## Format Table 8: objective + time analysis per instance

In [131]:
# Format Table 8: obj and time analysis
TABLE8 = obj_and_time_df.copy(deep=True)

# Move instance names into a column
TABLE8.reset_index(inplace=True)
TABLE8.drop('disj_terms',axis=1,inplace=True)

# Create new column index
TABLE8.columns = pd.MultiIndex.from_tuples(
    [('','Instance'),
    ('Objectives','Obj'),
    ('Objectives','Succ'),
    ('Objectives','Fails'),
    ('Objectives','\% fails'),
    ('Time (s)','Total'),
    ('Time (s)','(s) / obj'),
    ('Time (s)','(s) / cut')]
)

# Format instance column correctly
TABLE8[('',"Instance")] = TABLE8[('',"Instance")].apply(remove_presolved_from_name)
TABLE8[('',"Instance")] = TABLE8[('',"Instance")].apply(tex_escape)

# Format SKIP_CHAR correctly
for col in TABLE8.columns:
    TABLE8[col] = TABLE8[col].apply(enclose_in_braces, val_to_match=SKIP_CHAR)

# If we are not using the automatic tex-escaper, we need to do it ourselves
for col in TABLE8.columns:
    # curr_col = '{' + tex_escape(col) + '}'
    curr_col = tex_escape(str(col))
    TABLE8.rename({col: curr_col}, inplace=True, axis=1)

# Finally, apply the desired style
table8_str = TABLE8.style.\
    hide(axis=0).\
    to_latex(
        column_format="""
        @{}
        l
        *{3}{S[table-format=3.0,table-auto-round,table-number-alignment=center]}
        *{1}{S[table-format=2.1,table-auto-round,table-number-alignment=center]}
        *{1}{S[table-format=4.1,table-auto-round,table-number-alignment=center]}
        *{2}{S[table-format=4.1,table-auto-round,table-number-alignment=center]}
        @{}""",
        hrules = True,
        sparse_index = True,
        multirow_align = "c",
        siunitx=True,
        convert_css = True,
        environment = "table",
        position_float = "centering",
        label = "app:tab:obj-and-time-best",
        caption = """
            Information about objectives and time to generate cuts corresponding to the results in Table~\ref{app:tab:gap-closed}.
        """,
        )

# Adjustbox environment sets width to pagewidth
# table8_str = add_adjustbox_environment(table8_str)

table8_str = add_midrule(table8_str, -2)

# Set default siunitx options for this table
table8_str = add_sisetup(table8_str)

print(table8_str)


{
\sisetup{
    table-alignment-mode = format,
    table-number-alignment = center,
    table-format = 2.2,
}
\begin{table}
\centering
\caption{
            Information about objectives and time to generate cuts corresponding to the results in Table~
ef{app:tab:gap-closed}.
        }
\label{app:tab:obj-and-time-best}
\begin{tabular}{@{}
        l
        *{3}{S[table-format=3.0,table-auto-round,table-number-alignment=center]}
        *{1}{S[table-format=2.1,table-auto-round,table-number-alignment=center]}
        *{1}{S[table-format=4.1,table-auto-round,table-number-alignment=center]}
        *{2}{S[table-format=4.1,table-auto-round,table-number-alignment=center]}
        @{}}
\toprule
{} & \multicolumn{4}{r}{Objectives} & \multicolumn{3}{r}{Time (s)} \\
{Instance} & {Obj} & {Succ} & {Fails} & {\% fails} & {Total} & {(s) / obj} & {(s) / cut} \\
\midrule
10teams & 322.0 & 51.0 & 271.0 & 84.16149068322981 & 1542.09 & 4.789099378881987 & 30.23705882352941 \\
23588 & 76.0 & 75.0 & 1.0 & 1

## Format Table 9: number of leaf nodes yielding the best result for each experiment per instance

In [132]:
# Format Table 9: frequency of when each size is best
TABLE9 = pd.concat([best_disj_gap_df,best_disj_time_df],axis=1)
TABLE9.drop('Best', axis=0, inplace=True)

# Create new column index
TABLE9.columns = pd.MultiIndex.from_tuples(
    [('Gap',col) for col in best_disj_gap_df.columns]
    + [('Time',col) for col in best_disj_time_df.columns]
)

# If we are not using the automatic tex-escaper, we need to do it ourselves
for col in TABLE9.columns:
    # curr_col = '{' + tex_escape(col) + '}'
    curr_col = tex_escape(str(col))
    TABLE9.rename({col: curr_col}, inplace=True, axis=1)

# Finally, apply the desired style
    # hide(axis=0).\
table9_str = TABLE9.style.\
    to_latex(
        column_format="""@{}l*4{S}*3{S}@{}""",
        hrules = True,
        sparse_index = True,
        multirow_align = "c",
        siunitx=True,
        convert_css = True,
        environment = "table",
        position_float = "centering",
        label = "app:tab:size",
        caption = """
            Number of leaf nodes yielding the best result for each experiment per instance.
        """,
        )

# Adjustbox environment sets width to pagewidth
# table9_str = add_adjustbox_environment(table9_str)

# table9_str = add_midrule(table9_str, -2)

# Set default siunitx options for this table
table9_str = add_sisetup(table9_str, table_format="3.0")

print(table9_str)


{
\sisetup{
    table-alignment-mode = format,
    table-number-alignment = center,
    table-format = 3.0,
}
\begin{table}
\centering
\caption{
            Number of leaf nodes yielding the best result for each experiment per instance.
        }
\label{app:tab:size}
\begin{tabular}{@{}l*4{S}*3{S}@{}}
\toprule
{} & \multicolumn{4}{r}{Gap} & \multicolumn{2}{r}{Time} \\
{} & {V} & {V+G} & {V+GurF} & {V+GurL} & {All} & {6 trees} \\
\midrule
No improvement & 215 & 136 & 153 & 204 & 116 & 69 \\
2 leaves & 0 & 9 & 35 & 15 & 48 & 30 \\
4 leaves & 4 & 8 & 42 & 25 & 33 & 23 \\
8 leaves & 4 & 22 & 40 & 24 & 35 & 21 \\
16 leaves & 6 & 23 & 47 & 22 & 27 & 19 \\
32 leaves & 18 & 35 & 55 & 20 & 31 & 27 \\
64 leaves & 81 & 109 & 85 & 57 & 16 & 13 \\
\bottomrule
\end{tabular}
\end{table}

}


## Format Table 10: density statistics

In [133]:
# Format Table 10: density statistics
TABLE10 = density_df.copy(deep=True)

# Make # inst and wins row int
tmp_df = TABLE10.iloc[0:2].applymap(int_format, num_digits=3, add_phantom=False)
TABLE10.iloc[0:2] = tmp_df
# tmp_df = TABLE10.loc['\# wins by time',:].apply(int_format, num_digits=2, add_phantom=False)
# TABLE10.loc['\# wins by time'] = tmp_df

# Finally, apply the desired style
    # hide(axis=0).\
table10_str = TABLE10.style.\
    to_latex(
        column_format="""@{}l*{6}{S[table-format=0.3,table-auto-round,table-number-alignment=center]}@{}""",
        hrules = True,
        sparse_index = True,
        multirow_align = "c",
        siunitx=True,
        convert_css = True,
        environment = "table",
        position_float = "centering",
        label = "app:tab:density",
        caption = """
            Statistics about the density of generated cuts broken down by partial tree size.
        """,
        )

# Adjustbox environment sets width to pagewidth
# table10_str = add_adjustbox_environment(table10_str)

# table10_str = add_midrule(table10_str, -2)

# Set default siunitx options for this table
table10_str = add_sisetup(table10_str, table_format="0.3")

print(table10_str)


{
\sisetup{
    table-alignment-mode = format,
    table-number-alignment = center,
    table-format = 0.3,
}
\begin{table}
\centering
\caption{
            Statistics about the density of generated cuts broken down by partial tree size.
        }
\label{app:tab:density}
\begin{tabular}{@{}l*{6}{S[table-format=0.3,table-auto-round,table-number-alignment=center]}@{}}
\toprule
{} & {V (2)} & {V (4)} & {V (8)} & {V (16)} & {V (32)} & {V (64)} \\
\midrule
\# inst w/VPCs and time < 3600s & {\tablenum[table-format=3.0]{251.0}} & {\tablenum[table-format=3.0]{251.0}} & {\tablenum[table-format=3.0]{242.0}} & {\tablenum[table-format=3.0]{241.0}} & {\tablenum[table-format=3.0]{222.0}} & {\tablenum[table-format=3.0]{203.0}} \\
\# wins by time & {\tablenum[table-format=3.0]{100.0}} & {\tablenum[table-format=3.0]{112.0}} & {\tablenum[table-format=3.0]{111.0}} & {\tablenum[table-format=3.0]{101.0}} & {\tablenum[table-format=3.0]{134.0}} & {\tablenum[table-format=3.0]{136.0}} \\
Avg min cut density &

## Format Table 11: failures

In [134]:
# Format Table 11: failures
TABLE11 = obj_fails_df.copy(deep=True)

# Make num inst rows int
tmp_df = TABLE11.iloc[0:5].applymap(int_format, num_digits=3, add_phantom=False)
TABLE11.iloc[0:5] = tmp_df

# Finally, apply the desired style
    # hide(axis=0).\
table11_str = TABLE11.style.\
    to_latex(
        column_format="""@{}l*{6}{S[table-format=2.2,table-auto-round,table-number-alignment=center]}@{}""",
        hrules = True,
        sparse_index = True,
        multirow_align = "c",
        siunitx=True,
        convert_css = True,
        environment = "table",
        position_float = "centering",
        label = "app:tab:objectives",
        caption = """
            Statistics about the objectives leading to failures, broken down by partial tree size used for cut generation.
        """,
        )

# Adjustbox environment sets width to pagewidth
# table11_str = add_adjustbox_environment(table11_str)

table11_str = add_midrule(table11_str, -4)
table11_str = add_midrule(table11_str, -8)
table11_str = add_midrule(table11_str, -13)
table11_str = add_midrule(table11_str, -15)

# Set default siunitx options for this table
table11_str = add_sisetup(table11_str, table_format="2.2")

print(table11_str)


{
\sisetup{
    table-alignment-mode = format,
    table-number-alignment = center,
    table-format = 2.2,
}
\begin{table}
\centering
\caption{
            Statistics about the objectives leading to failures, broken down by partial tree size used for cut generation.
        }
\label{app:tab:objectives}
\begin{tabular}{@{}l*{6}{S[table-format=2.2,table-auto-round,table-number-alignment=center]}@{}}
\toprule
{} & {V (2)} & {V (4)} & {V (8)} & {V (16)} & {V (32)} & {V (64)} \\
\midrule
\# inst w/obj & {\tablenum[table-format=3.0]{307.0}} & {\tablenum[table-format=3.0]{308.0}} & {\tablenum[table-format=3.0]{293.0}} & {\tablenum[table-format=3.0]{296.0}} & {\tablenum[table-format=3.0]{270.0}} & {\tablenum[table-format=3.0]{246.0}} \\
\# inst w/succ obj & {\tablenum[table-format=3.0]{305.0}} & {\tablenum[table-format=3.0]{304.0}} & {\tablenum[table-format=3.0]{289.0}} & {\tablenum[table-format=3.0]{292.0}} & {\tablenum[table-format=3.0]{264.0}} & {\tablenum[table-format=3.0]{238.0}} \\
\# 

## Format Table 12: active cuts

In [135]:
# Format Table 12: active cuts
TABLE12 = active_cuts_df.copy(deep=True)

# Finally, apply the desired style
    # hide(axis=0).\
table12_str = TABLE12.style.\
    to_latex(
        column_format="""@{}l*{6}{S[table-format=3.2,table-auto-round,table-number-alignment=center]}@{}""",
        hrules = True,
        sparse_index = True,
        multirow_align = "c",
        siunitx=True,
        convert_css = True,
        environment = "table",
        position_float = "centering",
        label = "app:tab:activity",
        caption = """
            Statistics about when generated cuts are active, broken down by partial tree size.
        """,
        )

# Adjustbox environment sets width to pagewidth
# table12_str = add_adjustbox_environment(table12_str)

# Set default siunitx options for this table
table12_str = add_sisetup(table12_str, table_format="2.2")

print(table12_str)


{
\sisetup{
    table-alignment-mode = format,
    table-number-alignment = center,
    table-format = 2.2,
}
\begin{table}
\centering
\caption{
            Statistics about when generated cuts are active, broken down by partial tree size.
        }
\label{app:tab:activity}
\begin{tabular}{@{}l*{6}{S[table-format=3.2,table-auto-round,table-number-alignment=center]}@{}}
\toprule
{} & {V+G (2)} & {V+G (4)} & {V+G (8)} & {V+G (16)} & {V+G (32)} & {V+G (64)} \\
\midrule
\% active GMIC & 44.305691 & 43.376751 & 42.289000 & 41.678714 & 41.120183 & 41.267641 \\
\% active VPC & 30.242083 & 31.611173 & 32.078155 & 35.635763 & 35.253731 & 34.802351 \\
\% cuts one-sided & 0.775203 & 0.748656 & 0.770219 & 1.145125 & 0.880503 & 0.875300 \\
\% active one-sided & 100.000000 & 100.000000 & 100.000000 & 100.000000 & 100.000000 & 100.000000 \\
\% cuts all ones & 11.759815 & 6.853341 & 7.931222 & 8.649239 & 7.966688 & 9.229225 \\
\% active all ones & 91.095890 & 84.210526 & 81.021898 & 73.793103 & 79.16

## Format Table 13: instances with best bb improvement from VPCs

In [136]:
# Format Table 13: "best" time/nodes results
TABLE13 = all_bb_results_df.copy(deep=True)

# Drop all rows in which (time_col_header, 'Gur7') is ''
TABLE13 = TABLE13[TABLE13[(time_col_header, 'Gur7')] != '']

# TABLE13[('','V7-Gur7')]
# Change column (time_col_header, V7) to be float valued
#TABLE13[(time_col_header, 'V7')] = TABLE13[(time_col_header, 'V7')].apply(float_format, num_digits=4)
TABLE13 = TABLE13.astype({(time_col_header, 'V7'): float})
TABLE13 = TABLE13.astype({(time_col_header, 'Gur7'): float})

# Add new column for difference between V7 and Gur7
TABLE13[('Time (s)','V7 - Gur7')] = TABLE13[('Time (s)','V7')] - TABLE13[('Time (s)','Gur7')]

# Sort by V7 - Gur7
TABLE13.sort_values(by=[(time_col_header,'V7 - Gur7')], inplace=True)

TABLE13.head(15)

# cuts  Time (s)                      \
                              Rows  Cols      V      Gur1      Gur7         V   
Instance                                                                        
cost266-UUE_presolved         1302  3882     34  3600.003  3600.001  3026.609   
neos-1330346_presolved        1620  2628     32  2447.254  2019.523   1727.42   
gen-ip002_presolved             24    41     18   991.459   899.233   640.739   
csched010_presolved            272  1678    124  1028.336   853.075   604.646   
iis-bupa-cov_presolved        4796   337     91  3600.001  3213.521  2979.352   
ran14x18-disj-8_presolved      449   506     86  1166.638  1166.638   982.947   
neos-911880_presolved           83   888     48  1093.894   206.349    31.831   
neos-1616732_presolved        1026   200     59  2907.053  2518.274  2352.922   
p80x400b_presolved             456   768      3  1106.526   947.710   811.231   
milo-v12-6-r2-40-1_presolved  4157  1897      1   610.417   438.801   356.433   
rococoC10-001000_presolved     617  2483     21   441.115   441.115   362.548   
prod2_presolved                 92   182     44   359.584   230.772   163.447   
icir97_tension_presolved       818  1816      3   307.219   307.219   249.817   
neos-831188_presolved         2153  4580     21   256.907   219.959   180.349   
neos-911970_presolved          107   888     43    3600.0    47.485    23.846   

                                                           Nodes (\#)  \
                                 Total        V7    Total7       Gur1   
Instance                                                                
cost266-UUE_presolved         3045.189  3026.609  3045.189      73229   
neos-1330346_presolved          1750.6  1727.420    1750.6     778923   
gen-ip002_presolved            641.239   640.739   641.239    4109532   
csched010_presolved            610.576   604.646   610.576      44259   
iis-bupa-cov_presolved        2987.282  2979.352  2987.282     207837   
ran14x18-disj-8_presolved      1497.38   982.947  1166.638     218599   
neos-911880_presolved           32.181    31.831    32.181     912075   
neos-1616732_presolved        2354.522  2352.922  2354.522    1208038   
p80x400b_presolved             811.441   811.231   811.441     208964   
milo-v12-6-r2-40-1_presolved   380.433   356.433   380.433      51974   
rococoC10-001000_presolved     367.648   362.548   367.648      28186   
prod2_presolved                178.817   163.447   178.817     129137   
icir97_tension_presolved       258.767   249.817   258.767      52342   
neos-831188_presolved          239.859   180.349   239.859       5753   
neos-911970_presolved           24.076    23.846    24.076    3617504   

                                                         Time (s)  
                                 Gur7        V       V7 V7 - Gur7  
Instance                                                           
cost266-UUE_presolved           55668    62683    62683  -573.392  
neos-1330346_presolved         379879   266435   266435  -292.103  
gen-ip002_presolved           3830968  3972559  3972559  -258.494  
csched010_presolved             31493    25752    25752  -248.429  
iis-bupa-cov_presolved         179734   130889   130889  -234.169  
ran14x18-disj-8_presolved      218599   169311   169311  -183.691  
neos-911880_presolved          209897    31023    31023  -174.518  
neos-1616732_presolved        1017810   945072   945072  -165.352  
p80x400b_presolved             176813   141684   141684  -136.479  
milo-v12-6-r2-40-1_presolved    38939    50022    50022   -82.368  
rococoC10-001000_presolved      27089    26023    26023   -78.567  
prod2_presolved                118808    77122    77122   -67.325  
icir97_tension_presolved        52342    28515    28515   -57.402  
neos-831188_presolved            4292     4875     4875   -39.610  
neos-911970_presolved           64521    21007    21007   -23.639

In [137]:
### DEBUG DEBUG DEBUG
# inst = 'cost266-UUE_presolved'
# hawea instance
inst = 'neos-3592146-hawea_presolved'
if inst in all_bb_results_df.index:
    display(all_bb_results_df.loc[inst])

            Rows           765
            Cols          4205
# cuts      V                8
Time (s)    Gur1      1405.455
            Gur7      1116.399
            V         1311.339
            Total     1327.109
            V7        1311.339
            Total7    1327.109
Nodes (\#)  Gur1         24794
            Gur7         24794
            V            26137
            V7           24794
Name: neos-3592146-hawea_presolved, dtype: object

In [138]:
# Format Table 13: "best" time/nodes results
TABLE13 = all_bb_results_df.copy(deep=True)

# Drop all rows in which (time_col_header, 'Gur7') is ''
TABLE13 = TABLE13[TABLE13[(time_col_header, 'Gur7')] != '']

# TABLE13[('','V7-Gur7')]
# Change column (time_col_header, V7) to be float valued
#TABLE13[(time_col_header, 'V7')] = TABLE13[(time_col_header, 'V7')].apply(float_format, num_digits=4)
TABLE13 = TABLE13.astype({(time_col_header, 'V7'): float})
TABLE13 = TABLE13.astype({(time_col_header, 'Gur7'): float})

# Add new column for difference between V7 and Gur7
TABLE13[('Time (s)','V7 - Gur7')] = TABLE13[('Time (s)','V7')] - TABLE13[('Time (s)','Gur7')]

# Sort by V7 - Gur7
TABLE13.sort_values(by=[(time_col_header,'V7 - Gur7')], inplace=True)

# Rename summary rows to reflect the set
rename_metrics_all = {metric : metric + ' (All)' for metric in bb_metrics}
TABLE13.rename(rename_metrics_all, inplace=True)

# Add summary rows from 6 trees set
summary_metrics_6trees = all6_bb_results_df.tail(3).copy(deep=True)
rename_metrics_6trees = {metric : metric + ' (6 trees)' for metric in bb_metrics}
summary_metrics_6trees.rename(rename_metrics_6trees, inplace=True)

TABLE13 = pd.concat([TABLE13, summary_metrics_6trees])

# Drop rows, cols, (time,V7)
TABLE13.drop([('','Rows'),('','Cols'),(node_col_header,map_cols_to_short_time[mintime_col])], axis=1, inplace=True)

# Set wins row to be integer valued
TABLE13.loc['Wins1 (All)'] = TABLE13.loc['Wins1 (All)'].apply(int_format)
# TABLE13.loc['Wins7 (All)'] = TABLE13.loc['Wins7 (All)'].apply(int_format)
# TABLE13.loc['Wins1 (6 trees)'] = TABLE13.loc['Wins1 (6 trees)'].apply(int_format)
# TABLE13.loc['Wins7 (6 trees)'] = TABLE13.loc['Wins7 (6 trees)'].apply(int_format)
# TABLE13.iloc[len(TABLE13)-1] = TABLE13.iloc[len(TABLE13)-1].apply(int_format)

# Move instance names into a column
TABLE13.reset_index(inplace=True, col_level=1)

# Store indices of rows of 6-tree instances
six_trees_instances = list(all6_instances_dict.keys())
# mask = TABLE13[('','Instance')].isin(six_trees_instances)
# six_trees_indices = TABLE13.loc[mask, :].index.tolist()

# Remove presolved from name and escape
TABLE13[('',"Instance")] = TABLE13[('',"Instance")].apply(remove_presolved_from_name)
TABLE13[('',"Instance")] = TABLE13[('',"Instance")].apply(tex_escape)

# If we are not using the automatic tex-escaper, we need to do it ourselves
for col in TABLE13.columns:
    # curr_col = '{' + tex_escape(col) + '}'
    curr_col = tex_escape(str(col))
    TABLE13.rename({col: curr_col}, inplace=True, axis=1)

# Finally, apply the desired style
    # format(formatter = int_format).\
table13_str = TABLE13.style.\
    hide(axis=0).\
    to_latex(
        column_format="""
	@{}l % instance
	*{1}{S[table-format=4.0,table-auto-round,table-number-alignment=center]} % # cuts
	*{2}{S[table-format=4.2,table-auto-round]} % Gur1, Gur7
	*{2}{H} % V, Total
	*{2}{S[table-format=4.2,table-auto-round]} % V7, Total7
	*{3}{S[table-format=8.0,table-auto-round,table-number-alignment=center]} % Nodes
	@{}
        """,
        hrules = True,
        sparse_index = True,
        multirow_align = "c",
        siunitx=True,
        convert_css = True,
        environment = "table",
        position_float = "centering",
        label = "app:tab:bb",
        caption = """
            Time (in seconds) and number nodes taken to solve each instance.
            The table is sorted by column 4 (``V'' under ``Time (s)'').
            ``Gur1'' indicates \Gurobi{} run with one random seed.
            ``Gur7'' indicates the minimum from seven runs of \Gurobi{} with different random seeds.
        """,
        )

# Adjustbox environment sets width to pagewidth
# table13_str = add_adjustbox_environment(table13_str)

# Set default siunitx options for this table
table13_str = add_sisetup(table13_str)

# Add a midrule between the instances and 3 summary rows; the "6" is hand-coded but can be automated
table13_str = add_midrule(table13_str, -6)
table13_str = add_midrule(table13_str, -10)

# Add color to six tree instances
splitlines = table13_str.splitlines()
for i in range(len(splitlines)):
    line = splitlines[i]
    curr_line = line.split('&')
    if len(curr_line) > 0 and curr_line[0].strip()+'_presolved' in six_trees_instances:
        splitlines[i] = '\\rowcolor{lightgray!30} ' + line
table13_str = '\n'.join(splitlines).replace('NaN', '')

print(table13_str)


{
\sisetup{
    table-alignment-mode = format,
    table-number-alignment = center,
    table-format = 2.2,
}
\begin{table}
\centering
\caption{
            Time (in seconds) and number nodes taken to solve each instance.
            The table is sorted by column 4 (``V'' under ``Time (s)'').
            ``Gur1'' indicates \Gurobi{} run with one random seed.
            ``Gur7'' indicates the minimum from seven runs of \Gurobi{} with different random seeds.
        }
\label{app:tab:bb}
\begin{tabular}{@{}l % instance
	*{1}{S[table-format=4.0,table-auto-round,table-number-alignment=center]} % # cuts
	*{2}{S[table-format=4.2,table-auto-round]} % Gur1, Gur7
	*{2}{H} % V, Total
	*{2}{S[table-format=4.2,table-auto-round]} % V7, Total7
	*{3}{S[table-format=8.0,table-auto-round,table-number-alignment=center]} % Nodes
	@{}}
\toprule
{} & {# cuts} & \multicolumn{6}{r}{Time (s)} & \multicolumn{3}{r}{Nodes (\#)} & {Time (s)} \\
{Instance} & {V} & {Gur1} & {Gur7} & {V} & {Total} & {V7} & {Total7}